
# Data Mining for Space Habitats
## *Analyzing ISS Environmental Telemetry for Safer Off-World Living*
**Kenny Jenkins**  
**Department of Computer Science**  
**University of Colorado Boulder**  

### Section 1: Setup and Imports
#### Purpose
Initialize libraries, plotting, and configuration for a multi-mission, multi-modal pipeline that supports exploratory analysis, forecasting, anomaly detection, and biological linkage.
#### What is implemented
1. Core stack: NumPy, Pandas, SciPy, statsmodels, scikit-learn, TensorFlow/Keras.
2. Visualization: Matplotlib, Seaborn, optional Plotly.
3. Time-series utilities: STL, FFT, SARIMAX, VAR, Kalman Filter.
4. Streamlit-ready configuration guard (runs cleanly in Jupyter or Streamlit).
5. Central `Settings` dataclass (missions, paths, sampling, figure sizes, output dirs).
6. Reproducibility: unified random seeds (Python/NumPy/TensorFlow) and TF deterministic ops.
7. Clean logging and consistent Pandas/Matplotlib display options.
#### Future Work
- Integrate behavioral/event context (e.g., docking/EVA windows) into configuration.


In [1]:
# ==== Section 1: Setup & Imports ====

import os, sys, random, warnings, io, json, time
from dataclasses import dataclass, field
from datetime import datetime, timedelta
from pathlib import Path
from typing import Dict, Tuple, List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Optional visual QA
import plotly.express as px
import plotly.graph_objects as go

# Timeseries & stats
from scipy.fft import fft, fftfreq
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from pykalman import KalmanFilter

# Deep learning
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Jupyter-friendly progress
from tqdm.notebook import tqdm
from IPython.display import display

# ---- Streamlit guard (keep commented in Jupyter) ----
# try:
#     import streamlit as st
#     if os.environ.get("STREAMLIT_SERVER_RUNNING") == "1":
#         st.set_page_config(page_title="ISS Environmental Telemetry & Omics Explorer", layout="wide")
#         st.title("ISS Environmental Telemetry & Omics Explorer")
#         st.caption("Rodent Research: RR-1, RR-3, RR-6, RR-9, RR-12, RR-19 · CO₂, Temp, RH, Pressure, Radiation")
# except Exception:
#     pass

# ---- Global Settings object ----
@dataclass
class Settings:
    missions: List[str] = field(default_factory=lambda: [
        "RR-1","RR-3","RR-4","RR-5","RR-6","RR-8","RR-9","RR-12","RR-17","RR-19"
    ])
    sampling: str = "1min"
    seed: int = 42
    fig_size: Tuple[int,int] = (12, 6)

    # Data roots (prefer home-level; keep repo fallbacks)
    data_roots: List[Path] = field(default_factory=lambda: [
        Path(os.environ.get("SPACEHAB_DATA", "~/data/osdr_eda")).expanduser(),
        Path.cwd() / "data" / "osdr_eda",
        Path.cwd() / "data"
    ])
    omics_roots: List[Path] = field(default_factory=lambda: [
        Path(os.environ.get("SPACEHAB_OMICS", "~/data/genelab")).expanduser(),
        Path.cwd() / "data" / "genelab",
        Path.cwd() / "data" / "omics"
    ])

    # Outputs live inside repo (avoid macOS protected folders)
    outputs_root: Path = field(default_factory=lambda: Path.cwd() / "outputs")
    preprocessed_dir: Path = field(default_factory=lambda: Path.cwd() / "outputs" / "preprocessed")
    pattern_dir: Path = field(default_factory=lambda: Path.cwd() / "outputs" / "pattern_analysis")
    relationships_dir: Path = field(default_factory=lambda: Path.cwd() / "outputs" / "relationships")
    anomalies_dir: Path = field(default_factory=lambda: Path.cwd() / "outputs" / "anomaly_forecast")

S = Settings()

# Create output dirs
for d in [S.outputs_root, S.preprocessed_dir, S.pattern_dir, S.pattern_dir / "radiation",
          S.relationships_dir, S.anomalies_dir]:
    d.mkdir(parents=True, exist_ok=True)

# ---- Reproducibility ----
os.environ["PYTHONHASHSEED"] = str(S.seed)
os.environ["TF_DETERMINISTIC_OPS"] = "1"
random.seed(S.seed)
np.random.seed(S.seed)
tf.random.set_seed(S.seed)

# ---- Plotting / Pandas display ----
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = S.fig_size
plt.rcParams["axes.titlesize"] = 14
plt.rcParams["axes.labelsize"] = 12

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 200)
pd.set_option("display.float_format", "{:.2f}".format)

# ---- Clean warnings ----
warnings.filterwarnings("ignore")

print("Environment setup complete.")
print("TensorFlow:", tf.__version__)
print("Data roots:", [str(p) for p in S.data_roots])
print("Omics roots:", [str(p) for p in S.omics_roots])
print("Outputs root:", str(S.outputs_root))


Environment setup complete.
TensorFlow: 2.16.2
Data roots: ['/Users/kennethjenkins/data/osdr_eda', '/Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/data/osdr_eda', '/Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/data']
Omics roots: ['/Users/kennethjenkins/data/genelab', '/Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/data/genelab', '/Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/data/omics']
Outputs root: /Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/outputs


### Section 2: Load and Preview ISS Environmental Telemetry
#### Purpose
Load Rodent Research telemetry/radiation datasets and GeneLab omics tables for later linkage; standardize and validate inputs for cross-mission analysis.
#### What is implemented
1. Flexible file discovery across home and repo roots (nested and flat layouts).
2. Light schema checks and datetime parsing validation.
3. Mission-level ingestion for summary, minute-level telemetry, and radiation logs.
4. Omics ingestion (GLDS-98/99/104: differential expression, normalized counts, PCA/SampleTable).
5. Availability matrix by mission (summary/telemetry/radiation) and unified `rr_data` handle.
#### Future Work
- Loader for event context (docking/EVA/crew) for downstream overlays.
- Data availability matrix by mission and variable (dashboard display).
- Placeholders/hooks for behavioral video and phenotypic data.

In [2]:
# ==== Section 2: Load & Preview ISS Environmental + Omics ====
from pathlib import Path
from typing import List, Dict
import pandas as pd
import numpy as np

try:
    from IPython.display import display
except Exception:
    display = print

# --- Light validation ---
def require_cols(df: pd.DataFrame, cols: List[str], name: str) -> None:
    miss = [c for c in cols if c not in df.columns]
    if miss:
        raise ValueError(f"[SCHEMA] {name} missing columns: {miss}")

def assert_time_parse_ok(dt_series: pd.Series, name: str) -> None:
    if dt_series.notna().sum() == 0:
        raise ValueError(f"[TIME] Failed to parse datetime for {name}")

# --- File helpers ---
def find_file(rel_path: str, roots: List[Path]) -> Path | None:
    for root in roots:
        p = (root / rel_path).expanduser()
        if p.exists():
            return p
    return None

def try_candidates(candidates: List[str], roots: List[Path]) -> Path | None:
    for rel in candidates:
        p = find_file(rel, roots)
        if p:
            return p
    return None

def safe_read_csv(path: Path, **kwargs) -> pd.DataFrame | None:
    try:
        return pd.read_csv(path, **kwargs)
    except Exception as e:
        print(f"[READ FAIL] {path}: {e}")
        return None

# --- Candidate layouts (nested + flat) ---
def env_candidates(rr: str, kind: str) -> List[str]:
    suffix = {"summary": "_EDA_Summary_table.csv",
              "telemetry": "_EDA_Telemetry_data.csv",
              "radiation": "_EDA_Radiation_data.csv"}[kind]
    fname = f"{rr}{suffix}"
    return [f"EDA Rodent Research/{fname}", fname]

def omics_candidates(glds: str, kind: str) -> List[str]:
    fname = {
        "diff": f"{glds}_rna_seq_differential_expression.csv",
        "norm": f"{glds}_rna_seq_Normalized_Counts.csv",
        "meta": f"{glds}_rna_seq_visualization_PCA_table.csv" if glds == "GLDS-104"
                else f"{glds}_rna_seq_SampleTable.csv",
    }[kind]
    return [f"RR-1/{glds}/{fname}", fname]

telemetry_expected = ["Temp_degC_ISS", "RH_percent_ISS", "CO2_ppm_ISS"]
radiation_expected = ["GCR_Dose_mGy_d", "SAA_Dose_mGy_d", "Total_Dose_mGy_d", "Accumulated_Dose_mGy_d"]

summary_data: Dict[str, pd.DataFrame] = {}
telemetry_data: Dict[str, pd.DataFrame] = {}
radiation_data: Dict[str, pd.DataFrame] = {}
ok_summary, ok_telemetry, ok_radiation = set(), set(), set()

# --- Load EDA per mission ---
for rr in S.missions:
    # summary
    p = try_candidates(env_candidates(rr, "summary"), S.data_roots)
    if p:
        df = safe_read_csv(p)
        if df is not None and not df.empty:
            summary_data[rr] = df
            ok_summary.add(rr)
    else:
        print(f"[MISS] {rr} summary")

    # telemetry
    p = try_candidates(env_candidates(rr, "telemetry"), S.data_roots)
    if p:
        df = safe_read_csv(p)
        if df is not None and not df.empty:
            if "Controller_Time_GMT" in df.columns:
                dt = pd.to_datetime(df["Controller_Time_GMT"], errors="coerce", utc=True)
                assert_time_parse_ok(dt, f"{rr} telemetry time")
                df["Controller_Time_GMT"] = dt
            # soft schema check
            missing = [c for c in telemetry_expected if c not in df.columns]
            if missing:
                print(f"[WARN] {rr} telemetry missing: {missing}")
            telemetry_data[rr] = df
            ok_telemetry.add(rr)
    else:
        print(f"[MISS] {rr} telemetry")

    # radiation
    p = try_candidates(env_candidates(rr, "radiation"), S.data_roots)
    if p:
        df = safe_read_csv(p)
        if df is not None and not df.empty:
            if "Date" in df.columns:
                dt = pd.to_datetime(df["Date"], errors="coerce", utc=True)
                assert_time_parse_ok(dt, f"{rr} radiation time")
                df["Date"] = dt
            missing = [c for c in radiation_expected if c not in df.columns]
            if missing:
                print(f"[WARN] {rr} radiation missing: {missing}")
            radiation_data[rr] = df
            ok_radiation.add(rr)
    else:
        print(f"[MISS] {rr} radiation")

    print(f"Checked {rr}")

# --- Omics (RR-1-focused tables; extend later) ---
omics_data: Dict[str, Dict[str, pd.DataFrame]] = {}
for glds in ["GLDS-98", "GLDS-99", "GLDS-104"]:
    omics_data[glds] = {}
    for key in ["diff", "norm", "meta"]:
        p = try_candidates(omics_candidates(glds, key), S.omics_roots)
        if p:
            omics_data[glds][key] = safe_read_csv(p)
        else:
            print(f"[MISS] OMICS {glds}:{key}")

# --- Availability table ---
availability_rows = []
for rr in S.missions:
    availability_rows.append({
        "RR_Mission": rr,
        "Summary Available": rr in ok_summary,
        "Telemetry Available": rr in ok_telemetry,
        "Radiation Available": rr in ok_radiation
    })
availability_df = pd.DataFrame(availability_rows)
print("\nDataset Availability by Mission:")
display(availability_df)

# --- Unified handle for downstream sections ---
rr_data: Dict[str, Dict[str, pd.DataFrame | None]] = {
    rr: {
        "summary": summary_data.get(rr),
        "telemetry": telemetry_data.get(rr),
        "radiation": radiation_data.get(rr),
    }
    for rr in S.missions
}

# quick counts
print("Telemetry files loaded:", len(telemetry_data))
print("Radiation files loaded:", len(radiation_data))


Checked RR-1
Checked RR-3
Checked RR-4
Checked RR-5
Checked RR-6
Checked RR-8
Checked RR-9
Checked RR-12
Checked RR-17
Checked RR-19

Dataset Availability by Mission:


,RR_Mission,Summary Available,Telemetry Available,Radiation Available
0,RR-1,True,True,True
1,RR-3,True,True,True
2,RR-4,True,False,True
3,RR-5,True,True,False
4,RR-6,True,True,True
5,RR-8,True,False,True
6,RR-9,True,True,True
7,RR-12,True,True,True
8,RR-17,True,False,True
9,RR-19,True,True,True


Telemetry files loaded: 7
Radiation files loaded: 9


### Section 3: Data Preprocessing
#### Purpose
Create clean, aligned, feature-enriched one-minute time series for telemetry and radiation.
#### What is implemented
1. One-minute resampling with conservative interpolation of short gaps (≤5 min).
2. Rolling features (mean/std at 5/30/180 min) per variable.
3. Orbital day/night proxy from CO₂; crew-awake heuristic (UTC 06–22).
4. Z-scores for each native telemetry/radiation column.
5. Quality flags per column (orig/interp/missing) and CSV exports to `outputs/preprocessed/`.
#### Future Work
- Replace orbital proxy with explicit lighting if available from EDA.
- Persist a unified mission time index to simplify joins with omics and events.

In [3]:
# ==== Section 3: Data Preprocessing ====

def minute_index(df: pd.DataFrame, time_col: str) -> pd.DatetimeIndex:
    idx = pd.to_datetime(df[time_col], errors="coerce", utc=True).dropna()
    return pd.date_range(idx.min().floor("T"), idx.max().ceil("T"), freq=S.sampling)

def build_quality_flags(original: pd.Series, resampled: pd.Series, interp: pd.Series) -> pd.Series:
    # original points that landed exactly on the minute bins
    flag = pd.Series(index=resampled.index, data="missing", dtype="object")
    # mark where we have a value after resample before interpolate
    flag.loc[resampled.index[resampled.notna()]] = "orig"
    # after interpolation, any previously missing-but-now-present becomes 'interp'
    new_filled = interp.notna() & resampled.isna()
    flag.loc[new_filled.index[new_filled]] = "interp"
    return flag

cleaned_telemetry, cleaned_radiation = {}, {}

for rr in availability_df.query("`Telemetry Available` and `Radiation Available`")["RR_Mission"]:
    tel = rr_data[rr]["telemetry"].copy()
    rad = rr_data[rr]["radiation"].copy()

    # --- TELEMETRY ---
    require_cols(tel, ["Controller_Time_GMT"], f"{rr} telemetry")
    tel = tel.set_index(pd.to_datetime(tel["Controller_Time_GMT"], utc=True)).sort_index()
    # keep numeric cols only
    tel_num = tel.select_dtypes(include=[np.number])

    # resample
    tel_res = tel_num.resample(S.sampling).mean()
    tel_pre = tel_res.copy()
    # interpolate short gaps only (<=5 minutes)
    tel_int = tel_res.interpolate(limit=5)

    # rolling features
    wins = [5, 30, 180]
    for col in tel_num.columns:
        for w in wins:
            tel_int[f"{col}_mean_{w}min"] = tel_int[col].rolling(w, min_periods=1).mean()
            tel_int[f"{col}_std_{w}min"]  = tel_int[col].rolling(w, min_periods=1).std()

    # day/night proxy using CO2
    if "CO2_ppm_ISS" in tel_int.columns:
        thr = tel_int["CO2_ppm_ISS"].median()
        tel_int["Orbital_Day"] = (tel_int["CO2_ppm_ISS"] > thr).astype(int)

    # crew awake heuristic (UTC 06–22)
    tel_int["Hour"] = tel_int.index.hour
    tel_int["Crew_Awake"] = ((tel_int["Hour"] >= 6) & (tel_int["Hour"] < 22)).astype(int)
    tel_int.drop(columns="Hour", inplace=True)

    # z-scores for native telemetry columns
    for col in tel_num.columns:
        std = tel_int[col].std()
        if pd.notna(std) and std > 0:
            tel_int[f"{col}_zscore"] = (tel_int[col] - tel_int[col].mean()) / std

    # quality flags per column (orig/interp/missing)
    for col in tel_num.columns:
        flags = build_quality_flags(
            original=tel_num[col],
            resampled=tel_pre[col],
            interp=tel_int[col]
        )
        tel_int[f"{col}_qflag"] = flags.values

    cleaned_telemetry[rr] = tel_int
    tel_out = S.preprocessed_dir / f"{rr}_cleaned_telemetry.csv"
    tel_int.to_csv(tel_out)
    print(f"[✓] Telemetry preprocessed → {tel_out}")

    # --- RADIATION ---
    require_cols(rad, ["Date"], f"{rr} radiation")
    rad = rad.set_index(pd.to_datetime(rad["Date"], utc=True)).sort_index()
    rad_num = rad.select_dtypes(include=[np.number])

    rad_res = rad_num.resample(S.sampling).ffill()
    rad_pre = rad_res.copy()
    rad_int = rad_res.interpolate(limit=5)

    for col in rad_num.columns:
        for w in wins:
            rad_int[f"{col}_mean_{w}min"] = rad_int[col].rolling(w, min_periods=1).mean()
            rad_int[f"{col}_std_{w}min"]  = rad_int[col].rolling(w, min_periods=1).std()

    # z-scores
    for col in rad_num.columns:
        std = rad_int[col].std()
        if pd.notna(std) and std > 0:
            rad_int[f"{col}_zscore"] = (rad_int[col] - rad_int[col].mean()) / std

    # quality flags
    for col in rad_num.columns:
        flags = build_quality_flags(
            original=rad_num[col],
            resampled=rad_pre[col],
            interp=rad_int[col]
        )
        rad_int[f"{col}_qflag"] = flags.values

    cleaned_radiation[rr] = rad_int
    rad_out = S.preprocessed_dir / f"{rr}_cleaned_radiation.csv"
    rad_int.to_csv(rad_out)
    print(f"[✓] Radiation preprocessed → {rad_out}")

preprocessed_telemetry = cleaned_telemetry


[✓] Telemetry preprocessed → /Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/outputs/preprocessed/RR-1_cleaned_telemetry.csv
[✓] Radiation preprocessed → /Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/outputs/preprocessed/RR-1_cleaned_radiation.csv
[✓] Telemetry preprocessed → /Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/outputs/preprocessed/RR-3_cleaned_telemetry.csv
[✓] Radiation preprocessed → /Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/outputs/preprocessed/RR-3_cleaned_radiation.csv
[✓] Telemetry preprocessed → /Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/outputs/preprocessed/RR-6_cleaned_telemetry.csv
[✓] Radiation preprocessed → /Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/outputs/preprocessed/RR-6_cleaned_radiation.csv
[✓] Telemetry preprocessed → /Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/outputs/preprocessed/RR-9_

In [4]:
print("Telemetry files:", len(list(S.preprocessed_dir.glob("*_cleaned_telemetry.csv"))))
print("Radiation files:", len(list(S.preprocessed_dir.glob("*_cleaned_radiation.csv"))))


Telemetry files: 6
Radiation files: 6


### Section 4: Pattern Extraction
#### Purpose
Characterize seasonal and frequency structure to establish baseline rhythms and mission-specific signatures at 1-minute resolution.
#### What is implemented
1. STL decomposition (robust) for telemetry and radiation; raw/STL plots.
2. FFT spectrum with flat-signal safeguards; Welch PSD robustness check.
3. Per-series diagnostics (range, N, variance) and saved STL seasonal indices.
4. Cross-mission `seasonality_summary.csv` (seasonality strength, robust amplitude, dominant period).
#### Future Work
- Compare STL seasonality with SARIMAX seasonal terms for consistency.
- Automated report that ranks variables by seasonal strength per mission.

In [5]:
# ==== Section 4: Pattern Extraction (detailed tqdm + progress log) ====

import os, sys, time
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import welch
from statsmodels.tsa.seasonal import STL

# --- CPU threading knobs (effective if set before NumPy load; harmless otherwise) ---
os.environ.setdefault("OMP_NUM_THREADS", str(max(1, os.cpu_count() // 2)))
os.environ.setdefault("OPENBLAS_NUM_THREADS", os.environ["OMP_NUM_THREADS"])
os.environ.setdefault("MKL_NUM_THREADS", os.environ["OMP_NUM_THREADS"])
os.environ.setdefault("NUMEXPR_NUM_THREADS", os.environ["OMP_NUM_THREADS"])

# --- Robust tqdm (no ipywidgets dependency) ---
try:
    from tqdm import tqdm
    def _tqdm(iterable=None, **kwargs):
        # Text-mode bar with dynamic width and responsive refresh
        return tqdm(iterable, dynamic_ncols=True, miniters=1, smoothing=0.05, **kwargs)
    def _tqdm_write(msg):
        print(msg, file=sys.stdout, flush=True)
except Exception:
    def _tqdm(it, **kwargs): return it
    def _tqdm_write(msg): print(msg)

# ---- Config ----
telemetry_cols = ["Temp_degC_ISS", "RH_percent_ISS", "CO2_ppm_ISS"]
radiation_cols = ["GCR_Dose_mGy_d", "SAA_Dose_mGy_d", "Total_Dose_mGy_d", "Accumulated_Dose_mGy_d"]

stl_period = 1440          # 24h @ 1-min sampling
min_required_points = stl_period * 2
fft_min_std = 1e-6
save_seasonal_indices = True
SKIP_IF_DONE = False       # set True to skip variables with all artifacts already written

# paths
(S.pattern_dir / "radiation").mkdir(parents=True, exist_ok=True)
seasonal_dir = S.pattern_dir / "seasonal_indices"
seasonal_dir.mkdir(parents=True, exist_ok=True)
progress_csv = S.pattern_dir / "pattern_progress.csv"

# --- IO helpers (prefer in-memory from Section 3; fallback to disk) ---
def _load_preprocessed(rr: str, kind: str) -> pd.DataFrame | None:
    try:
        if kind == "telemetry":
            if 'preprocessed_telemetry' in globals() and rr in preprocessed_telemetry:
                return preprocessed_telemetry[rr].copy()
            p = S.preprocessed_dir / f"{rr}_cleaned_telemetry.csv"
        else:
            if 'cleaned_radiation' in globals() and rr in cleaned_radiation:
                return cleaned_radiation[rr].copy()
            p = S.preprocessed_dir / f"{rr}_cleaned_radiation.csv"
        if p.exists():
            df = pd.read_csv(p, index_col=0, parse_dates=True).sort_index()
            return df
    except Exception as e:
        _tqdm_write(f"[LOAD FAIL] {rr} {kind}: {e}")
    return None

# --- Metrics helpers ---
def _seasonality_strength(seasonal: pd.Series, resid: pd.Series) -> float:
    x = pd.Series(seasonal, dtype=float)
    r = pd.Series(resid, dtype=float)
    denom = np.nanvar(x + r)
    if not np.isfinite(denom) or denom <= 0: return 0.0
    val = 1.0 - (np.nanvar(r) / denom)
    return float(np.clip(val, 0.0, 1.0))

def _dominant_period_minutes_fft(y: np.ndarray, fs_hz: float) -> tuple[float, float]:
    y = np.asarray(y, dtype=float)
    y = y - np.nanmean(y)
    if np.nanstd(y) < fft_min_std: return (0.0, np.inf)
    N = len(y)
    freqs = np.fft.rfftfreq(N, d=1.0/fs_hz)  # Hz
    spec = np.abs(np.fft.rfft(y))
    if len(freqs) < 2: return (0.0, np.inf)
    spec[0] = 0.0
    k = int(np.nanargmax(spec))
    f = float(freqs[k])
    if f <= 0: return (0.0, np.inf)
    period_min = (1.0 / f) / 60.0
    return (f, period_min)

def _safe_fft_plot(ts: pd.Series, title: str, out_path: Path, fs_hz: float):
    y = ts.to_numpy(dtype=float)
    y = y - np.nanmean(y)
    if np.nanstd(y) < fft_min_std:
        _tqdm_write(f"[SKIP FFT] {title} too flat.")
        return
    N = len(y)
    freqs = np.fft.rfftfreq(N, d=1.0/fs_hz)
    spec = np.abs(np.fft.rfft(y))
    plt.figure(figsize=(12, 5))
    plt.plot(freqs, spec, color="black")
    plt.title(f"{title} — Frequency Spectrum (rFFT)")
    plt.xlabel("Frequency (Hz)"); plt.ylabel("Amplitude")
    plt.tight_layout(); plt.savefig(out_path); plt.close()

def _welch_plot(ts: pd.Series, title: str, out_path: Path, fs_hz: float):
    y = ts.to_numpy(dtype=float)
    y = y - np.nanmean(y)
    if np.nanstd(y) < fft_min_std:
        _tqdm_write(f"[SKIP WELCH] {title} too flat.")
        return
    nperseg = min(len(y), 4096)
    if nperseg < 256:
        _tqdm_write(f"[SKIP WELCH] {title} too short (n={len(y)}).")
        return
    f, Pxx = welch(y, fs=fs_hz, nperseg=nperseg)
    plt.figure(figsize=(12, 5))
    plt.semilogy(f, Pxx, color="black")
    plt.title(f"{title} — Welch PSD")
    plt.xlabel("Frequency (Hz)"); plt.ylabel("Power Spectral Density")
    plt.tight_layout(); plt.savefig(out_path); plt.close()

def _stl_decompose(ts: pd.Series, period: int):
    return STL(ts, period=period, robust=True).fit()

# --- Checkpointing (optional) ---
def _artifacts_exist(rr: str, kind: str, col: str) -> bool:
    if kind == "telemetry":
        base = S.pattern_dir
    else:
        base = S.pattern_dir / "radiation"
    need = [
        base / f"{col}_{rr}_Raw.png",
        base / f"{col}_{rr}_STL.png",
        base / f"{col}_{rr}_FFT.png",
        base / f"{col}_{rr}_WELCH.png",
    ]
    if save_seasonal_indices:
        need.append(seasonal_dir / f"{rr}_{col}_seasonal.csv")
    return all(p.exists() for p in need)

# ----- Run analysis -----
summary_rows = []
fs_hz = 1.0 / 60.0  # one sample every 60 seconds

missions_to_run = []
for rr in S.missions:
    tel_ok = (S.preprocessed_dir / f"{rr}_cleaned_telemetry.csv").exists() or \
             ('preprocessed_telemetry' in globals() and rr in preprocessed_telemetry)
    rad_ok = (S.preprocessed_dir / f"{rr}_cleaned_radiation.csv").exists() or \
             ('cleaned_radiation' in globals() and rr in cleaned_radiation)
    if tel_ok or rad_ok:
        missions_to_run.append(rr)

_tqdm_write(f"Pattern extraction for missions: {missions_to_run}")

mission_bar = _tqdm(missions_to_run, desc="Missions", total=len(missions_to_run))
for rr in mission_bar:
    # prepare lists of variables present
    df_tel = _load_preprocessed(rr, "telemetry")
    df_rad = _load_preprocessed(rr, "radiation")
    tel_vars = [c for c in telemetry_cols if (df_tel is not None and c in df_tel.columns)]
    rad_vars = [c for c in radiation_cols  if (df_rad is not None and c in df_rad.columns)]
    total_vars = len(tel_vars) + len(rad_vars)

    var_bar = _tqdm(range(total_vars), desc=f"{rr} vars", total=total_vars, position=1, leave=False)
    processed = 0

    # ---- Telemetry ----
    if df_tel is not None and len(df_tel) >= min_required_points:
        for col in tel_vars:
            t0 = time.time()
            if SKIP_IF_DONE and _artifacts_exist(rr, "telemetry", col):
                var_bar.set_postfix_str(f"{col}: skip (exists)")
                var_bar.update(1); processed += 1
                continue
            try:
                ts = df_tel[col].dropna().resample("1min").mean().interpolate(limit=5)
                ts = ts[np.isfinite(ts)]
                if len(ts) < min_required_points or ts.std() < fft_min_std:
                    var_bar.set_postfix_str(f"{col}: flat/short")
                    var_bar.update(1); processed += 1
                    continue

                # Raw
                plt.figure(figsize=(12,4))
                plt.plot(ts.index, ts.values, color="black")
                plt.title(f"{col} Raw Signal ({rr})")
                plt.xlabel("Time"); plt.ylabel(col)
                plt.tight_layout(); plt.savefig(S.pattern_dir / f"{col}_{rr}_Raw.png"); plt.close()

                # STL
                res = _stl_decompose(ts, period=stl_period)
                fig, axes = plt.subplots(4, 1, figsize=(14, 10), sharex=True)
                axes[0].plot(ts.index, res.observed.values, color='black'); axes[0].set_title(f"{col} — Original ({rr})")
                axes[1].plot(res.trend.index, res.trend.values,   color='blue');  axes[1].set_title("Trend")
                axes[2].plot(res.seasonal.index, res.seasonal.values, color='green'); axes[2].set_title("Seasonal")
                axes[3].plot(res.resid.index, res.resid.values,   color='red');   axes[3].set_title("Residual")
                plt.tight_layout(); plt.savefig(S.pattern_dir / f"{col}_{rr}_STL.png"); plt.close()

                # Save seasonal index
                if save_seasonal_indices:
                    pd.DataFrame({"timestamp": res.seasonal.index, "seasonal": res.seasonal.values}) \
                      .to_csv(seasonal_dir / f"{rr}_{col}_seasonal.csv", index=False)

                # FFT + Welch
                _safe_fft_plot(ts, f"{col} ({rr})", S.pattern_dir / f"{col}_{rr}_FFT.png", fs_hz)
                _welch_plot(ts, f"{col} ({rr})", S.pattern_dir / f"{col}_{rr}_WELCH.png", fs_hz)
                dom_f_hz, dom_period_min = _dominant_period_minutes_fft(ts.values, fs_hz)

                strength = _seasonality_strength(res.seasonal, res.resid)
                amp_robust = float(np.nanpercentile(res.seasonal, 95) - np.nanpercentile(res.seasonal, 5))
                summary_rows.append({
                    "Mission": rr, "Type": "telemetry", "Variable": col,
                    "N": len(ts), "Std": float(ts.std()),
                    "Seasonality_Strength": strength,
                    "Seasonal_Amplitude_P95_P05": amp_robust,
                    "Dominant_Freq_Hz": dom_f_hz,
                    "Dominant_Period_min": dom_period_min,
                    "Seconds": round(time.time() - t0, 2)
                })

                var_bar.set_postfix_str(f"{col}: ok · {len(ts)} pts · {time.time()-t0:.1f}s")
            except Exception as e:
                var_bar.set_postfix_str(f"{col}: ERROR")
                _tqdm_write(f"[ERROR] {rr} {col}: {e}")
            finally:
                var_bar.update(1); processed += 1

    # ---- Radiation ----
    if df_rad is not None and len(df_rad) >= min_required_points:
        for col in rad_vars:
            t0 = time.time()
            if SKIP_IF_DONE and _artifacts_exist(rr, "radiation", col):
                var_bar.set_postfix_str(f"{col}: skip (exists)")
                var_bar.update(1); processed += 1
                continue
            try:
                ts = df_rad[col].dropna().resample("1min").interpolate(limit=5)
                ts = ts[np.isfinite(ts)]
                if len(ts) < min_required_points or ts.std() < fft_min_std:
                    var_bar.set_postfix_str(f"{col}: flat/short")
                    var_bar.update(1); processed += 1
                    continue

                # Raw
                plt.figure(figsize=(12,4))
                plt.plot(ts.index, ts.values, color="black")
                plt.title(f"{col} Raw Signal ({rr})")
                plt.xlabel("Time"); plt.ylabel(col)
                plt.tight_layout(); plt.savefig(S.pattern_dir / "radiation" / f"{col}_{rr}_Raw.png"); plt.close()

                # STL
                res = _stl_decompose(ts, period=stl_period)
                fig, axes = plt.subplots(4, 1, figsize=(14, 10), sharex=True)
                axes[0].plot(ts.index, res.observed.values, color='black'); axes[0].set_title(f"{col} — Original ({rr})")
                axes[1].plot(res.trend.index, res.trend.values,   color='blue');  axes[1].set_title("Trend")
                axes[2].plot(res.seasonal.index, res.seasonal.values, color='green'); axes[2].set_title("Seasonal")
                axes[3].plot(res.resid.index, res.resid.values,   color='red');   axes[3].set_title("Residual")
                plt.tight_layout(); plt.savefig(S.pattern_dir / "radiation" / f"{col}_{rr}_STL.png"); plt.close()

                if save_seasonal_indices:
                    pd.DataFrame({"timestamp": res.seasonal.index, "seasonal": res.seasonal.values}) \
                      .to_csv(seasonal_dir / f"{rr}_{col}_seasonal.csv", index=False)

                _safe_fft_plot(ts, f"{col} ({rr})", S.pattern_dir / "radiation" / f"{col}_{rr}_FFT.png", fs_hz)
                _welch_plot(ts, f"{col} ({rr})", S.pattern_dir / "radiation" / f"{col}_{rr}_WELCH.png", fs_hz)
                dom_f_hz, dom_period_min = _dominant_period_minutes_fft(ts.values, fs_hz)

                strength = _seasonality_strength(res.seasonal, res.resid)
                amp_robust = float(np.nanpercentile(res.seasonal, 95) - np.nanpercentile(res.seasonal, 5))
                summary_rows.append({
                    "Mission": rr, "Type": "radiation", "Variable": col,
                    "N": len(ts), "Std": float(ts.std()),
                    "Seasonality_Strength": strength,
                    "Seasonal_Amplitude_P95_P05": amp_robust,
                    "Dominant_Freq_Hz": dom_f_hz,
                    "Dominant_Period_min": dom_period_min,
                    "Seconds": round(time.time() - t0, 2)
                })

                var_bar.set_postfix_str(f"{col}: ok · {len(ts)} pts · {time.time()-t0:.1f}s")
            except Exception as e:
                var_bar.set_postfix_str(f"{col}: ERROR")
                _tqdm_write(f"[ERROR] {rr} {col}: {e}")
            finally:
                var_bar.update(1); processed += 1

    var_bar.close()
    mission_bar.set_postfix_str(f"{processed}/{total_vars} vars")

# ---- Save cross-mission summary + append progress log ----
seasonality_summary = pd.DataFrame(summary_rows)
out_csv = S.pattern_dir / "seasonality_summary.csv"
seasonality_summary.to_csv(out_csv, index=False)
_tqdm_write(f"[✓] Wrote seasonality summary → {out_csv}")

# keep a lightweight cumulative progress log
try:
    mode = "a" if progress_csv.exists() else "w"
    header = not progress_csv.exists()
    seasonality_summary.assign(ts=pd.Timestamp.utcnow()).to_csv(progress_csv, index=False, mode=mode, header=header)
    _tqdm_write(f"[✓] Appended progress log → {progress_csv}")
except Exception as e:
    _tqdm_write(f"[LOG WARN] Could not append progress log: {e}")

# Optional: quick peek at strongest seasonality
try:
    from IPython.display import display
    display(seasonality_summary.sort_values(
        ["Seasonality_Strength", "Seasonal_Amplitude_P95_P05"], ascending=[False, False]
    ).head(20))
except Exception:
    pass


Pattern extraction for missions: ['RR-1', 'RR-3', 'RR-6', 'RR-9', 'RR-12', 'RR-19']


RR-1 vars:   0%|  | 0/7 [03:55<?, ?it/s, Temp_degC_ISS: ok · 60445 pts · 235.9s]
RR-1 vars:  14%|▏| 1/7 [03:55<23:35, 235.94s/it, Temp_degC_ISS: ok · 60445 pts ·
RR-1 vars:  14%|▏| 1/7 [08:05<23:35, 235.94s/it, RH_percent_ISS: ok · 60445 pts 
RR-1 vars:  29%|▎| 2/7 [08:05<20:14, 243.00s/it, RH_percent_ISS: ok · 60445 pts 
RR-1 vars:  29%|▎| 2/7 [11:54<20:14, 243.00s/it, CO2_ppm_ISS: ok · 60445 pts · 2
RR-1 vars:  43%|▍| 3/7 [11:54<15:52, 238.08s/it, CO2_ppm_ISS: ok · 60445 pts · 2
RR-1 vars:  43%|▍| 3/7 [15:21<15:52, 238.08s/it, GCR_Dose_mGy_d: ok · 54721 pts 
RR-1 vars:  57%|▌| 4/7 [15:21<11:29, 229.76s/it, GCR_Dose_mGy_d: ok · 54721 pts 
RR-1 vars:  57%|▌| 4/7 [18:48<11:29, 229.76s/it, SAA_Dose_mGy_d: ok · 54721 pts 
RR-1 vars:  71%|▋| 5/7 [18:48<07:29, 224.58s/it, SAA_Dose_mGy_d: ok · 54721 pts 
RR-1 vars:  71%|▋| 5/7 [22:16<07:29, 224.58s/it, Total_Dose_mGy_d: ok · 54721 pt
RR-1 vars:  86%|▊| 6/7 [22:16<03:41, 221.47s/it, Total_Dose_mGy_d: ok · 54721 pt
RR-1 vars:  86%|▊| 6/7 [25:4

[✓] Wrote seasonality summary → /Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/outputs/pattern_analysis/seasonality_summary.csv
[✓] Appended progress log → /Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/outputs/pattern_analysis/pattern_progress.csv


,Mission,Type,Variable,N,Std,Seasonality_Strength,Seasonal_Amplitude_P95_P05,Dominant_Freq_Hz,Dominant_Period_min,Seconds
20,RR-6,radiation,Accumulated_Dose_mGy_d,69121,3.91,1.00,0.25,0.00,69121.00,261.82
6,RR-1,radiation,Accumulated_Dose_mGy_d,54721,2.24,1.00,0.19,0.00,54721.00,207.47
34,RR-12,radiation,Accumulated_Dose_mGy_d,51841,3.74,1.00,0.32,0.00,51841.00,196.96
41,RR-19,radiation,Accumulated_Dose_mGy_d,48961,3.59,1.00,0.32,0.00,48961.00,184.60
13,RR-3,radiation,Accumulated_Dose_mGy_d,60481,2.80,1.00,0.21,0.00,60481.00,227.87
27,RR-9,radiation,Accumulated_Dose_mGy_d,44641,2.84,0.97,0.29,0.00,44641.00,169.60
7,RR-3,telemetry,Temp_degC_ISS,71968,0.96,0.66,1.71,0.00,71968.00,276.85
37,RR-19,telemetry,CO2_ppm_ISS,50967,277.67,0.62,609.03,0.00,1456.20,194.65
30,RR-12,telemetry,CO2_ppm_ISS,58940,434.31,0.54,631.23,0.00,1437.56,224.30
16,RR-6,telemetry,CO2_ppm_ISS,87872,817.94,0.51,596.05,0.00,87872.00,331.62


### Section 5: Relationship Mapping
#### Purpose
Quantify associations and temporal dependencies across telemetry and radiation; prepare for biological/behavioral linkage.
#### What is implemented
1. Pearson and Spearman correlation matrices with heatmap exports.
2. Conditional Granger causality via VAR: AIC-based lag selection, optional de-seasonalization using Section 4 indices, stationarity checks (ADF/diff), and BH FDR control with directed heatmaps and tidy CSV.
3. Correlation network graphs for strong edges (thresholded).
#### Future Work
- Partial correlations controlling for time of day and orbital phase.
- Non-linear directionality (transfer entropy or directed information).
- Extend analyses to additional tissues (skin) and behavioral/phenotypic signals when available.
- Formal interpretability (e.g., SHAP) once multivariate models are introduced.

In [6]:
# ==== Section 5: Relationship Mapping (VAR Granger + Partial Corr + optional MI) ====
import os, warnings, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from scipy.stats import spearmanr

# tqdm fallback
try:
    from tqdm.auto import tqdm
    def tprint(*a, **k): tqdm.write(*a, **k)
except Exception:
    def tqdm(x, **k): return x
    def tprint(*a, **k): print(*a, **k)

# Optional deps
try:
    import networkx as nx
    HAS_NX = True
except Exception:
    HAS_NX = False

try:
    from sklearn.feature_selection import mutual_info_regression
    HAS_MI = True
except Exception:
    HAS_MI = False

warnings.filterwarnings("ignore")

# ---- Config ----
telemetry_metrics = ["Temp_degC_ISS", "RH_percent_ISS", "CO2_ppm_ISS"]
radiation_metrics  = ["GCR_Dose_mGy_d", "SAA_Dose_mGy_d", "Total_Dose_mGy_d", "Accumulated_Dose_mGy_d"]

resample_freq = "5min"
maxlags_minutes = 60
min_rows = 1500
pearson_abs_threshold = 0.6
alpha_fdr = 0.05
RUN_NONLINEAR = False  # set True to scan lagged MI (nonlinear lead–lag)

REL_DIR = S.relationships_dir
PLOTS_DIR = REL_DIR / "plots"
REL_DIR.mkdir(parents=True, exist_ok=True)
PLOTS_DIR.mkdir(parents=True, exist_ok=True)

seasonal_dir = S.pattern_dir / "seasonal_indices"  # from Section 4

# ---- Helpers ----
def _load_preprocessed(rr: str, kind: str) -> pd.DataFrame | None:
    try:
        if kind == "telemetry":
            if "preprocessed_telemetry" in globals() and rr in preprocessed_telemetry:
                return preprocessed_telemetry[rr].copy()
            p = S.preprocessed_dir / f"{rr}_cleaned_telemetry.csv"
        else:
            if "cleaned_radiation" in globals() and rr in cleaned_radiation:
                return cleaned_radiation[rr].copy()
            p = S.preprocessed_dir / f"{rr}_cleaned_radiation.csv"
        if p.exists():
            return pd.read_csv(p, index_col=0, parse_dates=True).sort_index()
    except Exception as e:
        tprint(f"[LOAD FAIL] {rr} {kind}: {e}")
    return None

def _missions_with_both() -> list[str]:
    ms = []
    for rr in S.missions:
        tel_ok = (S.preprocessed_dir / f"{rr}_cleaned_telemetry.csv").exists() or \
                 ("preprocessed_telemetry" in globals() and rr in preprocessed_telemetry)
        rad_ok = (S.preprocessed_dir / f"{rr}_cleaned_radiation.csv").exists() or \
                 ("cleaned_radiation" in globals() and rr in cleaned_radiation)
        if tel_ok and rad_ok:
            ms.append(rr)
    return ms

def _maybe_deseason(rr: str, col: str, ts: pd.Series) -> pd.Series:
    f = seasonal_dir / f"{rr}_{col}_seasonal.csv"
    x = ts.resample(resample_freq).mean().interpolate(limit=5)
    if f.exists():
        try:
            s = pd.read_csv(f, parse_dates=["timestamp"]).set_index("timestamp")["seasonal"]
            s = s.resample(resample_freq).mean().interpolate(limit=5)
            return (x.reindex(s.index) - s).dropna()
        except Exception:
            return x.dropna()
    return x.dropna()

def _adf_stationarize(s: pd.Series) -> pd.Series:
    s = s.dropna()
    if len(s) < 200: return s
    try:
        p = adfuller(s, autolag="AIC")[1]
        return s if p <= 0.05 else s.diff().dropna()
    except Exception:
        return s

def _bh_fdr(pvals: pd.Series, alpha=0.05) -> pd.Series:
    pv = pvals.astype(float).copy()
    m = pv.notna().sum()
    if m == 0: return pv
    order = pv.sort_values().index
    ranks = pd.Series(range(1, m+1), index=order, dtype=float)
    q = pv.copy()
    prev = 1.0
    for idx in reversed(order):
        rank = ranks[idx]
        q[idx] = min(prev, pv[idx] * m / rank)
        prev = q[idx]
    return q

def _build_covariates(idx: pd.DatetimeIndex, tel_df: pd.DataFrame | None) -> pd.DataFrame:
    h = idx.hour + idx.minute/60.0
    rad = 2*np.pi*h/24.0
    cov = pd.DataFrame({
        "const": 1.0,
        "tod_sin": np.sin(rad),
        "tod_cos": np.cos(rad)
    }, index=idx)
    # add mission-derived proxies if present
    if tel_df is not None:
        for c in ["Orbital_Day", "Crew_Awake"]:
            if c in tel_df.columns:
                z = tel_df[c].resample(resample_freq).median()
                cov[c] = z.reindex(idx).interpolate(limit=5)
    return cov.dropna()

def _residualize_matrix(df: pd.DataFrame, cov: pd.DataFrame) -> pd.DataFrame:
    X = cov.values
    XtX_inv = None
    try:
        XtX_inv = np.linalg.inv(X.T @ X)
    except Exception:
        XtX_inv = np.linalg.pinv(X.T @ X)
    beta = XtX_inv @ X.T @ df.values
    resid = df.values - X @ beta
    return pd.DataFrame(resid, index=df.index, columns=df.columns)

def _save_heatmap(mat: pd.DataFrame, title: str, out_png: Path, annot=False):
    plt.figure(figsize=(10,8))
    sns.heatmap(mat, cmap="coolwarm", annot=annot, fmt=".2f", vmin=-1, vmax=1)
    plt.title(title); plt.tight_layout(); plt.savefig(out_png); plt.close()

def _network_from_corr(corr: pd.DataFrame, out_png: Path, thr=0.6):
    if not HAS_NX: return
    G = nx.Graph()
    cols = list(corr.columns)
    for i in range(len(cols)):
        for j in range(i+1, len(cols)):
            w = float(corr.iloc[i,j])
            if abs(w) >= thr:
                G.add_edge(cols[i], cols[j], weight=round(w,2))
    if G.number_of_edges() == 0: return
    plt.figure(figsize=(8,6))
    pos = nx.spring_layout(G, seed=42)
    nx.draw(G, pos, with_labels=True, node_color="lightblue", edge_color="gray",
            node_size=2000, font_size=9, width=1.5)
    nx.draw_networkx_edge_labels(G, pos, edge_labels=nx.get_edge_attributes(G, "weight"))
    plt.title(out_png.stem); plt.tight_layout(); plt.savefig(out_png); plt.close()

# ---- Main ----
missions = _missions_with_both()
tprint(f"Relationship mapping for missions: {missions}")

all_granger = []
all_corr_tidy = []
all_mi = []

for rr in tqdm(missions, desc="Missions"):
    tel = _load_preprocessed(rr, "telemetry")
    rad = _load_preprocessed(rr, "radiation")
    if tel is None or rad is None:
        tprint(f"[SKIP] {rr}: missing data")
        continue

    cols = [c for c in telemetry_metrics if c in tel.columns] + \
           [c for c in radiation_metrics if c in rad.columns]
    if len(cols) < 3:
        tprint(f"[SKIP] {rr}: too few variables")
        continue

    df = tel[telemetry_metrics].join(rad[radiation_metrics], how="inner")
    df = df[[c for c in cols]]
    df = df.resample(resample_freq).mean().interpolate(limit=5).dropna()
    if len(df) < min_rows:
        tprint(f"[SKIP] {rr}: insufficient rows ({len(df)})")
        continue

    # deseason + stationarity
    for c in list(df.columns):
        df[c] = _maybe_deseason(rr, c, df[c])
        df[c] = _adf_stationarize(df[c])
    df = df.dropna()
    if len(df) < min_rows:
        tprint(f"[SKIP] {rr}: insufficient rows after transforms ({len(df)})")
        continue

    # correlations
    pearson_corr = df.corr(method="pearson")
    scorr, _ = spearmanr(df.values)
    spearman_corr = pd.DataFrame(scorr, index=df.columns, columns=df.columns)

    # partial correlations (control for time-of-day + proxies)
    cov = _build_covariates(df.index, tel)
    df_aligned = df.reindex(cov.index).dropna()
    cov = cov.reindex(df_aligned.index).dropna()
    df_aligned = df_aligned.loc[cov.index]
    resid = _residualize_matrix(df_aligned, cov)
    partial_corr = resid.corr(method="pearson")

    # save corr outputs
    pearson_corr.to_csv(REL_DIR / f"{rr}_pearson_corr.csv")
    spearman_corr.to_csv(REL_DIR / f"{rr}_spearman_corr.csv")
    partial_corr.to_csv(REL_DIR / f"{rr}_partial_corr.csv")
    _save_heatmap(pearson_corr, f"Pearson — {rr}", PLOTS_DIR / f"{rr}_pearson.png")
    _save_heatmap(spearman_corr, f"Spearman — {rr}", PLOTS_DIR / f"{rr}_spearman.png")
    _save_heatmap(partial_corr, f"Partial (controls: TOD/Orbital/Crew) — {rr}", PLOTS_DIR / f"{rr}_partial.png")
    _network_from_corr(pearson_corr, PLOTS_DIR / f"{rr}_network.png", thr=pearson_abs_threshold)

    # tidy corr for dashboard
    for mname, mat in [("pearson", pearson_corr), ("spearman", spearman_corr), ("partial", partial_corr)]:
        tri = mat.where(np.triu(np.ones(mat.shape), k=1).astype(bool))
        tidy = tri.stack().reset_index()
        tidy.columns = ["Var1", "Var2", "Value"]
        tidy["Mission"] = rr
        tidy["Method"]  = mname
        all_corr_tidy.append(tidy)

    # VAR Granger (conditional)
    step = int(resample_freq.replace("min",""))
    maxlags = max(1, int(maxlags_minutes/step))
    try:
        sel = VAR(df).select_order(maxlags=maxlags)
        k_ar = int(sel.aic or max(1, sel.selected_orders.get("aic", 1)))
        k_ar = max(1, min(k_ar, maxlags))
        model = VAR(df).fit(k_ar)
    except Exception as e:
        tprint(f"[VAR FAIL] {rr}: {e}")
        continue

    pairs = [(y, x) for y in df.columns for x in df.columns if x != y]
    rows = []
    for (target, source) in pairs:
        try:
            test = model.test_causality(caused=target, causing=[source], kind="f")
            rows.append({
                "Mission": rr, "Lag": k_ar,
                "Target": target, "Source": source,
                "F": float(getattr(test, "test_statistic", np.nan)),
                "pvalue": float(getattr(test, "pvalue", np.nan))
            })
        except Exception:
            rows.append({"Mission": rr, "Lag": k_ar, "Target": target, "Source": source, "F": np.nan, "pvalue": np.nan})

    res_df = pd.DataFrame(rows)
    res_df["qvalue"] = _bh_fdr(res_df["pvalue"], alpha=alpha_fdr)
    res_df["Significant"] = res_df["qvalue"] <= alpha_fdr
    res_df["MinusLog10_q"] = -np.log10(res_df["qvalue"].clip(lower=1e-300))
    all_granger.append(res_df)

    mat = res_df.pivot(index="Target", columns="Source", values="MinusLog10_q").reindex(index=df.columns, columns=df.columns)
    plt.figure(figsize=(10,8))
    sns.heatmap(mat, cmap="viridis")
    plt.title(f"Directed Granger (−log10 q) — {rr} [lag={k_ar}]")
    plt.tight_layout(); plt.savefig(PLOTS_DIR / f"{rr}_granger_directed.png"); plt.close()

    # optional: nonlinear lag scan (mutual information)
    if RUN_NONLINEAR and HAS_MI:
        maxlag_steps = maxlags
        mi_rows = []
        for tgt in df.columns:
            y = df[tgt].values.reshape(-1, 1)
            for src in df.columns:
                if src == tgt: continue
                x = df[src].values
                best_mi, best_lag = 0.0, 0
                for L in range(1, maxlag_steps+1):
                    Xlag = x[:-L].reshape(-1, 1)
                    Yt   = y[L:]
                    mi = float(mutual_info_regression(Xlag, Yt.ravel(), random_state=S.seed, n_neighbors=3))
                    if mi > best_mi:
                        best_mi, best_lag = mi, L
                mi_rows.append({"Mission": rr, "Target": tgt, "Source": src, "BestLagSteps": best_lag, "BestLagMin": best_lag*step, "MI": best_mi})
        mi_df = pd.DataFrame(mi_rows)
        mi_df.to_csv(REL_DIR / f"{rr}_lagged_mi.csv", index=False)
        all_mi.append(mi_df)
        # heatmap of best MI
        H = mi_df.pivot(index="Target", columns="Source", values="MI").reindex(index=df.columns, columns=df.columns)
        plt.figure(figsize=(10,8))
        sns.heatmap(H, cmap="magma", vmin=0)
        plt.title(f"Lagged MI (best over ≤{maxlags_minutes} min) — {rr}")
        plt.tight_layout(); plt.savefig(PLOTS_DIR / f"{rr}_lagged_mi.png"); plt.close()

# ---- Save combined outputs ----
if len(all_granger):
    gr_all = pd.concat(all_granger, ignore_index=True)
    gr_all.to_csv(REL_DIR / "granger_var_results.csv", index=False)
    tprint(f"[✓] Wrote VAR-Granger results → {REL_DIR/'granger_var_results.csv'} (rows={len(gr_all)})")
else:
    tprint("[!] No VAR-Granger results to save.")

if len(all_corr_tidy):
    corr_all = pd.concat(all_corr_tidy, ignore_index=True)
    corr_all.to_csv(REL_DIR / "correlations_tidy.csv", index=False)
    tprint(f"[✓] Wrote correlations tidy → {REL_DIR/'correlations_tidy.csv'} (rows={len(corr_all)})")

if RUN_NONLINEAR and len(all_mi):
    mi_all = pd.concat(all_mi, ignore_index=True)
    mi_all.to_csv(REL_DIR / "lagged_mi_all.csv", index=False)
    tprint(f"[✓] Wrote lagged MI (all) → {REL_DIR/'lagged_mi_all.csv'} (rows={len(mi_all)})")


Relationship mapping for missions: ['RR-1', 'RR-3', 'RR-6', 'RR-9', 'RR-12', 'RR-19']


Missions:  50%|█████████████████▌                 | 3/6 [00:18<00:18,  6.31s/it]

[SKIP] RR-9: insufficient rows (0)


Missions: 100%|███████████████████████████████████| 6/6 [00:27<00:00,  4.56s/it]

[✓] Wrote VAR-Granger results → /Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/outputs/relationships/granger_var_results.csv (rows=210)
[✓] Wrote correlations tidy → /Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/outputs/relationships/correlations_tidy.csv (rows=315)


### Section 6: Anomaly Detection and Forecasting
#### Purpose
Detect environmental deviations and produce short-term forecasts useful for life-support planning.
#### What is implemented
1. Statistical detectors: Z-score and EWMA with plots per variable.
2. LSTM autoencoder (reconstruction error) and one-step LSTM forecaster with split evaluation.
3. macOS GPU/MPS awareness; metrics CSVs and consolidated scoreboard.
#### Future Work
- Compact SARIMAX grid and mission-level multivariate VAR.
- Broader SARIMAX grid search with AIC selection and saved diagnostics.
- Residual diagnostics and Ljung–Box tests for each baseline model.
- Conformal prediction intervals or quantile regression for uncertainty bands.
- Roll out the LSTM/anomaly suite beyond RR-1 (RR-3/RR-6/RR-9/RR-12/RR-19) using same evaluation.
- Path to multivariate forecasting (e.g., multivariate LSTM/attention) where feasible.

In [1]:
# ==== Section 6: Anomaly Detection & Forecasting (restart-safe, with progress logs) ====

import os, time, warnings, platform
from dataclasses import dataclass, field
from pathlib import Path
from typing import List, Tuple
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

import tensorflow as tf
from tensorflow.keras import mixed_precision
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense
from tensorflow.keras.callbacks import EarlyStopping, Callback

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import acorr_ljungbox

warnings.filterwarnings("ignore")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")

# ---------- tqdm (robust text mode) ----------
try:
    from tqdm import tqdm
    def _tqdm(iterable=None, **kwargs):
        return tqdm(iterable, dynamic_ncols=True, miniters=1, smoothing=0.05, **kwargs)
except Exception:
    def _tqdm(it, **kwargs): return it

# ---------- Minimal Settings fallback ----------
if "S" not in globals():
    @dataclass
    class Settings:
        missions: List[str] = field(default_factory=lambda: ["RR-1","RR-3","RR-6","RR-9","RR-12","RR-19"])
        sampling: str = "1min"
        outputs_root: Path = field(default_factory=lambda: Path.cwd() / "outputs")
        preprocessed_dir: Path = field(default_factory=lambda: Path.cwd() / "outputs" / "preprocessed")
        pattern_dir: Path = field(default_factory=lambda: Path.cwd() / "outputs" / "pattern_analysis")
        anomalies_dir: Path = field(default_factory=lambda: Path.cwd() / "outputs" / "anomaly_forecast")
        relationships_dir: Path = field(default_factory=lambda: Path.cwd() / "outputs" / "relationships")
        omics_roots: List[Path] = field(default_factory=lambda: [])
        data_roots: List[Path] = field(default_factory=lambda: [])
    S = Settings()
    for d in [S.outputs_root, S.preprocessed_dir, S.pattern_dir, S.anomalies_dir]:
        d.mkdir(parents=True, exist_ok=True)
    print("[Sec6 bootstrap] Using fallback Settings (S).")

# ---------- Lightweight logger (stdout + file) ----------
OUTDIR = S.anomalies_dir
OUTDIR.mkdir(parents=True, exist_ok=True)
PROGRESS_LOG = OUTDIR / "sec6_progress.log"

def log(msg: str):
    ts = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
    line = f"[{ts} UTC] {msg}"
    print(line, flush=True)
    try:
        with open(PROGRESS_LOG, "a") as f:
            f.write(line + "\n")
    except Exception:
        pass

# ---------- Device policy ----------
def _has_cuda_gpu() -> bool:
    try:
        gpus = tf.config.list_physical_devices("GPU")
        for g in gpus:
            name = str(tf.config.experimental.get_device_details(g).get("device_name", "")).lower()
            if ("nvidia" in name) or ("cuda" in name):
                return True
    except Exception:
        pass
    return False

gpus = tf.config.list_physical_devices("GPU")
for d in gpus:
    try: tf.config.experimental.set_memory_growth(d, True)
    except Exception: pass

if platform.system() == "Darwin" or not _has_cuda_gpu():
    # CPU or Apple M-series (MPS): keep float32 for speed/stability
    log(f"[Sec6] TF {tf.__version__} · {platform.system()} {platform.release()} · GPUs={len(gpus)} (no CUDA) → FP32")
else:
    mixed_precision.set_global_policy("mixed_float16")
    log(f"[Sec6] TF {tf.__version__} · CUDA GPU detected → policy {mixed_precision.global_policy()}")

# ---------- Config ----------
# Choose which missions to process (you can widen to S.missions later)
MISSIONS = ["RR-1"]

telemetry_cols = ["Temp_degC_ISS", "RH_percent_ISS", "CO2_ppm_ISS"]
radiation_cols  = ["GCR_Dose_mGy_d", "SAA_Dose_mGy_d", "Total_Dose_mGy_d", "Accumulated_Dose_mGy_d"]

# Runtime toggles
ENABLE_LSTM     = True
ENABLE_SARIMAX  = False   # <— default OFF to avoid long runs; flip to True when benchmarking
ENABLE_VAR      = True
RESUME_SAFE     = True    # skip series if artifacts already exist
WRITE_AFTER_EACH_SERIES = True  # incrementally flush scoreboard

# SARIMAX bounds (if/when enabled)
SARIMAX_TIME_BUDGET_S = 600      # per-series wall time budget
SARIMAX_GRID = {
    "p": (0,1,2),
    "d": (0,1),
    "q": (0,1,2),
    "P": (0,1),
    "D": (0,1),
    "Q": (0,1),
}
SARIMAX_ONLY_DOWNSAMPLE = "5min"  # fit SARIMAX on coarser grid to speed up

resample_freq = S.sampling   # minute-level data for detectors/LSTM
seq_len = 45                  # LSTM window length (minutes)
epochs = 3                    # keep small for first pass; raise when ready
batch_size = 32
val_frac = 0.2
threshold_z = 3.0             # z-score anomaly threshold
ewma_span = 60                 # EWMA window (minutes)
min_points = 1000             # minimal usable length
min_std = 1e-6                # guard against flat signals
alpha_pi = 0.1                # 90% conformal interval

early_stop = EarlyStopping(monitor="val_loss", patience=1, restore_best_weights=True)

# ---------- Keras mini-callback for concise epoch logs ----------
class EpochLogger(Callback):
    def __init__(self, tag: str): self.tag = tag
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        l = logs.get("loss", np.nan); vl = logs.get("val_loss", np.nan)
        log(f"{self.tag} epoch {epoch+1}: loss={l:.5f} val_loss={vl:.5f}")

# ---------- IO helpers (rely on Section 3 outputs; restart-safe) ----------
seasonal_dir = S.pattern_dir / "seasonal_indices"

def _load_preprocessed(rr: str, kind: str) -> pd.DataFrame | None:
    try:
        if kind == "telemetry":
            p = S.preprocessed_dir / f"{rr}_cleaned_telemetry.csv"
        else:
            p = S.preprocessed_dir / f"{rr}_cleaned_radiation.csv"
        if p.exists():
            return pd.read_csv(p, index_col=0, parse_dates=True).sort_index()
    except Exception as e:
        log(f"[LOAD FAIL] {rr} {kind}: {e}")
    return None

def _maybe_deseason(rr: str, col: str, ts: pd.Series) -> pd.Series:
    """Subtract STL seasonal index from Section 4 if present; otherwise just clean/resample."""
    f = seasonal_dir / f"{rr}_{col}_seasonal.csv"
    x = ts.resample(resample_freq).mean().interpolate(limit=5)
    if f.exists():
        try:
            s = pd.read_csv(f, parse_dates=["timestamp"]).set_index("timestamp")["seasonal"]
            s = s.resample(resample_freq).mean().interpolate(limit=5)
            x = x.reindex(s.index).interpolate(limit=5)
            return (x - s).dropna()
        except Exception:
            return x.dropna()
    return x.dropna()

def _adf_p(s: pd.Series) -> float:
    try: return adfuller(s.dropna(), autolag="AIC")[1]
    except Exception: return 1.0

def _lb_p(resid: np.ndarray, lags: int = 24) -> float:
    try:
        lb = acorr_ljungbox(resid, lags=[lags], return_df=True)
        return float(lb["lb_pvalue"].iloc[0])
    except Exception:
        return np.nan

def _train_test_split_series(y: pd.Series, frac=0.2) -> Tuple[pd.Series, pd.Series]:
    n = len(y); n_tr = max(int(n*(1-frac)), 10)
    return y.iloc[:n_tr], y.iloc[n_tr:]

def _create_sequences(arr: np.ndarray, L: int):
    X, y = [], []
    for i in range(len(arr)-L):
        X.append(arr[i:i+L])
        y.append(arr[i+L])
    return np.array(X), np.array(y)

def _savefig(path: Path):
    plt.tight_layout(); plt.savefig(path, dpi=160); plt.close()

# ---------- Plotters (dashboard-compatible filenames) ----------
def _plot_z_anoms(ts: pd.Series, z_mask: pd.Series, title: str, outdir: Path):
    plt.figure(figsize=(12,4))
    plt.plot(ts.index, ts.values, label="value", color="black")
    if z_mask.any():
        plt.scatter(ts.index[z_mask], ts.values[z_mask], s=8, color="red", label="|z|>3")
    plt.title(title); plt.legend()
    _savefig(outdir / "zscore_anomalies.png")       # dashboard
    # legacy duplicate for backward compatibility
    _savefig(outdir / "anoms.png")

def _plot_ewma_anoms(ts: pd.Series, ewma: pd.Series, ew_mask: pd.Series, title: str, outdir: Path):
    plt.figure(figsize=(12,4))
    plt.plot(ts.index, ts.values, label="value", color="black")
    plt.plot(ewma.index, ewma.values, color="blue", alpha=0.8, label="EWMA")
    if ew_mask.any():
        plt.scatter(ts.index[ew_mask], ts.values[ew_mask], s=8, color="orange", label="EWMA anom")
    plt.title(title); plt.legend()
    _savefig(outdir / "ewma_anomalies.png")

# ---------- Metrics writer ----------
def _append_metrics_row(row: dict, scoreboard_path: Path):
    df = pd.DataFrame([row])
    if scoreboard_path.exists():
        try:
            df_existing = pd.read_csv(scoreboard_path)
            df = pd.concat([df_existing, df], ignore_index=True)
        except Exception:
            pass
    df.to_csv(scoreboard_path, index=False)

# ---------- Models ----------
def run_univariate(rr: str, signal_type: str, col: str, y: pd.Series, outdir: Path,
                   all_metrics: list, scoreboard_path: Path):
    t0 = time.time()
    if len(y) < min_points or y.std() < min_std:
        log(f"[SKIP] {rr} {signal_type}:{col} — short/flat"); return

    # Resume-skip if artifacts already present
    if RESUME_SAFE:
        needed = [outdir/"zscore_anomalies.png", outdir/"ewma_anomalies.png",
                  outdir/"lstm_reconstruction_error.png", outdir/"lstm_forecast.png"]
        if all(p.exists() for p in needed):
            log(f"[SKIP] {rr} {signal_type}:{col} — artifacts already present")
            return

    # 1) Statistical detectors (always)
    z = (y - y.mean())/y.std()
    z_mask = z.abs() > threshold_z
    ew = y.ewm(span=ewma_span).mean()
    resid = y - ew
    ew_mask = resid.abs() > (2*resid.std())
    _plot_z_anoms(y, z_mask, f"{col} Z-score anomalies ({rr})", outdir)
    _plot_ewma_anoms(y, ew, ew_mask, f"{col} EWMA anomalies ({rr})", outdir)
    log(f"[{rr} {signal_type}:{col}] detectors done in {time.time()-t0:.1f}s")

    # Scale (MinMax on the single variable)
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(y.to_frame("v").values).astype(np.float32)

    # 2) LSTM Autoencoder (reconstruction error → anomaly score)
    if ENABLE_LSTM:
        X_ae, _ = _create_sequences(scaled, seq_len)
        if len(X_ae) < 100:
            log(f"[SKIP] {rr} {signal_type}:{col} — too few AE seqs")
        else:
            split = int(len(X_ae)*(1-val_frac))
            Xtr, Xte = X_ae[:split], X_ae[split:]

            inputs = Input(shape=(seq_len, 1))
            enc = LSTM(16, activation="relu", return_sequences=False)(inputs)
            dec = RepeatVector(seq_len)(enc)
            dec = LSTM(16, activation="relu", return_sequences=True)(dec)
            outs = TimeDistributed(Dense(1))(dec)
            ae = Model(inputs, outs)
            ae.compile(optimizer="adam", loss="mse")

            ta0 = time.time()
            val_split = 0.1 if len(Xtr) > 20 else 0.0  # guard tiny splits
            ae.fit(Xtr, Xtr, epochs=epochs, batch_size=batch_size, validation_split=val_split,
                   shuffle=False, callbacks=[early_stop, EpochLogger(f"AE {rr}:{col}")], verbose=0)
            Xte_pred = ae.predict(Xte, verbose=0)
            mse = np.mean(np.square(Xte - Xte_pred), axis=(1,2))
            ae_thr = float(np.percentile(mse, 95))

            plt.figure(figsize=(12,4))
            plt.plot(mse, color="black"); plt.axhline(ae_thr, color="red", ls="--")
            plt.title(f"{col} AE reconstruction error ({rr})")
            _savefig(outdir / "lstm_reconstruction_error.png")  # dashboard name
            _savefig(outdir / "ae_error.png")                   # legacy duplicate
            log(f"[{rr} {signal_type}:{col}] AE done in {time.time()-ta0:.1f}s")
    else:
        mse, ae_thr = np.array([]), np.nan

    # 3) LSTM forecaster (one-step) + conformal prediction interval
    if ENABLE_LSTM:
        tf0 = time.time()
        Xf, yf = _create_sequences(scaled, seq_len)
        split_f = int(len(Xf)*(1-val_frac))
        Xtrf, ytrf = Xf[:split_f], yf[:split_f]
        Xtef, ytef = Xf[split_f:], yf[split_f:]
        fnet = Sequential([LSTM(16, activation="relu", input_shape=(seq_len,1)), Dense(1)])
        fnet.compile(optimizer="adam", loss="mse")
        fnet.fit(Xtrf, ytrf, epochs=epochs, batch_size=batch_size, shuffle=False,
                 callbacks=[early_stop, EpochLogger(f"FC {rr}:{col}")], verbose=0)

        # calibration for conformal PI
        n_cal = max(50, int(0.4*len(Xtef)))
        n_cal = min(n_cal, len(Xtef)-1) if len(Xtef) > 1 else 0
        y_cal_pred = fnet.predict(Xtef[:n_cal], verbose=0).reshape(-1,1) if n_cal > 0 else np.zeros((0,1))
        y_cal_true = ytef[:n_cal].reshape(-1,1) if n_cal > 0 else np.zeros((0,1))
        cal_resid = np.abs(y_cal_true - y_cal_pred)
        q = float(np.quantile(cal_resid, 1 - alpha_pi)) if len(cal_resid) else 0.0

        y_pred = fnet.predict(Xtef, verbose=0).reshape(-1,1)
        y_true = ytef.reshape(-1,1)
        # invert scale
        y_true_inv = scaler.inverse_transform(y_true)
        y_pred_inv = scaler.inverse_transform(y_pred)
        # scale-aware PI width
        denom = (y_true.std() + 1e-9)
        q_inv = float(q * (y_true_inv.std() / denom)) if denom > 0 else float(q)

        rmse = float(np.sqrt(mean_squared_error(y_true_inv, y_pred_inv)))
        mae  = float(mean_absolute_error(y_true_inv, y_pred_inv))

        # plot forecast + conformal band (after calibration region)
        plt.figure(figsize=(12,4))
        plt.plot(y_true_inv, label="actual", color="black")
        plt.plot(y_pred_inv, label="lstm", color="green")
        lo = y_pred_inv - q_inv; hi = y_pred_inv + q_inv
        if len(y_pred_inv) and n_cal < len(y_pred_inv):
            xs = np.arange(n_cal, len(y_pred_inv))
            plt.fill_between(xs, lo[n_cal:, 0], hi[n_cal:, 0], alpha=0.2, label="90% PI")
        plt.title(f"{col} LSTM forecast ({rr})  RMSE={rmse:.2f}, MAE={mae:.2f}")
        plt.legend()
        _savefig(outdir / "lstm_forecast.png")
        log(f"[{rr} {signal_type}:{col}] forecaster done in {time.time()-tf0:.1f}s")

        row = {
            "RR": rr, "Type": signal_type, "Variable": col, "Model": "LSTM",
            "RMSE": rmse, "MAE": mae,
            "AE_Threshold": float(ae_thr) if np.isfinite(ae_thr) else np.nan,
            "AE_MeanMSE": float(np.mean(mse)) if mse.size else np.nan,
            "AE_Time_s": float(time.time()-ta0) if ENABLE_LSTM and 'ta0' in locals() else np.nan,
            "SeqLen": seq_len, "Resample": resample_freq, "Conformal_q_inv": q_inv
        }
        all_metrics.append(row)
        if WRITE_AFTER_EACH_SERIES:
            _append_metrics_row(row, scoreboard_path)

    # 4) SARIMAX baseline (bounded) — optional
    if ENABLE_SARIMAX:
        tb0 = time.time()
        try:
            # downsample only for SARIMAX to speed up
            y_sar = y.asfreq(resample_freq).resample(SARIMAX_ONLY_DOWNSAMPLE).mean().interpolate(limit=3)
            # daily season at the chosen grid
            step_min = int(str(SARIMAX_ONLY_DOWNSAMPLE).replace("min","").replace("T",""))
            s = max(2, int(1440 / max(1, step_min)))
            train, test = _train_test_split_series(y_sar)
            best = None

            def _time_ok(): return (time.time()-tb0) < SARIMAX_TIME_BUDGET_S

            for p in SARIMAX_GRID["p"]:
                if not _time_ok(): break
                for d in SARIMAX_GRID["d"]:
                    if not _time_ok(): break
                    for q_ in SARIMAX_GRID["q"]:
                        if not _time_ok(): break
                        for P in SARIMAX_GRID["P"]:
                            if not _time_ok(): break
                            for D in SARIMAX_GRID["D"]:
                                if not _time_ok(): break
                                for Q in SARIMAX_GRID["Q"]:
                                    if not _time_ok(): break
                                    order=(p,d,q_); seas=(P,D,Q,s)
                                    try:
                                        m = SARIMAX(train, order=order, seasonal_order=seas,
                                                    enforce_stationarity=False, enforce_invertibility=False)
                                        r = m.fit(disp=False)
                                        aic = r.aic
                                        if np.isfinite(aic) and (best is None or aic < best[0]):
                                            best = (aic, order, seas, r)
                                    except Exception:
                                        continue

            if best is not None:
                _, order, seas, rfit = best
                pred = rfit.get_forecast(steps=len(test))
                mean = pred.predicted_mean
                ci = pred.conf_int(alpha=alpha_pi)
                rmse_s = float(np.sqrt(mean_squared_error(test.values, mean.values)))
                mae_s  = float(mean_absolute_error(test.values, mean.values))
                lbp = _lb_p(rfit.resid.dropna().values, lags=24)

                plt.figure(figsize=(12,4))
                plt.plot(train.index, train.values, color="gray", alpha=0.6, label="train")
                plt.plot(test.index, test.values, color="black", label="actual")
                plt.plot(test.index, mean.values, color="purple", label=f"SARIMAX{order}x{seas}")
                if isinstance(ci, pd.DataFrame):
                    plt.fill_between(test.index, ci.iloc[:,0].values, ci.iloc[:,1].values, alpha=0.2, label="PI")
                ttl = f"{col} SARIMAX ({rr})  RMSE={rmse_s:.2f}, MAE={mae_s:.2f} | Ljung-Box p={lbp:.3f}"
                plt.title(ttl); plt.legend()
                _savefig(outdir / "sarimax_forecast.png")

                row = {
                    "RR": rr, "Type": signal_type, "Variable": col,
                    "Model": f"SARIMAX{order}x{seas}", "RMSE": rmse_s, "MAE": mae_s, "LB_p": lbp,
                    "GridMin": SARIMAX_ONLY_DOWNSAMPLE, "TimeBudget_s": SARIMAX_TIME_BUDGET_S
                }
                all_metrics.append(row)
                if WRITE_AFTER_EACH_SERIES:
                    _append_metrics_row(row, scoreboard_path)

            took = time.time()-tb0
            log(f"[{rr} {signal_type}:{col}] SARIMAX done in {took:.1f}s | total {time.time()-t0:.1f}s")
        except Exception as e:
            log(f"[SARIMAX ERR] {rr} {signal_type}:{col}: {e}")
    else:
        log(f"[{rr} {signal_type}:{col}] SARIMAX skipped (disabled) | total {time.time()-t0:.1f}s")

def run_var_baseline(rr: str, df_multi: pd.DataFrame) -> pd.DataFrame | None:
    """Simple mission-level multivariate VAR baseline (forecast split)."""
    try:
        df = df_multi.copy().dropna()
        if len(df) < 800: return None
        try:
            if _adf_p(df.iloc[:,0]) > 0.05:
                df = df.diff().dropna()
        except Exception:
            pass
        # choose lags up to ~30 minutes relative to S.sampling
        step = int(str(S.sampling).replace("min","").replace("T",""))
        maxlags = max(1, int(30/step))
        sel = VAR(df).select_order(maxlags=maxlags)
        # defensive: prefer AIC if available else 1
        k = getattr(sel, "aic", None)
        if k is None or (isinstance(k, float) and not np.isfinite(k)):
            k = sel.selected_orders.get("aic", 1) if hasattr(sel, "selected_orders") else 1
        k = int(max(1, min(int(k), maxlags)))
        n = len(df); n_tr = int(n*0.8)
        model = VAR(df.iloc[:n_tr]).fit(k)
        fc = model.forecast(model.y, steps=n-n_tr)
        pred = pd.DataFrame(fc, index=df.index[n_tr:], columns=df.columns)
        out = []
        for c in df.columns:
            yt = df[c].iloc[n_tr:]; yp = pred[c].reindex(yt.index)
            out.append({"RR": rr, "Type": "multivariate", "Variable": c,
                        "Model": f"VAR(k={k})", "RMSE": float(np.sqrt(mean_squared_error(yt, yp))),
                        "MAE": float(mean_absolute_error(yt, yp))})
        return pd.DataFrame(out)
    except Exception as e:
        log(f"[VAR BASELINE ERR] {rr}: {e}")
        return None

# ---------- Build worklist ----------
work = []
for rr in MISSIONS:
    tel = _load_preprocessed(rr, "telemetry")
    rad = _load_preprocessed(rr, "radiation")
    if tel is not None:
        for c in telemetry_cols:
            if c in tel.columns: work.append((rr, "telemetry", c))
    if rad is not None:
        for c in radiation_cols:
            if c in rad.columns: work.append((rr, "radiation", c))

log(f"[Sec6] Jobs queued: {len(work)}")

# ---------- Main loop with progress ----------
all_metrics = []
scoreboard_path = OUTDIR / f"{'-'.join(MISSIONS)}_forecast_scoreboard.csv"

pbar = _tqdm(work, desc="Section 6 tasks")
for (rr, kind, col) in pbar:
    pbar.set_postfix_str(f"{rr}:{kind}:{col}")
    df = _load_preprocessed(rr, "telemetry" if kind=="telemetry" else "radiation")
    if df is None or col not in df.columns:
        log(f"[SKIP] {rr} {kind}:{col} — missing")
        continue
    s_raw = df[col].dropna()
    if len(s_raw) < min_points:
        log(f"[SKIP] {rr} {kind}:{col} — too short ({len(s_raw)} pts)")
        continue
    y = _maybe_deseason(rr, col, s_raw)
    y = y.replace([np.inf, -np.inf], np.nan).dropna()
    if len(y) < min_points or y.std() < min_std:
        log(f"[SKIP] {rr} {kind}:{col} — post-clean short/flat")
        continue

    outdir = OUTDIR / rr / f"{kind}_{col}"
    outdir.mkdir(parents=True, exist_ok=True)
    log(f"[START] {rr} {kind}:{col}  ({len(y)} pts)")
    run_univariate(rr, kind, col, y, outdir, all_metrics, scoreboard_path)
    log(f"[DONE ] {rr} {kind}:{col}")

# ---------- VAR baseline once per mission ----------
if ENABLE_VAR:
    for rr in MISSIONS:
        tel = _load_preprocessed(rr, "telemetry")
        rad = _load_preprocessed(rr, "radiation")
        if (tel is None) or (rad is None):
            continue
        cols_multi = [c for c in telemetry_cols if c in tel.columns] + \
                     [c for c in radiation_cols if c in rad.columns]
        if len(cols_multi) >= 2:
            mdf = tel[telemetry_cols].join(rad[radiation_cols], how="inner")
            mdf = mdf[[c for c in cols_multi]]
            for c in list(mdf.columns):
                mdf[c] = _maybe_deseason(rr, c, mdf[c])
            mdf = mdf.dropna().replace([np.inf, -np.inf], np.nan).dropna()
            log(f"[START] VAR baseline {rr} on {len(mdf)} rows, {len(mdf.columns)} vars")
            var_metrics = run_var_baseline(rr, mdf)
            if var_metrics is not None and len(var_metrics):
                # append VAR rows to scoreboard immediately
                for _, row in var_metrics.iterrows():
                    _append_metrics_row(row.to_dict(), scoreboard_path)
                all_metrics.extend(var_metrics.to_dict("records"))
            log(f"[DONE ] VAR baseline {rr}")

# ---------- Save/flush scoreboard (final) ----------
score = pd.DataFrame(all_metrics)
# final de-dupe (just in case of restarts)
if not score.empty:
    if scoreboard_path.exists():
        try:
            prev = pd.read_csv(scoreboard_path)
            score = pd.concat([prev, score], ignore_index=True)
        except Exception:
            pass
    # simple de-dup by subset of columns if present
    keep_cols = [c for c in ["RR","Type","Variable","Model","RMSE","MAE"] if c in score.columns]
    if keep_cols:
        score = score.drop_duplicates(subset=keep_cols, keep="last")
    score.to_csv(scoreboard_path, index=False)

log(f"[✓] Saved scoreboard → {scoreboard_path} ({len(score)} rows)")

# Optional quick peek (Jupyter)
try:
    from IPython.display import display
    display(score.sort_values(["RR","Variable","RMSE"]).head(20))
except Exception:
    pass


[Sec6 bootstrap] Using fallback Settings (S).
[2025-09-13 16:04:40 UTC] [Sec6] TF 2.16.2 · Darwin 24.5.0 · GPUs=1 (no CUDA) → FP32
[2025-09-13 16:04:41 UTC] [Sec6] Jobs queued: 7


Section 6 tasks:   0%|      | 0/7 [00:00<?, ?it/s, RR-1:telemetry:Temp_degC_ISS]

[2025-09-13 16:04:41 UTC] [START] RR-1 telemetry:Temp_degC_ISS  (60455 pts)
[2025-09-13 16:04:42 UTC] [RR-1 telemetry:Temp_degC_ISS] detectors done in 1.4s
[2025-09-13 16:55:12 UTC] AE RR-1:Temp_degC_ISS epoch 1: loss=0.00518 val_loss=0.00003
[2025-09-13 17:46:06 UTC] AE RR-1:Temp_degC_ISS epoch 2: loss=0.00850 val_loss=0.00019
[2025-09-13 17:47:13 UTC] [RR-1 telemetry:Temp_degC_ISS] AE done in 6150.5s
[2025-09-13 18:10:14 UTC] FC RR-1:Temp_degC_ISS epoch 1: loss=0.00507 val_loss=nan
[2025-09-13 18:33:22 UTC] FC RR-1:Temp_degC_ISS epoch 2: loss=0.00027 val_loss=nan
[2025-09-13 18:56:29 UTC] FC RR-1:Temp_degC_ISS epoch 3: loss=0.00024 val_loss=nan
[2025-09-13 18:57:18 UTC] [RR-1 telemetry:Temp_degC_ISS] forecaster done in 4204.6s
[2025-09-13 18:57:18 UTC] [RR-1 telemetry:Temp_degC_ISS] SARIMAX skipped (disabled) | total 10356.7s
[2025-09-13 18:57:18 UTC] [DONE ] RR-1 telemetry:Temp_degC_ISS


Section 6 tasks:  14%|▏| 1/7 [2:52:37<17:15:43, 10357.20s/it, RR-1:telemetry:RH_

[2025-09-13 18:57:19 UTC] [START] RR-1 telemetry:RH_percent_ISS  (60455 pts)
[2025-09-13 18:57:20 UTC] [RR-1 telemetry:RH_percent_ISS] detectors done in 1.2s
[2025-09-13 19:48:17 UTC] AE RR-1:RH_percent_ISS epoch 1: loss=0.05514 val_loss=0.00134
[2025-09-13 20:38:42 UTC] AE RR-1:RH_percent_ISS epoch 2: loss=0.00203 val_loss=0.00056
[2025-09-13 20:39:49 UTC] [RR-1 telemetry:RH_percent_ISS] AE done in 6149.0s
[2025-09-13 21:02:55 UTC] FC RR-1:RH_percent_ISS epoch 1: loss=0.00330 val_loss=nan
[2025-09-13 21:25:57 UTC] FC RR-1:RH_percent_ISS epoch 2: loss=0.00061 val_loss=nan
[2025-09-13 21:48:58 UTC] FC RR-1:RH_percent_ISS epoch 3: loss=0.00060 val_loss=nan
[2025-09-13 21:49:46 UTC] [RR-1 telemetry:RH_percent_ISS] forecaster done in 4197.4s
[2025-09-13 21:49:46 UTC] [RR-1 telemetry:RH_percent_ISS] SARIMAX skipped (disabled) | total 10347.7s
[2025-09-13 21:49:46 UTC] [DONE ] RR-1 telemetry:RH_percent_ISS


...

Section 6 tasks:  86%|▊| 6/7 [16:27:05<2:43:54, 9834.43s/it, RR-1:radiation:Accu

[2025-09-14 08:31:47 UTC] [START] RR-1 radiation:Accumulated_Dose_mGy_d  (54721 pts)
[2025-09-14 08:31:49 UTC] [RR-1 radiation:Accumulated_Dose_mGy_d] detectors done in 2.0s
[2025-09-14 09:18:04 UTC] AE RR-1:Accumulated_Dose_mGy_d epoch 1: loss=0.00000 val_loss=0.00169
[2025-09-14 10:03:56 UTC] AE RR-1:Accumulated_Dose_mGy_d epoch 2: loss=0.00087 val_loss=0.03770
[2025-09-14 10:04:56 UTC] [RR-1 radiation:Accumulated_Dose_mGy_d] AE done in 5587.5s
[2025-09-14 10:25:43 UTC] FC RR-1:Accumulated_Dose_mGy_d epoch 1: loss=0.00000 val_loss=nan
[2025-09-14 10:46:30 UTC] FC RR-1:Accumulated_Dose_mGy_d epoch 2: loss=0.00009 val_loss=nan
[2025-09-14 11:07:20 UTC] FC RR-1:Accumulated_Dose_mGy_d epoch 3: loss=0.00003 val_loss=nan
[2025-09-14 11:08:04 UTC] [RR-1 radiation:Accumulated_Dose_mGy_d] forecaster done in 3787.8s
[2025-09-14 11:08:04 UTC] [RR-1 radiation:Accumulated_Dose_mGy_d] SARIMAX skipped (disabled) | total 9377.4s
[2025-09-14 11:08:04 UTC] [DONE ] RR-1 radiation:Accumulated_Dose_mGy_d

Section 6 tasks: 100%|█| 7/7 [19:03:23<00:00, 9800.52s/it, RR-1:radiation:Accumu


[2025-09-14 11:08:06 UTC] [START] VAR baseline RR-1 on 54700 rows, 7 vars
[2025-09-14 11:08:24 UTC] [VAR BASELINE ERR] RR-1: 'VARResults' object has no attribute 'y'
[2025-09-14 11:08:24 UTC] [DONE ] VAR baseline RR-1
[2025-09-14 11:08:24 UTC] [✓] Saved scoreboard → /Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/outputs/anomaly_forecast/RR-1_forecast_scoreboard.csv (10 rows)


,RR,Type,Variable,Model,RMSE,MAE,AE_Threshold,AE_MeanMSE,AE_Time_s,SeqLen,Resample,Conformal_q_inv
13,RR-1,radiation,Accumulated_Dose_mGy_d,LSTM,0.642128,0.537537,1.286641,0.316788,9375.284692,45,1min,0.329161
9,RR-1,telemetry,CO2_ppm_ISS,LSTM,57.470621,45.486809,0.003686,0.001429,10332.888385,45,1min,110.456475
10,RR-1,radiation,GCR_Dose_mGy_d,LSTM,0.000059,0.000059,0.024436,0.024295,9351.403144,45,1min,0.000069
8,RR-1,telemetry,RH_percent_ISS,LSTM,1.025136,0.843621,0.003064,0.001615,10346.441202,45,1min,1.558475
4,RR-1,radiation,SAA_Dose_mGy_d,LSTM,0.001281,0.000980,0.009078,0.003982,9458.126925,45,1min,0.001238
11,RR-1,radiation,SAA_Dose_mGy_d,LSTM,0.001281,0.000980,0.009078,0.003982,9458.126925,45,1min,0.001238
0,RR-1,telemetry,Temp_degC_ISS,LSTM,0.073985,0.057713,0.000085,0.000020,10355.082372,45,1min,0.093678
7,RR-1,telemetry,Temp_degC_ISS,LSTM,0.073985,0.057713,0.000085,0.000020,10355.082372,45,1min,0.093678
5,RR-1,radiation,Total_Dose_mGy_d,LSTM,0.001362,0.001318,0.022289,0.004682,9370.028767,45,1min,0.001471
12,RR-1,radiation,Total_Dose_mGy_d,LSTM,0.001362,0.001318,0.022289,0.004682,9370.028767,45,1min,0.001471


### Section 7: Evaluation and Insights
#### Purpose
Aggregate results across missions, spotlight volatility/forecastability, and link environmental anomalies to biological findings where timing allows.
#### What is implemented
1. Aggregation of per-mission metrics; top forecastable variables and strongest anomaly intensities.
2. Composite anomaly timeline for RR-1 from z-spikes across key telemetry/radiation.
3. RR-1 omics linkage to GLDS-98, GLDS-99, and GLDS-104: DEGs and PCA summaries; exports to insights directory.
4. Visual summaries: anomaly intensity by type and by mission.
#### Future Work
- Compare missions with pronounced environmental anomalies against relatively stable missions.
- Anomaly calendar per mission with merged telemetry/radiation spike timestamps.
- If event context is available, annotate anomaly calendars with docking/EVA intervals and compute overlap rates.
- Pathway enrichment summaries per anomaly window and concise narrative per mission.

In [4]:
# ==== Section 7: Evaluation & Insights (restart-safe; explicit GLDS file map) ====
# Purpose: Aggregate results across missions, spotlight volatility/forecastability,
# and link environmental anomalies to biological findings where timing allows.

import os, json, warnings, platform
from pathlib import Path
from typing import List, Dict, Tuple, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")

# --- tqdm (quiet if not available) ---
try:
    from tqdm import tqdm as _tqdm
    def tqdm(it, **kw): return _tqdm(it, **kw)
    def _tw(msg): print(msg)
    tqdm.write = _tw
except Exception:
    def tqdm(it, **kw): return it
    tqdm.write = print

# ---------------- Settings bootstrap ----------------
if "S" not in globals():
    from dataclasses import dataclass, field
    @dataclass
    class Settings:
        missions: List[str] = field(default_factory=lambda: ["RR-1","RR-3","RR-6","RR-9","RR-12","RR-19"])
        sampling: str = "1min"
        outputs_root: Path = field(default_factory=lambda: Path.cwd() / "outputs")
        preprocessed_dir: Path = field(default_factory=lambda: Path.cwd() / "outputs" / "preprocessed")
        pattern_dir: Path = field(default_factory=lambda: Path.cwd() / "outputs" / "pattern_analysis")
        anomalies_dir: Path = field(default_factory=lambda: Path.cwd() / "outputs" / "anomaly_forecast")
    S = Settings()
    for d in [S.outputs_root, S.preprocessed_dir, S.pattern_dir, S.anomalies_dir]:
        d.mkdir(parents=True, exist_ok=True)
    print("[Sec7 bootstrap] Using fallback Settings (S).")

# add optional attrs if absent
if not hasattr(S, "relationships_dir"): S.relationships_dir = S.outputs_root / "relationships"
if not hasattr(S, "omics_roots"):       S.omics_roots       = [S.outputs_root / "omics"]
if not hasattr(S, "data_roots"):        S.data_roots        = [S.outputs_root]

ANOM_DIR     = S.anomalies_dir
PATTERN_DIR  = S.pattern_dir
REL_DIR      = S.relationships_dir
INSIGHTS_DIR = S.outputs_root / "insights"
CAL_DIR      = INSIGHTS_DIR / "anomaly_calendars"
PLOTS_DIR    = INSIGHTS_DIR / "plots"
for d in [INSIGHTS_DIR, CAL_DIR, PLOTS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# ---------------- Your GLDS file paths (authoritative) ----------------
# If a listed file is missing, code will try sensible GLbulkRNAseq fallbacks.
GLDS_FILES: Dict[str, Dict[str, Optional[str]]] = {
    "GLDS-98": {
        "contrasts": "/Users/kennethjenkins/data/genelab/GLDS-98_rna_seq_contrasts.csv",
        "diff":      "/Users/kennethjenkins/data/genelab/GLDS-98_rna_seq_differential_expression.csv",
        "counts":    "/Users/kennethjenkins/data/genelab/GLDS-98_rna_seq_Normalized_Counts.csv",
        "sample":    "/Users/kennethjenkins/data/genelab/GLDS-98_rna_seq_SampleTable.csv",
    },
    "GLDS-99": {
        "contrasts": "/Users/kennethjenkins/data/genelab/GLDS-99_rna_seq_contrasts.csv",
        "diff":      "/Users/kennethjenkins/data/genelab/GLDS-99_rna_seq_differential_expression.csv",
        "counts":    "/Users/kennethjenkins/data/genelab/GLDS-99_rna_seq_Normalized_Counts.csv",
        "sample":    "/Users/kennethjenkins/data/genelab/GLDS-99_rna_seq_SampleTable.csv",
    },
    "GLDS-104": {
        "contrasts": "/Users/kennethjenkins/data/genelab/GLDS-104_rna_seq_contrasts_GLbulkRNAseq.csv",
        "diff":      "/Users/kennethjenkins/data/genelab/GLDS-104_rna_seq_differential_expression.csv",
        "counts":    "/Users/kennethjenkins/data/genelab/GLDS-104_rna_seq_Normalized_Counts.csv",
        "sample":    "/Users/kennethjenkins/data/genelab/GLDS-104_rna_seq_SampleTable_GLbulkRNAseq.csv",
    },
}

# Fallback filename patterns for GLDS-104 (post-2025 updates), used if the explicit path above is missing.
GLDS_104_ALTS = {
    "diff": [
        "/Users/kennethjenkins/data/genelab/GLDS-104_rna_seq_differential_expression_GLbulkRNAseq.csv",
        "/Users/kennethjenkins/data/genelab/GLDS-104_rna_seq_differential_expression_rRNArm_GLbulkRNAseq.csv",
    ],
    "counts": [
        "/Users/kennethjenkins/data/genelab/GLDS-104_rna_seq_Normalized_Counts_GLbulkRNAseq.csv",
        "/Users/kennethjenkins/data/genelab/GLDS-104_rna_seq_Normalized_Counts_rRNArm_GLbulkRNAseq.csv",
        "/Users/kennethjenkins/data/genelab/GLDS-104_rna_seq_VST_Counts_GLbulkRNAseq.csv",
        "/Users/kennethjenkins/data/genelab/GLDS-104_rna_seq_VST_Counts_rRNArm_GLbulkRNAseq.csv",
    ],
    "sample": [
        "/Users/kennethjenkins/data/genelab/GLDS-104_rna_seq_SampleTable_GLbulkRNAseq.csv",
    ],
    "contrasts": [
        "/Users/kennethjenkins/data/genelab/GLDS-104_rna_seq_contrasts_GLbulkRNAseq.csv",
    ],
}

# ---------------- Utility: logging ----------------
from datetime import datetime
def log(msg: str):
    ts = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{ts} UTC] {msg}", flush=True)

# ---------------- IO helpers for telemetry/radiation ----------------
def _load_preprocessed(rr: str, kind: str) -> Optional[pd.DataFrame]:
    try:
        if kind == "telemetry":
            if "preprocessed_telemetry" in globals() and rr in preprocessed_telemetry:
                return preprocessed_telemetry[rr].copy()
            p = S.preprocessed_dir / f"{rr}_cleaned_telemetry.csv"
        else:
            if "cleaned_radiation" in globals() and rr in cleaned_radiation:
                return cleaned_radiation[rr].copy()
            p = S.preprocessed_dir / f"{rr}_cleaned_radiation.csv"
        if p.exists():
            return pd.read_csv(p, index_col=0, parse_dates=True).sort_index()
    except Exception as e:
        log(f"[LOAD FAIL] {rr} {kind}: {e}")
    return None

# ---------------- Section 6 results collection ----------------
scoreboards   = list(ANOM_DIR.glob("*_forecast_scoreboard.csv"))
rr_summaries  = list(ANOM_DIR.glob("**/*_forecast_summary.csv"))

dfs = []
if scoreboards:
    for p in scoreboards:
        try:
            dfs.append(pd.read_csv(p))
        except Exception as e:
            log(f"[WARN] Could not read scoreboard {p.name}: {e}")
if rr_summaries:
    for p in rr_summaries:
        try:
            df = pd.read_csv(p)
            if "RR" not in df.columns:
                df["RR"] = p.name.split("_")[0]
            dfs.append(df)
        except Exception as e:
            log(f"[WARN] Could not read summary {p.name}: {e}")

if not dfs:
    raise RuntimeError("No Section 6 results found in outputs. Run Section 6 first.")

metrics_all = pd.concat(dfs, ignore_index=True)
metrics_all["RR"] = metrics_all["RR"].astype(str)
(INSIGHTS_DIR / "all_missions_metrics_raw.csv").parent.mkdir(parents=True, exist_ok=True)
metrics_all.to_csv(INSIGHTS_DIR / "all_missions_metrics_raw.csv", index=False)
log(f"[Sec7] Loaded metrics rows: {len(metrics_all)}")

# ---------------- Seasonality (optional) ----------------
seas_path = PATTERN_DIR / "seasonality_summary.csv"
seasonality_df = pd.read_csv(seas_path) if seas_path.exists() else None

telemetry_cols = ["Temp_degC_ISS", "RH_percent_ISS", "CO2_ppm_ISS"]
radiation_cols = ["GCR_Dose_mGy_d", "SAA_Dose_mGy_d", "Total_Dose_mGy_d", "Accumulated_Dose_mGy_d"]

def _mission_volatility(rr: str) -> pd.DataFrame:
    out = []
    for kind, cols in [("telemetry", telemetry_cols), ("radiation", radiation_cols)]:
        df = _load_preprocessed(rr, kind)
        if df is None: 
            continue
        for c in cols:
            if c in df.columns:
                s = df[c].dropna()
                out.append({"RR": rr, "Type": kind, "Variable": c, "Std": float(s.std())})
    return pd.DataFrame(out)

def _anom_flags(series: pd.Series, z_thr=3.0, ewma_span=60) -> pd.DataFrame:
    s = series.dropna()
    if len(s) == 0 or s.std() == 0: 
        return pd.DataFrame(columns=["timestamp","atype"])
    z = (s - s.mean())/s.std()
    z_mask = z.abs() > z_thr
    ew = s.ewm(span=ewma_span).mean()
    resid = s - ew
    ew_mask = (resid.abs() > (2*resid.std()))
    out = []
    if z_mask.any():   out.extend([(t, "z") for t in s.index[z_mask]])
    if ew_mask.any():  out.extend([(t, "ewma") for t in s.index[ew_mask]])
    if not out:
        return pd.DataFrame(columns=["timestamp","atype"])
    df = pd.DataFrame(out, columns=["timestamp","atype"]).drop_duplicates().sort_values("timestamp")
    return df

def _mission_anomaly_calendar(rr: str) -> pd.DataFrame:
    rows = []
    for kind, cols in [("telemetry", telemetry_cols), ("radiation", radiation_cols)]:
        df = _load_preprocessed(rr, kind)
        if df is None: 
            continue
        for c in cols:
            if c not in df.columns: 
                continue
            y = df[c].dropna()
            if len(y) == 0 or y.std() == 0:
                continue
            y = y.resample(S.sampling).mean()
            flags = _anom_flags(y, z_thr=3.0, ewma_span=60)
            if len(flags):
                flags["RR"] = rr
                flags["Variable"] = c
                rows.append(flags)
    if not rows:
        return pd.DataFrame(columns=["timestamp","atype","RR","Variable"])
    cal = pd.concat(rows, ignore_index=True).drop_duplicates().sort_values("timestamp")
    return cal

# Build volatility & anomaly-calendars
vol_tabs, calendars = [], []
for rr in tqdm(S.missions, desc="Volatility/Calendar"):
    vt = _mission_volatility(rr)
    if len(vt): vol_tabs.append(vt)
    cal = _mission_anomaly_calendar(rr)
    if len(cal): calendars.append(cal.to_dict("records"))

vol_df = pd.concat(vol_tabs, ignore_index=True) if vol_tabs else pd.DataFrame(columns=["RR","Type","Variable","Std"])
cal_all = pd.DataFrame([r for mission in calendars for r in mission]) if calendars else pd.DataFrame(columns=["timestamp","atype","RR","Variable"])
vol_df.to_csv(INSIGHTS_DIR / "volatility_by_variable.csv", index=False)
cal_all.to_csv(INSIGHTS_DIR / "anomaly_calendar_all_missions.csv", index=False)

# ---------------- Event overlap (optional, if events CSVs exist) ----------------
def _scan_event_table(rr: str) -> Optional[pd.DataFrame]:
    candidates = list((S.outputs_root / "events").glob(f"*{rr}*events*.csv"))
    for root in S.data_roots:
        root = Path(root)
        candidates.extend(root.glob(f"*{rr}*events*.csv"))
        candidates.extend(root.glob("events/*events*.csv"))
    for p in candidates:
        try:
            df = pd.read_csv(p)
            cols = {c.lower(): c for c in df.columns}
            start = df[cols.get("start", next((c for c in df.columns if "start" in c.lower()), None))]
            end   = df[cols.get("end",   next((c for c in df.columns if "end"   in c.lower()), None))]
            labc  = cols.get("label", next((c for c in df.columns if "label" in c.lower() or "event" in c.lower()), None))
            lab = df[labc] if labc else pd.Series(["event"]*len(df))
            ev = pd.DataFrame({
                "start": pd.to_datetime(start, utc=True, errors="coerce"),
                "end":   pd.to_datetime(end,   utc=True, errors="coerce"),
                "label": lab.astype(str)
            }).dropna()
            if len(ev):
                return ev.sort_values("start")
        except Exception:
            continue
    return None

overlap_rows = []
for rr in cal_all["RR"].unique():
    ev = _scan_event_table(rr)
    if ev is None: 
        continue
    c = cal_all[cal_all["RR"] == rr].copy()
    for _, row in c.iterrows():
        t = pd.to_datetime(row["timestamp"], utc=True)
        hit = ev[(ev["start"] <= t) & (t <= ev["end"])]
        if len(hit):
            for _, h in hit.iterrows():
                overlap_rows.append({"RR": rr, "timestamp": t, "Variable": row["Variable"],
                                     "atype": row["atype"], "event": h["label"]})
if overlap_rows:
    overlap_df = pd.DataFrame(overlap_rows).sort_values(["RR","timestamp"])
    overlap_df.to_csv(INSIGHTS_DIR / "anomaly_event_overlaps.csv", index=False)
    log(f"[Sec7] Event overlap rows: {len(overlap_df)}")
else:
    log("[Sec7] No event context found; overlap skipped.")

# ---------------- Cross-mission summaries ----------------
if len(cal_all):
    rate = cal_all.groupby("RR")["timestamp"].nunique().rename("Anomaly_Timestamps").reset_index()
    norm_minutes = []
    for rr in rate["RR"]:
        mins = 0
        for kind in ("telemetry","radiation"):
            df = _load_preprocessed(rr, kind)
            if df is None or len(df) == 0: 
                continue
            i = df.index.sort_values()
            mins = max(mins, int((i.max() - i.min()).total_seconds() // 60))
        norm_minutes.append({"RR": rr, "Minutes": max(1, mins)})
    norm_df = pd.DataFrame(norm_minutes)
    rate = rate.merge(norm_df, on="RR", how="left")
    rate["Anomaly_per_10kmin"] = 1e4 * rate["Anomaly_Timestamps"] / rate["Minutes"].clip(lower=1)
    q_hi = rate["Anomaly_per_10kmin"].quantile(0.67)
    q_lo = rate["Anomaly_per_10kmin"].quantile(0.33)
    def _lab(x):
        if x >= q_hi: return "anomalous"
        if x <= q_lo: return "stable"
        return "middle"
    rate["Group"] = rate["Anomaly_per_10kmin"].apply(_lab)
    rate.to_csv(INSIGHTS_DIR / "mission_anomaly_rates.csv", index=False)

    err_m = metrics_all.groupby(["RR","Variable"], as_index=False)[["RMSE","MAE"]].mean()
    vol_m = vol_df[["RR","Variable","Std"]]
    comp = (err_m.merge(vol_m, on=["RR","Variable"], how="left")
                  .merge(rate[["RR","Group"]], on="RR", how="left"))
    comp.to_csv(INSIGHTS_DIR / "group_comparison_metrics.csv", index=False)
else:
    comp = pd.DataFrame()

# ---- “Top” tables & plots ----
top_forecast = (metrics_all.sort_values(["RR","RMSE"])
                .groupby("RR", as_index=False).head(3))
top_forecast.to_csv(INSIGHTS_DIR / "top_forecastable_by_mission.csv", index=False)

if "AE_MeanMSE" in metrics_all.columns:
    top_anom = (metrics_all.sort_values(["RR","AE_MeanMSE"], ascending=[True, False])
                .groupby("RR", as_index=False).head(3))
    top_anom.to_csv(INSIGHTS_DIR / "top_anomalous_by_mission.csv", index=False)

if seasonality_df is not None and not seasonality_df.empty:
    seas_keep = seasonality_df.rename(columns={"Mission":"RR",
                                               "Seasonality_Strength":"Seasonality_Strength"})
    best_seasonal = (seas_keep.sort_values(["RR","Seasonality_Strength"], ascending=[True, False])
                             .groupby("RR", as_index=False).head(3))
    best_seasonal.to_csv(INSIGHTS_DIR / "top_seasonality_by_mission.csv", index=False)

# ---------------- OMICS helpers ----------------
def _safe_read_csv(p: Optional[str]) -> Optional[pd.DataFrame]:
    if p is None: return None
    try:
        return pd.read_csv(p)
    except Exception:
        try:
            return pd.read_csv(p, encoding="utf-8", engine="python")
        except Exception as e:
            log(f"[OMICS WARN] Could not read {p}: {e}")
            return None

def _first_existing(paths: List[str]) -> Optional[str]:
    for p in paths:
        if p and Path(p).exists():
            return p
    return None

def _pick_contrast_suffix(cols: List[str], contrasts_df: Optional[pd.DataFrame]) -> Tuple[Optional[str], Optional[str]]:
    low = [c.lower() for c in cols]
    def _find_suffix(keywords):
        for c in cols:
            cl = c.lower()
            if all(k in cl for k in keywords):
                if '.' in c:
                    return c[c.rfind('.'):]  # ".FLT_vs_GC"
                return ""                   # already specific
        return None
    for kws in (["flight","ground"], ["flt","gc"], ["space","ground"]):
        suf = _find_suffix(kws)
        if suf is not None:
            return suf, suf
    if contrasts_df is not None and len(contrasts_df.columns):
        suf = _find_suffix(["flt","gc"]) or _find_suffix(["flight","ground"])
        return suf, suf
    return None, None

def _normalize_deg_columns(df: pd.DataFrame, contrasts_df: Optional[pd.DataFrame]) -> Optional[pd.DataFrame]:
    if df is None or df.empty:
        return None
    d = df.copy()
    d.columns = (d.columns
                   .str.strip()
                   .str.replace("\u200b","", regex=False)
                   .str.replace("\xa0","", regex=False))
    cols = list(d.columns)

    # gene column
    gene_cands = [c for c in cols if c.lower() in ("symbol","gene","genes","gene_symbol","gene name","gene_id","ensembl","ensembl_gene_id")]
    gene_col = gene_cands[0] if gene_cands else cols[0]

    # padj/lfc candidates
    padj_like = [c for c in cols if ("padj" in c.lower()) or ("fdr" in c.lower()) or ("qval" in c.lower()) or ("adj.p.value" in c.lower()) or ("fdr p-value" in c.lower())]
    lfc_like  = [c for c in cols if ("log2" in c.lower() and ("fc" in c.lower() or "fold" in c.lower())) or ("logfc" in c.lower()) or ("estimate" in c.lower())]

    suf_p, suf_l = _pick_contrast_suffix(cols, contrasts_df)

    padj_col = None
    lfc_col  = None

    if suf_p or suf_l:
        for c in padj_like:
            if suf_p and c.endswith(suf_p): padj_col = c; break
        for c in lfc_like:
            if suf_l and c.endswith(suf_l): lfc_col = c; break

    def _prefer(cols_list, prefer_keywords):
        ranked = []
        for c in cols_list:
            score = sum(k in c.lower() for k in prefer_keywords)
            ranked.append((score, c))
        ranked.sort(reverse=True)
        return ranked[0][1] if ranked else None

    if padj_col is None:
        padj_col = _prefer(padj_like, ["padj","fdr","qval","adj","fdr p-value","adjusted"])
    if lfc_col is None:
        lfc_col = _prefer(lfc_like, ["log2foldchange","log2fc","logfc","fold","estimate"])

    keep = [gene_col] + [c for c in [padj_col, lfc_col] if c is not None]
    d = d[keep].copy()

    rename_map = {gene_col: "gene"}
    if padj_col: rename_map[padj_col] = "padj"
    if lfc_col:  rename_map[lfc_col]  = "log2FoldChange"
    d = d.rename(columns=rename_map)

    for c in ["padj","log2FoldChange"]:
        if c in d.columns:
            d[c] = pd.to_numeric(d[c], errors="coerce")

    if ("log2FoldChange" not in d.columns) and ("padj" not in d.columns):
        return None
    return d

def _write_top_tables(glds: str, d: pd.DataFrame):
    if "log2FoldChange" in d.columns:
        if "padj" in d.columns:
            sig = d[d["padj"] < 0.05].copy()
        else:
            sig = d.assign(absfc=d["log2FoldChange"].abs()).sort_values("absfc", ascending=False).head(500).copy()
        up   = sig.sort_values("log2FoldChange", ascending=False).head(10)[["gene","log2FoldChange"]]
        down = sig.sort_values("log2FoldChange", ascending=True ).head(10)[["gene","log2FoldChange"]]
        up.to_csv(INSIGHTS_DIR / f"{glds}_top_upregulated.csv", index=False)
        down.to_csv(INSIGHTS_DIR / f"{glds}_top_downregulated.csv", index=False)
    else:
        best = d.sort_values("padj").head(20)
        best.to_csv(INSIGHTS_DIR / f"{glds}_most_significant.csv", index=False)

def _best_sample_id_column(sample: pd.DataFrame, sample_names: List[str]) -> Optional[str]:
    cand = None
    best = -1
    sset = set(map(str, sample_names))
    for c in sample.columns:
        vals = set(map(str, sample[c].astype(str).values))
        score = len(sset & vals)
        if score > best:
            best = score; cand = c
    return cand if best > 0 else None

def _make_pca_table(glds: str, counts: pd.DataFrame, sample: Optional[pd.DataFrame]) -> Optional[pd.DataFrame]:
    if counts is None or counts.empty:
        return None
    X = counts.copy()

    # If a gene column survived, move it to index
    if "gene" in X.columns:
        X = X.set_index("gene")

    # Keep only numeric (samples should be columns)
    X = X.select_dtypes(include=[np.number])
    if X.empty:
        return None

    # Heuristic: if columns >> rows, genes likely rows -> transpose to samples x genes
    if X.shape[1] > X.shape[0]:
        X = X.T

    # Log1p if values large
    if np.nanmedian(X.values) > 50:
        X = np.log1p(X)

    # Standardize features (genes)
    X = (X - X.mean(axis=0)) / (X.std(axis=0) + 1e-8)
    X = X.replace([np.inf, -np.inf], np.nan).fillna(0.0)

    # PCA(2)
    try:
        from sklearn.decomposition import PCA
        pca = PCA(n_components=2, random_state=0)
        coords = pca.fit_transform(X.values)  # samples x 2
    except Exception:
        # Fallback to SVD-based PCA (2 comps)
        U, Ssvd, Vt = np.linalg.svd(X.values - X.values.mean(axis=0), full_matrices=False)
        coords = U[:, :2] * Ssvd[:2]

    out = pd.DataFrame({"PC1": coords[:,0], "PC2": coords[:,1], "Sample": X.index.astype(str)})

    # Attach group/condition if possible
    if sample is not None and not sample.empty:
        s = sample.copy()
        s.columns = s.columns.str.strip()
        id_col = _best_sample_id_column(s, list(out["Sample"]))
        group_col = None
        for cand in ["Group","Condition","group","condition","SampleGroup","Treatment","Sample_Group"]:
            if cand in s.columns:
                group_col = cand; break
        if id_col and group_col:
            meta = s[[id_col, group_col]].rename(columns={id_col: "Sample", group_col: "Group"})
            out = out.merge(meta, on="Sample", how="left")

    out.to_csv(INSIGHTS_DIR / f"{glds}_pca_table.csv", index=False)
    log(f"[OMICS] {glds}: PCA table -> {INSIGHTS_DIR / f'{glds}_pca_table.csv'}")
    return out

# ---------------- OMICS ingest with fallbacks ----------------
def _omics_ingest(gl_map: Dict[str, Dict[str, Optional[str]]]) -> Dict[str, Dict[str, pd.DataFrame]]:
    out: Dict[str, Dict[str, pd.DataFrame]] = {}
    for glds, files in gl_map.items():
        # Resolve paths (with GLDS-104 fallbacks if needed)
        f_diff   = files.get("diff")
        f_counts = files.get("counts")
        f_sample = files.get("sample")
        f_contr  = files.get("contrasts")

        if glds == "GLDS-104":
            if not (f_diff and Path(f_diff).exists()):
                f_diff = _first_existing(GLDS_104_ALTS["diff"])
            if not (f_counts and Path(f_counts).exists()):
                f_counts = _first_existing(GLDS_104_ALTS["counts"])
            if not (f_sample and Path(f_sample).exists()):
                f_sample = _first_existing(GLDS_104_ALTS["sample"])
            if not (f_contr and Path(f_contr).exists()):
                f_contr = _first_existing(GLDS_104_ALTS["contrasts"])

        diff   = _safe_read_csv(f_diff)
        counts = _safe_read_csv(f_counts)
        sample = _safe_read_csv(f_sample)
        contr  = _safe_read_csv(f_contr)

        pack: Dict[str, pd.DataFrame] = {}
        if diff is not None:
            pack["diff_raw"] = diff.copy()
            dnorm = _normalize_deg_columns(diff, contr)
            if dnorm is not None:
                pack["diff"] = dnorm
        if counts is not None:
            c = counts.copy()
            # promote a "gene" column to index if present
            gcol = next((col for col in c.columns if col.lower() in {"gene","symbol","genes"}), None)
            if gcol:
                c = c.set_index(gcol)
            pack["counts"] = c
        if sample is not None:
            pack["sample"] = sample
        if contr is not None:
            pack["contrasts"] = contr

        out[glds] = pack
        have = ", ".join(pack.keys()) if pack else "none"
        log(f"[Sec7] OMICS loaded {glds}: {have}")
    return out

omics_data = _omics_ingest(GLDS_FILES)  # global; later sections can reuse

# ---------------- OMICS exports (tops + PCA) ----------------
omics_out_rows = []
for glds, dd in omics_data.items():
    if "diff" not in dd: 
        log(f"[OMICS] {glds}: Could not resolve DEG columns; check headers.")
    else:
        d = dd["diff"]
        _write_top_tables(glds, d)
        n_sig = int((d["padj"] < 0.05).sum()) if "padj" in d.columns else int(len(d))
        omics_out_rows.append({"Dataset": glds, "Rows_in_DEG": int(len(d)), "DEGs_significant_q<0.05": n_sig})

    if "counts" in dd:
        _make_pca_table(glds, dd["counts"], dd.get("sample"))

if omics_out_rows:
    pd.DataFrame(omics_out_rows).to_csv(INSIGHTS_DIR / "omics_datasets_summary.csv", index=False)

# ---------------- Simple ORA if a GMT exists ----------------
def _find_gmt() -> Optional[Path]:
    candidates = []
    for root in (list(S.omics_roots) + [S.outputs_root, S.outputs_root/"pathways"]):
        root = Path(root)
        for p in [root/"gene_sets.gmt", root/"pathways.gmt"]:
            if p.exists(): candidates.append(p)
        candidates.extend(list(root.glob("**/*.gmt")))
    return candidates[0] if candidates else None

def _load_gmt(p: Path) -> Dict[str, set]:
    gs = {}
    with open(p, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            parts = line.strip().split("\t")
            if len(parts) >= 3:
                gs[parts[0]] = set([g for g in parts[2:] if g])
    return gs

def _ora(sig_genes: set, universe: set, gene_sets: Dict[str, set], topn=20) -> pd.DataFrame:
    try:
        from scipy.stats import fisher_exact
        fisher_ok = True
    except Exception:
        fisher_ok = False
    rows = []
    for term, members in gene_sets.items():
        a = len(sig_genes & members)
        b = len(sig_genes - members)
        c = len((universe & members) - sig_genes)
        d = len(universe - members - sig_genes)
        p = 1.0
        if fisher_ok:
            try:
                _, p = __import__("scipy.stats").stats.fisher_exact([[a, b],[c, d]], alternative="greater")
            except Exception:
                p = 1.0/(1+a)
        else:
            p = 1.0/(1+a)
        rows.append({"term": term, "overlap": a, "pvalue": p})
    return pd.DataFrame(rows).sort_values("pvalue").head(topn)

gmt = _find_gmt()
if gmt:
    log(f"[Sec7] Pathways GMT: {gmt}")
    gs = _load_gmt(gmt)
    for glds, dd in omics_data.items():
        d = dd.get("diff")
        if d is None or "gene" not in d.columns:
            continue
        if "padj" in d.columns:
            sig = set(d.loc[d["padj"] < 0.05, "gene"].dropna().astype(str))
        else:
            if "log2FoldChange" not in d.columns: 
                continue
            sig = set(d.assign(absfc=d["log2FoldChange"].abs())
                        .sort_values("absfc", ascending=False).head(200)["gene"].astype(str))
        universe = set(d["gene"].dropna().astype(str))
        ora = _ora(sig, universe, gs, topn=20)
        ora.to_csv(INSIGHTS_DIR / f"{glds}_pathway_enrichment.csv", index=False)
else:
    log("[Sec7] No GMT pathway file found; enrichment skipped.")

# ---------------- Concise mission narratives ----------------
def _mission_narrative(rr: str) -> str:
    parts = [f"# {rr} — Mission Insights"]
    mins = 0
    for k in ("telemetry","radiation"):
        df = _load_preprocessed(rr, k)
        if df is not None and len(df):
            i = df.index.sort_values()
            mins = max(mins, int((i.max() - i.min()).total_seconds() // 60))
    parts.append(f"- Duration (approx): {mins:,} minutes")
    tf = top_forecast[top_forecast["RR"] == rr]
    if len(tf):
        v = tf.sort_values("RMSE").iloc[0]
        parts.append(f"- Best forecasted variable: {v['Variable']} (RMSE={v['RMSE']:.2f}, MAE={v['MAE']:.2f})")
    if "AE_MeanMSE" in metrics_all.columns:
        ta = metrics_all[metrics_all["RR"] == rr]
        if len(ta):
            mv = ta.sort_values("AE_MeanMSE", ascending=False).iloc[0]
            parts.append(f"- Strongest anomaly signal: {mv['Variable']} (AE_MeanMSE={mv['AE_MeanMSE']:.3f})")
    if seasonality_df is not None:
        sd = seasonality_df[seasonality_df["Mission"] == rr]
        if len(sd):
            srow = sd.sort_values("Seasonality_Strength", ascending=False).iloc[0]
            parts.append(f"- Strongest seasonality: {srow['Variable']} (strength={srow['Seasonality_Strength']:.3f})")
    if 'rate' in locals() and rr in rate["RR"].values:
        g = rate.loc[rate["RR"] == rr, "Group"].values[0]
        apr = rate.loc[rate["RR"] == rr, "Anomaly_per_10kmin"].values[0]
        parts.append(f"- Anomaly burden: {g} (≈{apr:.1f} timestamps per 10k minutes)")
    return "\n".join(parts) + "\n"

for rr in S.missions:
    (INSIGHTS_DIR / f"{rr}_narrative.md").write_text(_mission_narrative(rr))

# ---------------- Dashboard manifest scaffold ----------------
artifact = {
    "missions": S.missions,
    "scoreboard_csv": str((INSIGHTS_DIR / "all_missions_metrics_raw.csv").resolve()),
    "volatility_csv": str((INSIGHTS_DIR / "volatility_by_variable.csv").resolve()),
    "anomaly_calendar_csv": str((INSIGHTS_DIR / "anomaly_calendar_all_missions.csv").resolve()),
    "anomaly_rates_csv": str((INSIGHTS_DIR / "mission_anomaly_rates.csv").resolve()) if len(cal_all) else None,
    "group_comparison_csv": str((INSIGHTS_DIR / "group_comparison_metrics.csv").resolve()) if 'comp' in locals() and len(comp) else None,
    "top_forecast_csv": str((INSIGHTS_DIR / "top_forecastable_by_mission.csv").resolve()),
    "top_anomalous_csv": str((INSIGHTS_DIR / "top_anomalous_by_mission.csv").resolve()) if "AE_MeanMSE" in metrics_all.columns else None,
}
with open(INSIGHTS_DIR / "insights_manifest.json", "w") as f:
    json.dump(artifact, f, indent=2)

print("\n[✓] Section 7 complete.")
print(f"Outputs → {INSIGHTS_DIR}")


[2025-09-14 16:53:05 UTC] [Sec7] Loaded metrics rows: 10


Volatility/Calendar: 100%|████████████████████████| 6/6 [00:08<00:00,  1.40s/it]


[2025-09-14 16:53:13 UTC] [Sec7] No event context found; overlap skipped.
[2025-09-14 16:53:17 UTC] [Sec7] OMICS loaded GLDS-98: diff_raw, diff, counts, sample, contrasts
[2025-09-14 16:53:18 UTC] [Sec7] OMICS loaded GLDS-99: diff_raw, diff, counts, sample, contrasts
[2025-09-14 16:53:18 UTC] [Sec7] OMICS loaded GLDS-104: diff_raw, diff, counts, sample, contrasts
[2025-09-14 16:53:18 UTC] [OMICS] GLDS-98: PCA table -> /Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/outputs/insights/GLDS-98_pca_table.csv
[2025-09-14 16:53:18 UTC] [OMICS] GLDS-99: PCA table -> /Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/outputs/insights/GLDS-99_pca_table.csv
[2025-09-14 16:53:18 UTC] [OMICS] GLDS-104: PCA table -> /Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/outputs/insights/GLDS-104_pca_table.csv
[2025-09-14 16:53:18 UTC] [Sec7] No GMT pathway file found; enrichment skipped.

[✓] Section 7 complete.
Outputs → /Users/kennethjenkins/Cit

### Section 8: Prescriptive Insights
#### Purpose
Translate analytics into decision aids for operations: alert thresholds, scheduling guidance, and sampling triggers.
#### What is implemented
1. Preliminary thresholds for CO₂/RH/Temp grounded in anomaly behavior and forecast accuracy.
2. Radiation-aware scheduling guidance (SAA variability, cumulative dose).
3. Triggers for omics/physiological sampling aligned to anomaly criteria.
4. Policy export to JSON and optional compliance summaries for the dashboard.
#### Future Work
- Mark prescriptions as preliminary and revise after cross-mission Seasonal ARIMA benchmarking.
- Add placeholders for LSDA physiological endpoints once access is available.
- Calibrate thresholds using ROC curves once a labeled set exists.
- Unit-tested utilities to compute rolling compliance against the policy per mission.

In [5]:
# ==== Section 8: Prescriptive Insights ====
import os, json, warnings
from pathlib import Path
from datetime import datetime, timezone
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

# ---------- Settings bootstrap ----------
if "S" not in globals():
    from dataclasses import dataclass, field
    @dataclass
    class Settings:
        missions: list[str] = field(default_factory=lambda: ["RR-1","RR-3","RR-6","RR-9","RR-12","RR-19"])
        sampling: str = "1min"
        outputs_root: Path = field(default_factory=lambda: Path.cwd() / "outputs")
        preprocessed_dir: Path = field(default_factory=lambda: Path.cwd() / "outputs" / "preprocessed")
        anomalies_dir: Path = field(default_factory=lambda: Path.cwd() / "outputs" / "anomaly_forecast")
    S = Settings()
    for d in [S.outputs_root, S.preprocessed_dir, S.anomalies_dir]:
        d.mkdir(parents=True, exist_ok=True)
    print("[Sec8 bootstrap] Using fallback Settings (S).")

# ---------- Paths ----------
POLICY_DIR = S.outputs_root / "policy"
COMPLIANCE_DIR = POLICY_DIR / "compliance"
PLOTS_DIR = POLICY_DIR / "plots"
INSIGHTS_DIR = S.outputs_root / "insights"
for d in [POLICY_DIR, COMPLIANCE_DIR, PLOTS_DIR, INSIGHTS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

telemetry_cols = ["Temp_degC_ISS", "RH_percent_ISS", "CO2_ppm_ISS"]
radiation_cols = ["GCR_Dose_mGy_d", "SAA_Dose_mGy_d", "Total_Dose_mGy_d", "Accumulated_Dose_mGy_d"]

# ---------- Helpers ----------
def _load_mission(kind: str, rr: str) -> pd.DataFrame | None:
    try:
        if kind == "telemetry":
            if "preprocessed_telemetry" in globals() and rr in preprocessed_telemetry:
                return preprocessed_telemetry[rr].copy()
            p = S.preprocessed_dir / f"{rr}_cleaned_telemetry.csv"
        else:
            if "cleaned_radiation" in globals() and rr in cleaned_radiation:
                return cleaned_radiation[rr].copy()
            p = S.preprocessed_dir / f"{rr}_cleaned_radiation.csv"
        if p.exists():
            return pd.read_csv(p, index_col=0, parse_dates=True).sort_index()
    except Exception:
        pass
    return None

def _q(s: pd.Series, q): 
    s = s.dropna() if s is not None else pd.Series(dtype=float)
    return float(np.nanpercentile(s.values, q)) if len(s) else np.nan

def _hour_pref_list(co2_series: pd.Series) -> list[int]:
    """Return hours (UTC) when CO2 is characteristically low."""
    if co2_series is None or co2_series.empty:
        return []
    x = co2_series.dropna().resample(S.sampling).mean()
    if x.empty: return []
    h = x.groupby(x.index.hour).mean()
    if h.empty: return []
    thr = h.median() - 0.5*h.std(ddof=0)
    hours = [int(k) for k, v in h.items() if v <= thr]
    return sorted(hours)

def _load_metrics_all() -> pd.DataFrame | None:
    p = INSIGHTS_DIR / "all_missions_metrics_raw.csv"  # written by Section 7
    if p.exists():
        try: return pd.read_csv(p)
        except Exception: pass
    # fallback: look for per-mission summaries
    rows = []
    for rr in S.missions:
        p2 = S.anomalies_dir / f"{rr}_forecast_summary.csv"
        if p2.exists():
            try:
                df = pd.read_csv(p2)
                if "RR" not in df.columns: df["RR"] = rr
                rows.append(df)
            except Exception: pass
    return pd.concat(rows, ignore_index=True) if rows else None

def _ae_ref_value(rr: str, var: str, default=0.95) -> float:
    """
    Pull a reference anomaly score from metrics:
    If AE_MeanMSE exists, use the 95th percentile of that metric for (rr,var),
    else return a default quantile value for downstream logic.
    """
    df = _load_metrics_all()
    if df is None: return default
    need = {"RR","Variable","AE_MeanMSE"}
    if not need.issubset(df.columns): return default
    sub = df[(df["RR"].astype(str)==str(rr)) & (df["Variable"].astype(str)==str(var))]
    if not len(sub): return default
    try:
        return float(sub["AE_MeanMSE"].quantile(0.95))
    except Exception:
        return default

# ---------- Build per-mission policy ----------
def _policy_for_mission(rr: str) -> dict:
    tel = _load_mission("telemetry", rr)
    rad = _load_mission("radiation", rr)
    pol = {"thresholds": {}, "radiation": {}, "sampling_triggers": {}, "forecasting": {}, "scheduling": {}}

    # Telemetry thresholds & scheduling prefs
    if tel is not None and len(tel):
        if "CO2_ppm_ISS" in tel.columns:
            co2 = tel["CO2_ppm_ISS"].resample(S.sampling).mean()
            pol["thresholds"]["CO2_ppm_ISS"] = {"hi": _q(co2, 95)}
            pol["scheduling"]["prefer_hours_utc_low_CO2"] = _hour_pref_list(co2)
        if "Temp_degC_ISS" in tel.columns:
            t = tel["Temp_degC_ISS"].resample(S.sampling).mean()
            pol["thresholds"]["Temp_degC_ISS"] = {"lo": _q(t, 5), "hi": _q(t, 95)}
        if "RH_percent_ISS" in tel.columns:
            rh = tel["RH_percent_ISS"].resample(S.sampling).mean()
            pol["thresholds"]["RH_percent_ISS"] = {"lo": _q(rh, 5), "hi": _q(rh, 95)}

    # Radiation guidance (avoid high SAA windows; monitor cumulative increases)
    if rad is not None and len(rad):
        if "SAA_Dose_mGy_d" in rad.columns:
            saa = rad["SAA_Dose_mGy_d"].resample(S.sampling).ffill()
            pol["radiation"]["avoid_when"] = {"SAA_Dose_mGy_d": {"hi": _q(saa, 90), "sustain_minutes": 30}}
        if "Accumulated_Dose_mGy_d" in rad.columns:
            acc_delta = rad["Accumulated_Dose_mGy_d"].resample("60min").max().diff()
            pol["radiation"]["dose_monitor"] = {"Accumulated_Dose_mGy_d_delta": {"window_min": 1440, "hi": _q(acc_delta, 95)}}

    # Sampling triggers (aligned to anomaly detection suite)
    pol["sampling_triggers"] = {
        "omics": {"any_anomaly_minutes": 15, "ae_error_ref": 0.95},  # ae_error_ref replaced below per variable
        "physiology_placeholders": ["corticosterone", "bone_density", "muscle_mass"]
    }
    pol["forecasting"] = {
        "update_interval_min": 60,
        "model_baselines": ["LSTM_univariate", "SARIMAX_seasonal", "VAR_multivariate"]
    }
    return pol

def build_policy() -> dict:
    policy = {
        "meta": {
            "version": "0.3.0",
            "generated_utc": datetime.now(timezone.utc).isoformat(),
            "preliminary": True,
            "revision_pending": "Cross-mission SARIMAX benchmarking",
            "notes": "Percentile-derived bounds; tune with ROC once labeled events exist."
        },
        "missions": {}
    }
    for rr in S.missions:
        pol_rr = _policy_for_mission(rr)
        # Attach AE references per monitored var (used downstream by ops to set alert thresholds)
        for v in telemetry_cols + radiation_cols:
            pol_rr["sampling_triggers"][f"ae_ref_{v}"] = _ae_ref_value(rr, v, default=0.95)
        policy["missions"][rr] = pol_rr
    return policy

def save_policy(policy: dict, path: Path) -> None:
    with open(path, "w") as f:
        json.dump(policy, f, indent=2)

# ---------- Compliance ----------
def compliance_report(rr: str, policy: dict) -> pd.DataFrame:
    tel = _load_mission("telemetry", rr)
    rad = _load_mission("radiation", rr)
    if tel is None and rad is None:
        return pd.DataFrame(columns=["timestamp","rule","ok"])

    rows = []

    # Telemetry bound checks
    if tel is not None:
        for var, rule in policy["missions"][rr].get("thresholds", {}).items():
            if var not in tel.columns: 
                continue
            s = tel[var].resample(S.sampling).mean()
            ok = pd.Series(True, index=s.index)
            if "hi" in rule and np.isfinite(rule["hi"]): ok &= (s <= rule["hi"])
            if "lo" in rule and np.isfinite(rule["lo"]): ok &= (s >= rule["lo"])
            rows.append(pd.DataFrame({"timestamp": s.index, "rule": f"{var}_bounds", "ok": ok.astype(bool)}))

    # Radiation sustained-high avoidance
    if rad is not None:
        avoid = policy["missions"][rr].get("radiation", {}).get("avoid_when", {})
        for var, rule in avoid.items():
            if var not in rad.columns: continue
            s = rad[var].resample(S.sampling).ffill()
            hi = rule.get("hi", np.inf)
            sustain = int(rule.get("sustain_minutes", 30))
            above = (s > hi).astype(int)
            # Time-based rolling window (requires DatetimeIndex)
            sustained = above.rolling(f"{sustain}T").sum() >= sustain
            rows.append(pd.DataFrame({"timestamp": s.index, "rule": f"{var}_avoid_hi", "ok": (~sustained).astype(bool)}))

    rep = pd.concat(rows).sort_values("timestamp") if rows else pd.DataFrame(columns=["timestamp","rule","ok"])
    rep.to_csv(COMPLIANCE_DIR / f"{rr}_compliance.csv", index=False)
    return rep

def compliance_summary(policy: dict) -> pd.DataFrame:
    out = []
    for rr in policy["missions"].keys():
        rep = compliance_report(rr, policy)
        if len(rep):
            rate = rep.groupby("rule")["ok"].mean().reset_index()
            rate["RR"] = rr
            out.append(rate)
    summ = pd.concat(out, ignore_index=True) if out else pd.DataFrame(columns=["RR","rule","ok"])
    if len(summ):
        summ.to_csv(POLICY_DIR / "compliance_summary.csv", index=False)
    return summ

# ---------- Optional ROC calibration if labels exist ----------
def try_calibrate_with_labels(policy: dict) -> dict:
    try:
        from sklearn.metrics import roc_curve
    except Exception:
        return policy

    updated = json.loads(json.dumps(policy))
    for rr in S.missions:
        lab_path = S.outputs_root / "labels" / f"{rr}_labels.csv"
        if not lab_path.exists(): 
            continue
        try:
            lab = pd.read_csv(lab_path, parse_dates=["timestamp"])
            lab["timestamp"] = pd.to_datetime(lab["timestamp"], utc=True)
        except Exception:
            continue

        tel = _load_mission("telemetry", rr)
        if tel is None or "CO2_ppm_ISS" not in tel.columns: 
            continue

        s = tel["CO2_ppm_ISS"].resample(S.sampling).mean().dropna()
        df = s.to_frame("value").reset_index().rename(columns={"index":"timestamp"})
        # If labels include 'variable', align to CO2; otherwise assume single-task labels
        if "variable" in lab.columns:
            lab_sub = lab[lab["variable"].astype(str) == "CO2_ppm_ISS"].copy()
        else:
            lab_sub = lab.copy()
        merged = pd.merge_asof(df.sort_values("timestamp"),
                               lab_sub.sort_values("timestamp"),
                               on="timestamp", tolerance=pd.Timedelta("2min"), direction="nearest")
        if "label" not in merged.columns:
            continue
        y = merged["label"].fillna(0).astype(int).values
        x = merged["value"].values
        if y.sum() == 0 or len(np.unique(y)) == 1:
            continue
        fpr, tpr, thr = roc_curve(y, x)
        j = tpr - fpr
        best_thr = thr[int(np.argmax(j))]
        updated["missions"][rr].setdefault("thresholds", {}).setdefault("CO2_ppm_ISS", {})["hi"] = float(best_thr)
    return updated

# ---------- Policy generation ----------
policy = build_policy()
policy_path = POLICY_DIR / "prescriptive_policy.json"
save_policy(policy, policy_path)
print(f"[✓] Policy saved → {policy_path}")

# Quick human-readable preview table (one row per mission)
prev_rows = []
for rr, body in policy["missions"].items():
    th = body.get("thresholds", {})
    rad = body.get("radiation", {}).get("avoid_when", {})
    prev_rows.append({
        "RR": rr,
        "CO2_hi": th.get("CO2_ppm_ISS",{}).get("hi"),
        "Temp_lo": th.get("Temp_degC_ISS",{}).get("lo"),
        "Temp_hi": th.get("Temp_degC_ISS",{}).get("hi"),
        "RH_lo": th.get("RH_percent_ISS",{}).get("lo"),
        "RH_hi": th.get("RH_percent_ISS",{}).get("hi"),
        "SAA_hi": (list(rad.values())[0]["hi"] if isinstance(rad, dict) and len(rad) else np.nan),
        "Prefer_hours_low_CO2": ",".join(map(str, body.get("scheduling",{}).get("prefer_hours_utc_low_CO2",[])))
    })
preview_df = pd.DataFrame(prev_rows)
preview_csv = POLICY_DIR / "policy_preview.csv"
preview_df.to_csv(preview_csv, index=False)
print(f"[✓] Policy preview → {preview_csv}")

# Compliance (per-mission files + summary)
summary = compliance_summary(policy)
print(f"[✓] Compliance summary rows: {len(summary)}" if len(summary) else "[i] No compliance rows (no data found)")

# Optional calibration via labels (if present)
policy_cal = try_calibrate_with_labels(policy)
if json.dumps(policy_cal, sort_keys=True) != json.dumps(policy, sort_keys=True):
    policy_cal_path = POLICY_DIR / "prescriptive_policy_calibrated.json"
    save_policy(policy_cal, policy_cal_path)
    print(f"[✓] Calibrated policy saved → {policy_cal_path}")
else:
    print("[i] No labels found (or not useful for ROC); kept percentile thresholds.")


[✓] Policy saved → /Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/outputs/policy/prescriptive_policy.json
[✓] Policy preview → /Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/outputs/policy/policy_preview.csv
[✓] Compliance summary rows: 24
[i] No labels found (or not useful for ROC); kept percentile thresholds.


### Section 9: Global QA & Summary Statistics
#### Purpose
Validate data integrity across missions and summarize whole-pipeline behavior (seasonality, causality, anomalies, forecast accuracy) to support interpretation, publication figures, and policy tuning.

#### What is implemented
1. Coverage & quality: minute-level coverage per mission×variable (`coverage_summary.csv`) and quality composition (original/interpolated/missing; `quality_counts.csv`) with coverage heatmap and per-mission stacked bars.
2. Seasonality distribution: histogram of `Seasonality_Strength` by type and export of top-15 seasonal variables (`seasonality_top15.csv`).
3. Directed causality (VAR–Granger): significant edge counts by mission and direction (telemetry→radiation, radiation→telemetry, within-domain) with summary table and bar chart.
4. Forecast performance distributions: RMSE boxplot by model (LSTM, SARIMAX where present) and best-model table per (mission, variable) (`forecast_best_by_variable.csv`).
5. Mission-level anomaly rates: Z>3 anomaly rate per day, per variable, plus aggregated mission/type bar chart (`anomaly_rate_per_day.csv`).
6. Omics snapshot (GLDS-98/99/104): counts of significant DEGs (q<0.05) with Up/Down bar chart and export (`omics_deg_counts.csv`).

#### Future Work
- None at this time

In [7]:
# ==== Section 9: Global QA & Summary Statistics (with Key Findings) ====
import os, json, warnings
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")
sns.set(style="whitegrid")

# ---------- Settings bootstrap ----------
if "S" not in globals():
    from dataclasses import dataclass, field
    @dataclass
    class Settings:
        missions: list[str] = field(default_factory=lambda: ["RR-1","RR-3","RR-6","RR-9","RR-12","RR-19"])
        sampling: str = "1min"
        outputs_root: Path = field(default_factory=lambda: Path.cwd() / "outputs")
        preprocessed_dir: Path = field(default_factory=lambda: Path.cwd() / "outputs" / "preprocessed")
        pattern_dir: Path = field(default_factory=lambda: Path.cwd() / "outputs" / "pattern_analysis")
        relationships_dir: Path = field(default_factory=lambda: Path.cwd() / "outputs" / "relationships")
        anomalies_dir: Path = field(default_factory=lambda: Path.cwd() / "outputs" / "anomaly_forecast")
        omics_roots: list[Path] = field(default_factory=lambda: [Path("/Users/kennethjenkins/data/genelab"),
                                                                 Path.cwd()/ "outputs" / "omics"])
    S = Settings()
    for d in [S.outputs_root, S.preprocessed_dir, S.pattern_dir, S.relationships_dir, S.anomalies_dir]:
        d.mkdir(parents=True, exist_ok=True)
    print("[Sec9 bootstrap] Using fallback Settings (S).")

# ---------- Paths ----------
telemetry_cols = ["Temp_degC_ISS", "RH_percent_ISS", "CO2_ppm_ISS"]
radiation_cols = ["GCR_Dose_mGy_d", "SAA_Dose_mGy_d", "Total_Dose_mGy_d", "Accumulated_Dose_mGy_d"]
all_vars = telemetry_cols + radiation_cols

out_dir = S.outputs_root / "section9_global_qa"
tables_dir = out_dir / "tables"
out_dir.mkdir(parents=True, exist_ok=True)
tables_dir.mkdir(parents=True, exist_ok=True)

PRE = S.preprocessed_dir
PAT = S.pattern_dir
REL = S.relationships_dir
ANOM = S.anomalies_dir

# ---------- Helpers: load mission data ----------
def _load_pre(rr: str, kind: str) -> pd.DataFrame | None:
    p = PRE / f"{rr}_cleaned_{kind}.csv"
    if p.exists():
        try:
            return pd.read_csv(p, index_col=0, parse_dates=True).sort_index()
        except Exception:
            return None
    try:
        if kind == "telemetry" and "preprocessed_telemetry" in globals() and rr in preprocessed_telemetry:
            return preprocessed_telemetry[rr].copy()
        if kind == "radiation" and "cleaned_radiation" in globals() and rr in cleaned_radiation:
            return cleaned_radiation[rr].copy()
    except Exception:
        pass
    return None

def _z_anomaly_rate_per_day(s: pd.Series, zthr=3.0) -> float:
    s = s.dropna()
    if len(s) < 100 or s.std() == 0:
        return 0.0
    z = (s - s.mean()) / s.std()
    count = (z.abs() > zthr).sum()
    days = max((s.index[-1] - s.index[0]).days + 1, 1)
    return float(count) / days

# ---------- OMICS helpers (use Sec7 state, GLDS_FILES, or search) ----------
def _standardize_deg_cols(df: pd.DataFrame) -> pd.DataFrame:
    d = df.copy()
    d.columns = (d.columns
                   .str.strip()
                   .str.replace("\u200b","", regex=False)
                   .str.replace("\xa0","", regex=False))
    if "gene" not in d.columns:
        gc = next((c for c in d.columns if c.lower() in {"gene","symbol","genes"}), None)
        if gc: d = d.rename(columns={gc: "gene"})
    if "log2FoldChange" not in d.columns:
        l2 = next((c for c in d.columns if ("log2" in c.lower() and "fold" in c.lower()) or c.lower() in {"log2fc","logfc"}), None)
        if l2: d = d.rename(columns={l2: "log2FoldChange"})
    if "padj" not in d.columns:
        pj = next((c for c in d.columns if c.lower() in {"padj","adj.p.val","fdr","fdr p-value","adj.p.value_(space flight)v(ground control)"}), None)
        if pj: d = d.rename(columns={pj: "padj"})
    return d

def _omics_diff_df(glds: str) -> pd.DataFrame | None:
    # 1) From Section 7 global 'omics_data'
    if "omics_data" in globals() and isinstance(omics_data, dict):
        dd = omics_data.get(glds, {})
        if isinstance(dd, dict) and "diff" in dd and isinstance(dd["diff"], pd.DataFrame):
            return _standardize_deg_cols(dd["diff"])
    # 2) From explicit GLDS_FILES mapping if present
    if "GLDS_FILES" in globals():
        path = GLDS_FILES.get(glds, {}).get("diff")
        if path and Path(path).exists():
            try:
                return _standardize_deg_cols(pd.read_csv(path))
            except Exception:
                pass
    # 3) Search under S.omics_roots
    candidates = [
        f"{glds}_rna_seq_differential_expression_GLbulkRNAseq.csv",
        f"{glds}_rna_seq_differential_expression_rRNArm_GLbulkRNAseq.csv",
        f"{glds}_rna_seq_differential_expression.csv",
    ]
    for root in getattr(S, "omics_roots", []):
        root = Path(root)
        for name in candidates:
            for p in [root / "RR-1" / glds / name, root / name]:
                if p.exists():
                    try:
                        return _standardize_deg_cols(pd.read_csv(p))
                    except Exception:
                        continue
    return None

# =================== 9.1 Coverage & Quality ===================
coverage_rows, quality_rows = [], []
for rr in S.missions:
    tel = _load_pre(rr, "telemetry")
    rad = _load_pre(rr, "radiation")

    for kind, df, cols in [("telemetry", tel, telemetry_cols), ("radiation", rad, radiation_cols)]:
        if df is None:
            continue
        total = len(df.index)
        for c in cols:
            if c not in df.columns:
                continue
            present = df[c].notna().sum()
            cov = present / total if total > 0 else 0.0
            qcol = f"{c}_qflag"
            if qcol in df.columns:
                vc = df[qcol].value_counts(dropna=False)
                orig = int(vc.get("orig", 0)); interp = int(vc.get("interp", 0)); miss = int(vc.get("missing", 0))
            else:
                orig = int(present); interp = 0; miss = int(total - present)
            coverage_rows.append({
                "RR": rr, "Variable": c, "Type": kind,
                "Total_Min": total, "Present_Min": int(present),
                "Coverage_Pct": float(cov * 100.0),
                "Interp_Pct_of_Present": float((interp / present) * 100.0) if present else 0.0
            })
            quality_rows.append({
                "RR": rr, "Variable": c, "Type": kind,
                "Orig_Min": int(orig), "Interp_Min": int(interp), "Missing_Min": int(miss)
            })

coverage_df = pd.DataFrame(coverage_rows)
quality_df = pd.DataFrame(quality_rows)
coverage_df.to_csv(out_dir / "coverage_summary.csv", index=False)
quality_df.to_csv(out_dir / "quality_counts.csv", index=False)

if not coverage_df.empty:
    piv = coverage_df.pivot_table(index="RR", columns="Variable", values="Coverage_Pct", aggfunc="mean")
    plt.figure(figsize=(12, max(4, 0.6 * len(piv))))
    sns.heatmap(piv, vmin=0, vmax=100, cmap="viridis", annot=True, fmt=".0f", cbar_kws={"label": "Coverage (%)"})
    plt.title("Minute-level coverage by mission × variable")
    plt.tight_layout(); plt.savefig(out_dir / "fig_coverage_heatmap.png", dpi=200); plt.close()

if not quality_df.empty:
    qsum = quality_df.copy()
    qsum["Total"] = qsum["Orig_Min"] + qsum["Interp_Min"] + qsum["Missing_Min"]
    qsum = qsum[qsum["Total"] > 0]
    qsum["Orig%"] = 100 * qsum["Orig_Min"] / qsum["Total"]
    qsum["Interp%"] = 100 * qsum["Interp_Min"] / qsum["Total"]
    qsum["Miss%"] = 100 * qsum["Missing_Min"] / qsum["Total"]
    for rr in qsum["RR"].unique():
        sub = qsum[qsum["RR"] == rr]
        if sub.empty: continue
        idx = np.arange(len(sub))
        plt.figure(figsize=(12, 4 + 0.2 * len(sub)))
        plt.barh(idx, sub["Miss%"], label="Missing")
        plt.barh(idx, sub["Interp%"], left=sub["Miss%"], label="Interpolated")
        plt.barh(idx, sub["Orig%"], left=sub["Miss%"] + sub["Interp%"], label="Original")
        plt.yticks(idx, sub["Variable"])
        plt.xlabel("Percent of minutes"); plt.title(f"{rr} · quality composition")
        plt.legend(loc="lower right", fontsize=8)
        plt.tight_layout(); plt.savefig(out_dir / f"fig_quality_{rr}.png", dpi=200); plt.close()

# =================== 9.2 Seasonality distribution ===================
season_path = PAT / "seasonality_summary.csv"
seas = pd.read_csv(season_path) if season_path.exists() else None
if seas is not None and "Seasonality_Strength" in seas.columns:
    plt.figure(figsize=(10,5))
    sns.histplot(seas, x="Seasonality_Strength", hue="Type", bins=30, stat="percent", common_norm=False)
    plt.title("Distribution of seasonality strength"); plt.xlabel("Strength (0..1)")
    plt.tight_layout(); plt.savefig(out_dir / "fig_seasonality_strength_hist.png", dpi=200); plt.close()
    top_seasonal = (seas.sort_values(["Seasonality_Strength","Seasonal_Amplitude_P95_P05"], ascending=[False, False])
                       .head(15)[["Mission","Type","Variable","Seasonality_Strength","Seasonal_Amplitude_P95_P05"]])
    top_seasonal.to_csv(out_dir / "seasonality_top15.csv", index=False)
else:
    top_seasonal = pd.DataFrame()

# =================== 9.3 Directed-causality (VAR–Granger) ===================
gpath = REL / "granger_var_results.csv"
if gpath.exists():
    gdf = pd.read_csv(gpath)
    gdf["Significant"] = gdf["Significant"].astype(bool) if "Significant" in gdf.columns else False
    if "MinusLog10_q" not in gdf.columns and "qvalue" in gdf.columns:
        gdf["MinusLog10_q"] = -np.log10(gdf["qvalue"].clip(lower=1e-300))
    gdf["Direction"] = np.where(gdf["Source"].isin(telemetry_cols) & gdf["Target"].isin(radiation_cols), "telemetry→radiation",
                         np.where(gdf["Source"].isin(radiation_cols) & gdf["Target"].isin(telemetry_cols), "radiation→telemetry",
                                  "within-domain"))
    gsum = (gdf[gdf["Significant"]]
            .groupby(["Mission","Direction"])
            .size().rename("Edges").reset_index())
    gsum_piv = gsum.pivot_table(index="Mission", columns="Direction", values="Edges", fill_value=0)
    gsum_piv.to_csv(out_dir / "granger_edge_counts.csv")
    if not gsum.empty:
        plt.figure(figsize=(12,5))
        sns.barplot(data=gsum, x="Mission", y="Edges", hue="Direction", errorbar=None)
        plt.title("Significant Granger edges by mission and direction (q≤0.05)")
        plt.tight_layout(); plt.savefig(out_dir / "fig_granger_edges.png", dpi=200); plt.close()
else:
    gdf = pd.DataFrame()
    gsum = pd.DataFrame()
    gsum_piv = pd.DataFrame()

# =================== 9.4 Forecast performance distributions ===================
def _read_scoreboard() -> pd.DataFrame | None:
    cand = list(ANOM.glob("*scoreboard*.csv"))
    if cand:
        try: return pd.read_csv(cand[0])
        except Exception: pass
    parts=[]
    for rr in S.missions:
        p = ANOM / f"{rr}_forecast_summary.csv"
        if p.exists():
            try:
                df = pd.read_csv(p); df["RR"] = rr; parts.append(df)
            except Exception: pass
    return pd.concat(parts, ignore_index=True) if parts else None

score = _read_scoreboard()
if score is not None and not score.empty:
    df_score = score.copy()
    if "Model" not in df_score.columns: df_score["Model"] = "LSTM"
    df_score = df_score.dropna(subset=["Variable","RMSE","MAE"])
    df_score["ModelShort"] = df_score["Model"].astype(str).str.replace(r"SARIMAX.*","SARIMAX", regex=True)
    plt.figure(figsize=(12,6))
    sns.boxplot(data=df_score, x="ModelShort", y="RMSE")
    plt.title("Forecast error by model (RMSE)"); plt.tight_layout()
    plt.savefig(out_dir / "fig_rmse_by_model.png", dpi=200); plt.close()
    best = (df_score.sort_values("RMSE")
              .groupby(["RR","Variable"], as_index=False)
              .first()[["RR","Variable","ModelShort","RMSE","MAE"]])
    best.to_csv(out_dir / "forecast_best_by_variable.csv", index=False)
else:
    df_score = pd.DataFrame()
    best = pd.DataFrame()

# =================== 9.5 Mission-level Z>3 anomaly rates ===================
anom_rows = []
for rr in S.missions:
    tel = _load_pre(rr, "telemetry"); rad = _load_pre(rr, "radiation")
    for kind, df, cols in [("telemetry", tel, telemetry_cols), ("radiation", rad, radiation_cols)]:
        if df is None:
            continue
        for c in cols:
            if c not in df.columns:
                continue
            s = df[c].resample("1min").mean().interpolate(limit=5)
            rate = _z_anomaly_rate_per_day(s, zthr=3.0)
            anom_rows.append({"RR": rr, "Variable": c, "Type": kind, "Anomalies_per_day_Z3": rate})

anom_df = pd.DataFrame(anom_rows)
anom_df.to_csv(out_dir / "anomaly_rate_per_day.csv", index=False)
if not anom_df.empty:
    agg = (anom_df.groupby(["RR","Type"])["Anomalies_per_day_Z3"]
                 .mean().rename("Anoms/day").reset_index())
    plt.figure(figsize=(12,5))
    sns.barplot(data=agg, x="RR", y="Anoms/day", hue="Type", errorbar=None)
    plt.title("Average Z>3 anomaly rate (per day)"); plt.tight_layout()
    plt.savefig(out_dir / "fig_anomaly_rate_by_mission.png", dpi=200); plt.close()

# =================== 9.6 Omics DEG counts (GLDS-98/99/104) ===================
def _omics_deg_counts(glds: str) -> dict | None:
    df = _omics_diff_df(glds)
    if df is None:
        return None
    df = _standardize_deg_cols(df)
    if "padj" not in df.columns or "log2FoldChange" not in df.columns:
        return None
    sig = df[df["padj"] < 0.05].copy()
    up = (sig["log2FoldChange"] > 0).sum()
    down = (sig["log2FoldChange"] < 0).sum()
    # write detailed tops
    if not sig.empty:
        sig_up = sig.sort_values("log2FoldChange", ascending=False).head(25)[["gene","log2FoldChange","padj"]]
        sig_dn = sig.sort_values("log2FoldChange", ascending=True).head(25)[["gene","log2FoldChange","padj"]]
        sig_up.to_csv(tables_dir / f"{glds}_top25_up_DEGs.csv", index=False)
        sig_dn.to_csv(tables_dir / f"{glds}_top25_down_DEGs.csv", index=False)
    return {"GLDS": glds, "Total_DEG_q<0.05": int(len(sig)), "Up": int(up), "Down": int(down)}

deg_rows = [r for g in ["GLDS-98","GLDS-99","GLDS-104"] if (r := _omics_deg_counts(g))]
deg_df = pd.DataFrame(deg_rows)
deg_df.to_csv(out_dir / "omics_deg_counts.csv", index=False)
if not deg_df.empty:
    plt.figure(figsize=(8,5))
    dd = deg_df.melt(id_vars="GLDS", value_vars=["Up","Down"], var_name="Direction", value_name="Count")
    sns.barplot(data=dd, x="GLDS", y="Count", hue="Direction", errorbar=None)
    plt.title("Significant DEGs by dataset (q<0.05)"); plt.tight_layout()
    plt.savefig(out_dir / "fig_deg_counts.png", dpi=200); plt.close()

# =================== 9.7 KEY FINDINGS (MD + JSON + extra CSVs) ===================
def _fmt(x, nd=3):
    try:
        return float(np.round(float(x), nd))
    except Exception:
        return x

findings = {"missions": list(S.missions)}

# Coverage/quality summaries
if not coverage_df.empty:
    findings["coverage"] = {
        "avg_coverage_pct": _fmt(coverage_df["Coverage_Pct"].mean(), 2),
        "median_coverage_pct": _fmt(coverage_df["Coverage_Pct"].median(), 2),
        "worst_coverage_top10": (coverage_df.sort_values("Coverage_Pct")
                                              .head(10)[["RR","Variable","Type","Coverage_Pct"]]
                                              .to_dict("records"))
    }
    coverage_df.sort_values("Coverage_Pct").head(25).to_csv(tables_dir / "worst_coverage_top25.csv", index=False)

if not quality_df.empty:
    totals = quality_df[["Orig_Min","Interp_Min","Missing_Min"]].sum()
    tot = totals.sum() if totals.sum()>0 else 1
    findings["quality"] = {
        "orig_pct": _fmt(100 * totals["Orig_Min"] / tot, 2),
        "interp_pct": _fmt(100 * totals["Interp_Min"] / tot, 2),
        "missing_pct": _fmt(100 * totals["Missing_Min"] / tot, 2)
    }

# Seasonality
if seas is not None and not seas.empty and "Seasonality_Strength" in seas.columns:
    findings["seasonality"] = {
        "n_series": int(len(seas)),
        "median_strength": _fmt(seas["Seasonality_Strength"].median(), 3),
        "top5": (seas.sort_values("Seasonality_Strength", ascending=False)
                      .head(5)[["Mission","Type","Variable","Seasonality_Strength"]]
                      .to_dict("records"))
    }
    top_seasonal.to_csv(tables_dir / "seasonality_top15.csv", index=False)

# Granger
if not gsum_piv.empty:
    totals_by_dir = gsum.groupby("Direction")["Edges"].sum().to_dict()
    findings["granger"] = {
        "edges_total": int(gsum["Edges"].sum()),
        "edges_by_direction": {k:int(v) for k,v in totals_by_dir.items()}
    }
    # top edges by strength if available
    if not gdf.empty and "MinusLog10_q" in gdf.columns:
        top_edges = (gdf[gdf.get("Significant", False)==True]
                       .sort_values("MinusLog10_q", ascending=False)
                       .head(20)[["Mission","Source","Target","Lag","qvalue","MinusLog10_q"]])
        top_edges.to_csv(tables_dir / "granger_top20_edges.csv", index=False)
        findings["granger"]["top_edge_example"] = top_edges.head(1).to_dict("records")

# Forecast
if not df_score.empty:
    rmse_by_model = (df_score.groupby("ModelShort")["RMSE"].median().sort_values().to_dict())
    findings["forecast"] = {
        "median_rmse_by_model": {k:_fmt(v,3) for k,v in rmse_by_model.items()}
    }
if not best.empty:
    winners = (best.groupby("ModelShort").size().sort_values(ascending=False).to_dict())
    findings.setdefault("forecast", {})["best_model_counts"] = winners
    best.sort_values(["RR","Variable"]).to_csv(tables_dir / "forecast_best_by_variable.csv", index=False)

# Anomaly rates
if not anom_df.empty:
    agg = (anom_df.groupby(["RR","Type"])["Anomalies_per_day_Z3"]
                 .mean().rename("Anoms_per_day").reset_index())
    top_anoms = (anom_df.sort_values("Anomalies_per_day_Z3", ascending=False)
                       .head(20)[["RR","Variable","Type","Anomalies_per_day_Z3"]])
    findings["anomalies"] = {
        "avg_anoms_per_day_overall": _fmt(anom_df["Anomalies_per_day_Z3"].mean(), 3),
        "top10_rate_examples": top_anoms.head(10).to_dict("records")
    }
    top_anoms.to_csv(tables_dir / "anomaly_rate_top20.csv", index=False)

# Omics
if not deg_df.empty:
    findings["omics"] = {
        "dataset_counts": deg_df.to_dict("records")
    }

# Write JSON + Markdown summaries
(key_md := out_dir / "key_findings.md").write_text(
    "# Key Findings (Section 9)\n"
    f"- Missions analyzed: {len(findings.get('missions', []))}\n"
    + ("\n## Coverage\n"
       f"- Average coverage: {findings['coverage']['avg_coverage_pct']}%\n"
       f"- Median coverage: {findings['coverage']['median_coverage_pct']}%\n"
       if 'coverage' in findings else "")
    + ("\n## Quality mix\n"
       f"- Original: {findings['quality']['orig_pct']}% | "
       f"Interpolated: {findings['quality']['interp_pct']}% | "
       f"Missing: {findings['quality']['missing_pct']}%\n"
       if 'quality' in findings else "")
    + ("\n## Seasonality\n"
       f"- Series: {findings['seasonality']['n_series']}, "
       f"median strength: {findings['seasonality']['median_strength']}\n"
       if 'seasonality' in findings else "")
    + ("\n## Granger causality\n"
       f"- Total significant edges: {findings['granger']['edges_total']}\n"
       f"- By direction: {findings['granger']['edges_by_direction']}\n"
       if 'granger' in findings else "")
    + ("\n## Forecasting\n"
       f"- Median RMSE by model: {findings['forecast'].get('median_rmse_by_model', {})}\n"
       f"- Best-model wins: {findings['forecast'].get('best_model_counts', {})}\n"
       if 'forecast' in findings else "")
    + ("\n## Anomalies\n"
       f"- Overall average Z>3 anomalies/day: {findings['anomalies']['avg_anoms_per_day_overall']}\n"
       if 'anomalies' in findings else "")
    + ("\n## Omics\n"
       f"- DEG counts (q<0.05): {findings['omics']['dataset_counts']}\n"
       if 'omics' in findings else "")
)
with open(out_dir / "key_findings.json", "w") as f:
    json.dump(findings, f, indent=2, default=lambda x: float(x) if isinstance(x, (np.floating,)) else x)

# Also print a short console highlight:
print("\n[Section 9] Key highlights")
if 'coverage' in findings:
    print(" - Avg coverage:", findings['coverage']['avg_coverage_pct'], "%")
if 'forecast' in findings and 'median_rmse_by_model' in findings['forecast']:
    print(" - Median RMSE by model:", findings['forecast']['median_rmse_by_model'])
if 'granger' in findings:
    print(" - Granger edges (total):", findings['granger']['edges_total'])
if 'anomalies' in findings:
    print(" - Avg anomalies/day (Z>3):", findings['anomalies']['avg_anoms_per_day_overall'])
if 'omics' in findings:
    print(" - Omics DEG datasets:", findings['omics']['dataset_counts'])

print("\n[Section 9] Wrote summaries/figures →", str(out_dir))
print("[Section 9] Tables →", str(tables_dir))



[Section 9] Key highlights
 - Avg coverage: 99.86 %
 - Median RMSE by model: {'LSTM': 0.038}
 - Granger edges (total): 50
 - Avg anomalies/day (Z>3): 15.539
 - Omics DEG datasets: [{'GLDS': 'GLDS-98', 'Total_DEG_q<0.05': 67, 'Up': 26, 'Down': 41}, {'GLDS': 'GLDS-99', 'Total_DEG_q<0.05': 2809, 'Up': 1564, 'Down': 1245}, {'GLDS': 'GLDS-104', 'Total_DEG_q<0.05': 4931, 'Up': 2360, 'Down': 2571}]

[Section 9] Wrote summaries/figures → /Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/outputs/section9_global_qa
[Section 9] Tables → /Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/outputs/section9_global_qa/tables


### Section 10: Publication Figures & Repro Plots (GLDS-98/99/104)
#### Purpose
Produce clean, reproducible figures for the paper/deck: anomaly timelines, anomaly calendars, seasonal overlays, model comparison panels, and omics volcano/PCA for GLDS-98/99/104.

#### What is implemented
1. Composite anomaly timeline per mission with Z>3 markers saved under `outputs/anomaly_forecast/<RR>/fig_composite_timeline.png`.
2. Anomaly calendar heatmap (hour × date) per mission saved under `outputs/anomaly_forecast/<RR>/fig_anomaly_calendar.png`.
3. Cross-model scoreboard bar chart (RMSE) across variables saved as `outputs/anomaly_forecast/fig_model_scoreboard.png`.
4. Seasonal index overlay across missions for each variable saved under `outputs/pattern_analysis/fig_seasonal_overlay_<VAR>.png`.
5. Omics volcano plots for GLDS-98, GLDS-99, GLDS-104 saved under `outputs/omics/fig_volcano_<GLDS>.png`.
6. Optional GLDS PCA scatter (if a PCA table is available) saved under `outputs/omics/fig_pca_<GLDS>.png`.

#### Future Work
- SARIMAX residual diagnostics for the best (mission, variable) by RMSE: residual trace, ACF, PACF, Q–Q, Ljung–Box p-values saved under `outputs/anomaly_forecast/<RR>/fig_resid_diagnostics_<VAR>.png`.

In [9]:
# ==== Section 10: Publication Figures & Repro Plots (GLDS-98/99/104) ====

import os, warnings
from dataclasses import dataclass, field
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")
sns.set(style="whitegrid")

# -------------------- Settings fallback --------------------
if "S" not in globals():
    @dataclass
    class Settings:
        missions: list[str] = field(default_factory=lambda: ["RR-1","RR-3","RR-6","RR-9","RR-12","RR-19"])
        sampling: str = "1min"
        outputs_root: Path = field(default_factory=lambda: Path.cwd() / "outputs")
        preprocessed_dir: Path = field(default_factory=lambda: Path.cwd() / "outputs" / "preprocessed")
        pattern_dir: Path = field(default_factory=lambda: Path.cwd() / "outputs" / "pattern_analysis")
        anomalies_dir: Path = field(default_factory=lambda: Path.cwd() / "outputs" / "anomaly_forecast")
        relationships_dir: Path = field(default_factory=lambda: Path.cwd() / "outputs" / "relationships")
        omics_roots: list[Path] = field(default_factory=lambda: [Path("/Users/kennethjenkins/data/genelab"),
                                                                 Path.cwd() / "outputs" / "omics"])
    S = Settings()
    for d in [S.outputs_root, S.preprocessed_dir, S.pattern_dir, S.anomalies_dir, S.relationships_dir, *S.omics_roots]:
        Path(d).mkdir(parents=True, exist_ok=True)
    print("[Sec10 bootstrap] Using fallback Settings (S).")

# -------------------- Config & logging --------------------
DO_RESID_DIAG = False  # keep False (residual diagnostics are slow; enable later if needed)

telemetry_cols = ["Temp_degC_ISS", "RH_percent_ISS", "CO2_ppm_ISS"]
radiation_cols  = ["GCR_Dose_mGy_d", "SAA_Dose_mGy_d", "Total_Dose_mGy_d", "Accumulated_Dose_mGy_d"]
all_vars = telemetry_cols + radiation_cols
GLDS_LIST = ["GLDS-98", "GLDS-99", "GLDS-104"]

ANOM_DIR    = S.anomalies_dir
PAT_DIR     = S.pattern_dir
PRE_DIR     = S.preprocessed_dir
OMICS_DIRS  = getattr(S, "omics_roots", [S.outputs_root / "omics"])
INSIGHTS_DIR= S.outputs_root / "insights"

LOG_PATH = S.outputs_root / "section10_progress.log"
def log(msg: str):
    ts = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
    line = f"[{ts} UTC] {msg}"
    print(line, flush=True)
    try:
        with open(LOG_PATH, "a") as f:
            f.write(line + "\n")
    except Exception:
        pass

# -------------------- Helpers --------------------
def _ensure_dir(p: Path): p.mkdir(parents=True, exist_ok=True)

def _load_pre(rr: str, kind: str) -> pd.DataFrame | None:
    p = PRE_DIR / f"{rr}_cleaned_{kind}.csv"
    if p.exists():
        try:
            return pd.read_csv(p, index_col=0, parse_dates=True).sort_index()
        except Exception:
            return None
    try:
        if kind == "telemetry" and "preprocessed_telemetry" in globals() and rr in preprocessed_telemetry:
            return preprocessed_telemetry[rr].copy()
        if kind == "radiation" and "cleaned_radiation" in globals() and rr in cleaned_radiation:
            return cleaned_radiation[rr].copy()
    except Exception:
        pass
    return None

def _missions_with(kind_needed=("telemetry","radiation")) -> list[str]:
    ms = []
    for rr in S.missions:
        ok = True
        for k in kind_needed:
            if (PRE_DIR / f"{rr}_cleaned_{k}.csv").exists():
                continue
            if not ((k=="telemetry" and "preprocessed_telemetry" in globals() and rr in preprocessed_telemetry) or
                    (k=="radiation"  and "cleaned_radiation" in globals() and rr in cleaned_radiation)):
                ok = False; break
        if ok: ms.append(rr)
    return ms

def _z_anoms(s: pd.Series, zthr=3.0) -> pd.Series:
    s = s.dropna()
    if len(s) < 10 or s.std() == 0:
        return pd.Series(index=s.index, dtype=bool)
    z = (s - s.mean()) / s.std()
    return z.abs() > zthr

def _read_scoreboard() -> pd.DataFrame | None:
    cand = list(ANOM_DIR.glob("*scoreboard*.csv"))
    if cand:
        try:
            return pd.read_csv(cand[0])
        except Exception:
            pass
    parts = []
    for rr in S.missions:
        p = ANOM_DIR / f"{rr}_forecast_summary.csv"
        if p.exists():
            try:
                df = pd.read_csv(p)
                if "RR" not in df.columns:
                    df["RR"] = rr
                parts.append(df)
            except Exception:
                pass
    return pd.concat(parts, ignore_index=True) if parts else None

def _minute_of_day_index(idx: pd.DatetimeIndex) -> pd.Index:
    m = (idx.hour*60 + idx.minute).astype(int)
    return pd.Index(m, name="minute_of_day")

def _load_seasonal(rr: str, var: str) -> pd.Series | None:
    p = PAT_DIR / "seasonal_indices" / f"{rr}_{var}_seasonal.csv"
    if not p.exists(): return None
    try:
        s = pd.read_csv(p, parse_dates=["timestamp"]).set_index("timestamp")["seasonal"]
        return s.asfreq("1min").interpolate(limit=5).dropna()
    except Exception:
        return None

def _safe_read_csv(p: Path | str) -> pd.DataFrame | None:
    if p is None: return None
    try:
        return pd.read_csv(p)
    except Exception:
        try:
            return pd.read_csv(p, engine="python", encoding="utf-8")
        except Exception:
            return None

# ----- Omics file resolvers -----
def _omics_candidates_deg(glds: str) -> list[str]:
    # support both classic and GLbulkRNAseq / rRNArm variants
    base = f"{glds}_rna_seq_differential_expression"
    return [
        f"{base}.csv",
        f"{base}_GLbulkRNAseq.csv",
        f"{base}_rRNArm_GLbulkRNAseq.csv",
    ]

def _find_omics_file(glds: str, fname: str) -> Path | None:
    for root in OMICS_DIRS:
        root = Path(root)
        p1 = root / "RR-1" / glds / fname
        p2 = root / fname
        if p1.exists(): return p1
        if p2.exists(): return p2
    return None

def _find_deg_any(glds: str) -> Path | None:
    # 1) explicit GLDS_FILES mapping (if present)
    if "GLDS_FILES" in globals():
        p = GLDS_FILES.get(glds, {}).get("diff")
        if p and Path(p).exists(): return Path(p)
    # 2) search OMICS_DIRS for any supported DEG filename
    for name in _omics_candidates_deg(glds):
        p = _find_omics_file(glds, name)
        if p is not None:
            return p
    return None

def _find_pca_table(glds: str) -> Path | None:
    # Prefer Section 7 PCA output
    p1 = INSIGHTS_DIR / f"{glds}_pca_table.csv"
    if p1.exists(): return p1
    # Accept a few generic names under omics roots (rare)
    for cand in [f"{glds}_rna_seq_visualization_PCA_table.csv",
                 "rna_seq_visualization_PCA_table.csv",
                 "visualization_PCA_table.csv"]:
        p = _find_omics_file(glds, cand)
        if p is not None:
            return p
    return None

def _standardize_deg_cols(df: pd.DataFrame) -> pd.DataFrame:
    """Normalize common GeneLab DEG headers to: gene, log2FoldChange, padj."""
    d = df.copy()
    d.columns = (d.columns.astype(str)
                 .str.strip()
                 .str.replace("\u200b","", regex=False)
                 .str.replace("\xa0","",  regex=False))

    cols = list(d.columns)

    def pick(cond):
        for c in cols:
            if cond(c): return c
        return None

    # gene column
    gene_c = pick(lambda c: c.lower() in {"gene","genes","symbol","gene_symbol","geneid","gene_id"} 
                           or "symbol" in c.lower())
    # log2 fold-change (allow substrings like "Log2fc_(Space Flight)v(Ground Control)")
    l2fc_c = pick(lambda c: ("log2" in c.lower() and ("fc" in c.lower() or "fold" in c.lower()))
                           or any(k in c.lower() for k in ["log2fc","logfc","lfc"]))
    # adjusted p-value (padj/FDR variants; allow substrings)
    padj_c = pick(lambda c: "padj" in c.lower() or ("adj" in c.lower() and "p" in c.lower()) or "fdr" in c.lower())
    # fallback nominal p-value if padj absent
    pval_c = pick(lambda c: c.lower() in {"pvalue","p.value","pval"} or ("p" in c.lower() and "value" in c.lower()))

    if gene_c and gene_c != "gene":                d = d.rename(columns={gene_c: "gene"})
    if l2fc_c and l2fc_c != "log2FoldChange":      d = d.rename(columns={l2fc_c: "log2FoldChange"})
    if padj_c and padj_c != "padj":                d = d.rename(columns={padj_c: "padj"})
    if "padj" not in d.columns and pval_c:         d = d.rename(columns={pval_c: "padj"})

    return d


# -------------------- Figures --------------------
def figure_composite_timeline(rr: str):
    log(f"[Sec10] START timeline {rr}")
    tel = _load_pre(rr, "telemetry"); rad = _load_pre(rr, "radiation")
    if tel is None and rad is None:
        log(f"[Sec10] SKIP timeline {rr} (no data)")
        return
    cols = [c for c in telemetry_cols if tel is not None and c in tel.columns] + \
           [c for c in radiation_cols  if rad is not None and c in rad.columns]
    if not cols:
        log(f"[Sec10] SKIP timeline {rr} (no overlapping columns)")
        return
    df = None
    if tel is not None: df = tel[telemetry_cols].copy()
    if rad is not None:
        df = rad[radiation_cols] if df is None else df.join(rad[radiation_cols], how="outer")
    df = df[[c for c in cols]].sort_index().resample("1min").mean().interpolate(limit=5)

    n = len(cols); h = max(2.5, 1.5*n)
    fig, axes = plt.subplots(n, 1, figsize=(14, h), sharex=True)
    if n == 1: axes = [axes]
    for ax, c in zip(axes, cols):
        s = df[c].dropna(); mask = _z_anoms(s, 3.0)
        ax.plot(s.index, s.values, lw=0.6, color="black")
        ax.scatter(s.index[mask], s[mask], s=6, color="red")
        ax.set_ylabel(c)
    axes[0].set_title(f"{rr} · Composite anomaly timeline (Z>3)")
    axes[-1].set_xlabel("Time (UTC)")
    out = ANOM_DIR / rr / "fig_composite_timeline.png"; _ensure_dir(out.parent)
    plt.tight_layout(); plt.savefig(out, dpi=200); plt.close()
    log(f"[Sec10] DONE timeline {rr} -> {out}")

def figure_anomaly_calendar(rr: str):
    log(f"[Sec10] START calendar {rr}")
    tel = _load_pre(rr, "telemetry"); rad = _load_pre(rr, "radiation")
    if tel is None and rad is None:
        log(f"[Sec10] SKIP calendar {rr} (no data)")
        return
    cols = [c for c in telemetry_cols if tel is not None and c in tel.columns] + \
           [c for c in radiation_cols  if rad is not None and c in rad.columns]
    if not cols:
        log(f"[Sec10] SKIP calendar {rr} (no overlapping columns)")
        return
    df = None
    if tel is not None: df = tel[telemetry_cols].copy()
    if rad is not None:
        df = rad[radiation_cols] if df is None else df.join(rad[radiation_cols], how="outer")
    df = df[[c for c in cols]].sort_index().resample("1min").mean().interpolate(limit=5)

    mask_any = pd.Series(False, index=df.index)
    for c in cols:
        mask_any = mask_any | _z_anoms(df[c], 3.0).reindex(df.index, fill_value=False)
    if mask_any.sum() == 0:
        log(f"[Sec10] SKIP calendar {rr} (no anomalies)")
        return

    dt = df.index[mask_any]
    cal = pd.DataFrame({"date": dt.date, "hour": dt.hour})
    tbl = cal.value_counts().rename("count").reset_index()
    piv = tbl.pivot(index="date", columns="hour", values="count").fillna(0)

    plt.figure(figsize=(14, max(3, 0.2*len(piv))))
    sns.heatmap(piv, cmap="magma", cbar_kws={"label":"anomaly count"})
    plt.title(f"{rr} · Anomaly calendar (Z>3, per hour)")
    plt.xlabel("Hour"); plt.ylabel("Date")
    out = ANOM_DIR / rr / "fig_anomaly_calendar.png"; _ensure_dir(out.parent)
    plt.tight_layout(); plt.savefig(out, dpi=200); plt.close()
    log(f"[Sec10] DONE calendar {rr} -> {out}")

def figure_scoreboard(score: pd.DataFrame):
    if score is None or score.empty:
        log("[Sec10] SKIP scoreboard (no rows)")
        return
    log("[Sec10] START scoreboard")
    df = score.copy()
    if "Model" not in df.columns:
        df["Model"] = "LSTM"
    df = df.dropna(subset=["Variable","RMSE"])
    df["ModelShort"] = df["Model"].astype(str).str.replace(r"SARIMAX.*","SARIMAX", regex=True)

    plt.figure(figsize=(14, 8))
    sns.barplot(data=df, x="Variable", y="RMSE", hue="ModelShort", errorbar=None)
    plt.title("Model comparison (RMSE)"); plt.tight_layout()
    out = ANOM_DIR / "fig_model_scoreboard.png"; _ensure_dir(out.parent)
    plt.savefig(out, dpi=200); plt.close()
    log(f"[Sec10] DONE scoreboard -> {out}")

def figure_seasonal_overlay(var: str):
    parts = []
    for rr in S.missions:
        s = _load_seasonal(rr, var)
        if s is None or len(s) < 1440:
            continue
        mod = pd.Series(s.values, index=_minute_of_day_index(s.index))
        grp = mod.groupby(level=0).median()
        grp = grp - grp.mean()
        parts.append((rr, grp))
    if not parts:
        log(f"[Sec10] SKIP seasonal overlay {var} (no series)")
        return
    log(f"[Sec10] START seasonal overlay {var} (missions={len(parts)})")
    plt.figure(figsize=(12, 5))
    for rr, grp in parts:
        plt.plot(grp.index, grp.values, lw=1.2, label=rr, alpha=0.9)
    plt.title(f"Seasonal index overlay — {var}")
    plt.xlabel("Minute of day (0–1439)"); plt.ylabel("Seasonal (zero-mean)")
    plt.legend(ncol=3, fontsize=8)
    out = PAT_DIR / f"fig_seasonal_overlay_{var}.png"; _ensure_dir(out.parent)
    plt.tight_layout(); plt.savefig(out, dpi=200); plt.close()
    log(f"[Sec10] DONE seasonal overlay {var} -> {out}")

def figure_omics_volcano(glds: str):
    log(f"[Sec10] START volcano {glds}")
    deg_path = _find_deg_any(glds)
    if deg_path is None:
        log(f"[Sec10] SKIP volcano {glds} (no DEG file)")
        return

    df = _safe_read_csv(deg_path)
    if df is None or df.empty:
        log(f"[Sec10] SKIP volcano {glds} (DEG read failed)")
        return

    df = _standardize_deg_cols(df)
    log(f"[Sec10] {glds} DEG cols -> gene:{'gene' in df.columns}, l2fc:{'log2FoldChange' in df.columns}, padj:{'padj' in df.columns}")

    if "log2FoldChange" not in df.columns:
        log(f"[Sec10] SKIP volcano {glds} (no log2FoldChange after std)")
        return

    x = df["log2FoldChange"].astype(float)
    if "padj" in df.columns:
        y = -np.log10(pd.to_numeric(df["padj"], errors="coerce").clip(lower=1e-300))
        sig_mask = pd.to_numeric(df["padj"], errors="coerce") < 0.05
        ylab = "-log10(padj)"
    else:
        # proxy if only fold-changes exist
        y = np.log10(1 + x.abs())
        sig_mask = y > y.median()
        ylab = "proxy intensity"

    plt.figure(figsize=(7,5))
    plt.scatter(x, y, s=10, alpha=0.6, c=np.where(sig_mask, "crimson", "gray"))
    plt.axvline(0, color="black", lw=0.7)
    if ylab == "-log10(padj)":
        plt.axhline(-np.log10(0.05), color="black", lw=0.7, ls="--")
    plt.title(f"{glds} · Volcano"); plt.xlabel("log2 fold change"); plt.ylabel(ylab)
    out = S.outputs_root / "omics" / f"fig_volcano_{glds}.png"; _ensure_dir(out.parent)
    plt.tight_layout(); plt.savefig(out, dpi=200); plt.close()
    log(f"[Sec10] DONE volcano {glds} -> {out}")

def _infer_group_from_sample_names(samples: pd.Series) -> pd.Series:
    # heuristic: look for tokens like FLT/Flight vs GC/Control
    def lab(s):
        s = str(s).upper()
        if "FLT" in s or "FLIGHT" in s: return "FLT"
        if "GC" in s or "CONTROL" in s: return "GC"
        return "UNK"
    return samples.apply(lab)

def _load_sample_table_for_grouping(glds: str) -> dict | None:
    # Try GLDS_FILES mapping first
    if "GLDS_FILES" in globals():
        sp = GLDS_FILES.get(glds, {}).get("sample")
        if sp and Path(sp).exists():
            st = _safe_read_csv(sp)
            if st is not None:
                # guess id and group columns
                id_col = next((c for c in st.columns if c.lower() in {"sample","sampleid","sample_id","sample_name","samplename"}), None)
                group_col = next((c for c in st.columns if c.lower() in {"group","condition","treatment"}), None)
                if id_col and group_col:
                    return st.set_index(id_col)[group_col].astype(str).to_dict()
    # else none
    return None

def figure_omics_pca_if_available(glds: str):
    p = _find_pca_table(glds)
    if p is None:
        log(f"[Sec10] SKIP PCA {glds} (no PCA table)")
        return
    log(f"[Sec10] START PCA {glds}")
    pc = _safe_read_csv(p)
    if pc is None or pc.empty:
        log(f"[Sec10] SKIP PCA {glds} (read failed)")
        return
    # Flexible PC col detection
    x = y = None
    for cand in [("PC1","PC2"), ("PC_1","PC_2"), ("PC 1","PC 2"), ("C1","PC2"), ("Dim1","Dim2")]:
        if cand[0] in pc.columns and cand[1] in pc.columns:
            x, y = cand; break
    if not (x and y):
        log(f"[Sec10] SKIP PCA {glds} (PC columns missing)")
        return

    # Color by group, if available
    color_col = None
    for c in ["Group","Condition","group","condition","SampleGroup","Treatment"]:
        if c in pc.columns: color_col = c; break

    if color_col is None:
        # Try to infer from Sample names
        sample_col = next((c for c in pc.columns if c.lower() in {"sample","sampleid","sample_id","sample_name","samplename"}), None)
        if sample_col:
            pc["__GroupInferred"] = _infer_group_from_sample_names(pc[sample_col])
            color_col = "__GroupInferred"
        else:
            # last resort: try mapping from sample table
            sample_col = "Sample" if "Sample" in pc.columns else None
            mapping = _load_sample_table_for_grouping(glds)
            if sample_col and mapping:
                pc["__GroupFromTable"] = pc[sample_col].map(mapping).fillna("UNK")
                color_col = "__GroupFromTable"

    plt.figure(figsize=(6,5))
    if color_col:
        for g, sub in pc.groupby(color_col):
            plt.scatter(sub[x], sub[y], s=22, alpha=0.8, label=str(g))
        plt.legend(title=color_col, fontsize=8)
    else:
        plt.scatter(pc[x], pc[y], s=22, alpha=0.8)
    plt.title(f"{glds} · PCA"); plt.xlabel(x); plt.ylabel(y)
    out = S.outputs_root / "omics" / f"fig_pca_{glds}.png"; _ensure_dir(out.parent)
    plt.tight_layout(); plt.savefig(out, dpi=200); plt.close()
    log(f"[Sec10] DONE PCA {glds} -> {out}")

# -------------------- Run --------------------
log("[Sec10] Begin figure generation")

missions = _missions_with(("telemetry","radiation"))
scoreboard = _read_scoreboard()

for rr in missions:
    try:
        figure_composite_timeline(rr)
        figure_anomaly_calendar(rr)
        if DO_RESID_DIAG:
            log(f"[Sec10] Residual diagnostics are disabled (DO_RESID_DIAG=False).")
    except Exception as e:
        log(f"[Sec10] ERROR mission {rr}: {e}")

figure_scoreboard(scoreboard)

for v in all_vars:
    try:
        figure_seasonal_overlay(v)
    except Exception as e:
        log(f"[Sec10] ERROR seasonal overlay {v}: {e}")

for glds in GLDS_LIST:
    try:
        figure_omics_volcano(glds)
        figure_omics_pca_if_available(glds)
    except Exception as e:
        log(f"[Sec10] ERROR omics {glds}: {e}")

log(f"[Sec10] All done → {S.outputs_root}")
print("[Section 10] Figures written to:", str(S.outputs_root))



[2025-09-14 17:28:23 UTC] [Sec10] Begin figure generation
[2025-09-14 17:28:23 UTC] [Sec10] START timeline RR-1
[2025-09-14 17:28:26 UTC] [Sec10] DONE timeline RR-1 -> /Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/outputs/anomaly_forecast/RR-1/fig_composite_timeline.png
[2025-09-14 17:28:26 UTC] [Sec10] START calendar RR-1
[2025-09-14 17:28:27 UTC] [Sec10] DONE calendar RR-1 -> /Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/outputs/anomaly_forecast/RR-1/fig_anomaly_calendar.png
[2025-09-14 17:28:27 UTC] [Sec10] START timeline RR-3
[2025-09-14 17:28:30 UTC] [Sec10] DONE timeline RR-3 -> /Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/outputs/anomaly_forecast/RR-3/fig_composite_timeline.png
[2025-09-14 17:28:30 UTC] [Sec10] START calendar RR-3
[2025-09-14 17:28:31 UTC] [Sec10] DONE calendar RR-3 -> /Users/kennethjenkins/Citizen Science/Data Mining for Space Habitats/outputs/anomaly_forecast/RR-3/fig_anomaly_calendar.png
[2

### Section 11: Streamlit Dashboard
#### Purpose
Provide a compact UI to browse anomalies, forecasts, seasonality, relationships, policies, and (for RR-1) omics summaries.
#### What is implemented
1. Mission/type/variable selection; artifact discovery from `outputs/*` with caching.
2. Panels for anomaly/forecast images, STL/FFT/Welch plots, and relationship visuals (correlations, directed Granger).
3. Policy viewer (JSON) and access to key CSV exports (e.g., scoreboard, summaries).
4. RR-1 omics tables (DEG/PCA/anomaly timestamps).
#### Future Work
- Publish dashboard

In [10]:
# ==== Section 11: Streamlit Dashboard (app.py) ====
import os, json, math
from pathlib import Path
from datetime import datetime
from typing import Optional, Iterable

import pandas as pd
import numpy as np
import streamlit as st

# --------------------------- App setup ---------------------------
st.set_page_config(page_title="Space Habitat Dashboard", layout="wide")
st.title("Space Habitat • Anomalies • Forecasts • Policies • Omics")

# Root detection (run from project root)
BASE = Path.cwd()
OUT  = BASE / "outputs"

ANOM_DIR    = OUT / "anomaly_forecast"
PATTERN_DIR = OUT / "pattern_analysis"
REL_DIR     = OUT / "relationships"
INS_DIR     = OUT / "insights"
POLICY_DIR  = OUT / "policy"
PREP_DIR    = OUT / "preprocessed"
SEC9_DIR    = OUT / "section9_global_qa"
OMICS_DIR   = OUT / "omics"

# --------------------------- Helpers & cache ---------------------------
def _exists(p: Path) -> bool: return p.exists()

@st.cache_data(show_spinner=False)
def list_missions() -> list[str]:
    if not _exists(ANOM_DIR): return []
    return sorted([p.name for p in ANOM_DIR.iterdir() if p.is_dir()])

@st.cache_data(show_spinner=False)
def variables_for(rr: str) -> dict:
    out = {"telemetry": [], "radiation": []}
    root = ANOM_DIR / rr
    if not _exists(root): return out
    for p in root.iterdir():
        if not p.is_dir(): continue
        n = p.name
        if n.startswith("telemetry_"): out["telemetry"].append(n.replace("telemetry_",""))
        if n.startswith("radiation_"):  out["radiation"].append(n.replace("radiation_",""))
    out["telemetry"].sort(); out["radiation"].sort()
    return out

@st.cache_data(show_spinner=False)
def load_csv(path: Path, nrows: Optional[int] = None) -> Optional[pd.DataFrame]:
    try:
        return pd.read_csv(path, nrows=nrows)
    except Exception:
        return None

@st.cache_data(show_spinner=False)
def load_csv_dtindex(path: Path) -> Optional[pd.DataFrame]:
    try:
        return pd.read_csv(path, index_col=0, parse_dates=True).sort_index()
    except Exception:
        return None

@st.cache_data(show_spinner=False)
def load_text(path: Path, max_chars: int = 20000) -> Optional[str]:
    try:
        txt = path.read_text(encoding="utf-8", errors="ignore")
        return txt[:max_chars]
    except Exception:
        return None

def human_size(nbytes: int) -> str:
    if nbytes is None: return "?"
    units = ["B","KB","MB","GB","TB"]
    if nbytes == 0: return "0 B"
    i = int(math.floor(math.log(nbytes, 1024)))
    return f"{nbytes/1024**i:.1f} {units[i]}"

@st.cache_data(show_spinner=False)
def list_files_recursive(root: Path, exts: Optional[Iterable[str]]=None) -> pd.DataFrame:
    """Return table of all files under root with size and mtime."""
    rows = []
    if not _exists(root): return pd.DataFrame(columns=["relpath","size","mtime","ext","abs"])
    for p in root.rglob("*"):
        if p.is_file():
            ext = p.suffix.lower()
            if exts and ext not in exts: continue
            try:
                stat = p.stat()
                rows.append({
                    "relpath": str(p.relative_to(root)),
                    "size": stat.st_size,
                    "mtime": datetime.fromtimestamp(stat.st_mtime).isoformat(sep=" ", timespec="seconds"),
                    "ext": ext,
                    "abs": str(p.resolve())
                })
            except Exception:
                continue
    df = pd.DataFrame(rows)
    if len(df): df = df.sort_values(["ext","relpath"]).reset_index(drop=True)
    return df

def safe_image(path: Path, caption: str, use_col=True):
    if _exists(path):
        st.image(str(path), caption=caption, use_column_width=use_col)

def download_button_for(path: Path, label: Optional[str]=None):
    if _exists(path):
        label = label or f"Download {path.name}"
        st.download_button(label, data=path.read_bytes(), file_name=path.name)

# --------------------------- Sidebar ---------------------------
with st.sidebar:
    st.header("Controls")
    missions = list_missions()
    if missions:
        rr = st.selectbox("Mission", missions, index=0)
        kinds = variables_for(rr)
        vtype = st.radio("Type", ["telemetry","radiation"], horizontal=True)
        vlist = kinds.get(vtype, [])
        var = st.selectbox("Variable", vlist) if vlist else None
    else:
        rr = None; vtype = "telemetry"; var = None
    st.caption(f"Outputs root: {OUT.resolve()}")

# --------------------------- Tabs ---------------------------
tabs = st.tabs([
    "Overview",
    "Per-Mission (Anomaly/Forecast)",
    "Pattern Analysis",
    "Relationships",
    "Policy & Compliance",
    "Insights & Narratives",
    "Global QA (Sec 9)",
    "Omics (GLDS-98/99/104)",
    "Explorer & Downloads"
])

# ========================= Overview =========================
with tabs[0]:
    st.subheader("What’s available")
    cols = st.columns(3)
    with cols[0]:
        st.markdown("**Core directories**")
        for name, p in [
            ("anomaly_forecast", ANOM_DIR),
            ("pattern_analysis", PATTERN_DIR),
            ("relationships", REL_DIR),
            ("insights", INS_DIR),
            ("policy", POLICY_DIR),
            ("preprocessed", PREP_DIR),
            ("section9_global_qa", SEC9_DIR),
            ("omics (figures)", OMICS_DIR),
        ]:
            st.write(f"- {name}: {'✅' if _exists(p) else '❌'}")
    with cols[1]:
        st.markdown("**Key figures**")
        safe_image(ANOM_DIR / "fig_model_scoreboard.png", "Model scoreboard (RMSE)")
        safe_image(PATTERN_DIR / "fig_seasonal_overlay_CO2_ppm_ISS.png", "Seasonal overlay — CO₂")
    with cols[2]:
        st.markdown("**Key findings (Sec 9)**")
        kjson = SEC9_DIR / "key_findings.json"
        kmd   = SEC9_DIR / "key_findings.md"
        if _exists(kjson):
            st.json(json.loads(kjson.read_text()))
            download_button_for(kjson)
        if _exists(kmd):
            st.markdown("---")
            st.markdown(load_text(kmd) or "")
            download_button_for(kmd)

# ============== Per-Mission (Anomaly/Forecast) ==============
with tabs[1]:
    st.subheader("Anomaly timelines, calendars, forecasts")
    if rr is None:
        st.info("No missions discovered under outputs/anomaly_forecast.")
    else:
        # Mission-level images
        c1, c2 = st.columns(2)
        with c1: safe_image(ANOM_DIR / rr / "fig_composite_timeline.png", f"{rr} • Composite anomaly timeline (Z>3)")
        with c2: safe_image(ANOM_DIR / rr / "fig_anomaly_calendar.png", f"{rr} • Anomaly calendar (Z>3)")

        # Variable-specific artifacts
        st.markdown("---")
        st.markdown(f"**Artifacts for** `{rr}` · `{vtype}:{var}`")
        if var:
            col_dir = ANOM_DIR / rr / f"{vtype}_{var}"
            if _exists(col_dir):
                c1, c2 = st.columns(2)
                with c1:
                    safe_image(col_dir/"anoms.png", "Z & EWMA anomalies")
                    safe_image(col_dir/"ae_error.png", "LSTM AE reconstruction error")
                    if _exists(col_dir/"anoms.csv"):
                        st.caption("Anomaly flags CSV"); download_button_for(col_dir/"anoms.csv")
                with c2:
                    safe_image(col_dir/"lstm_forecast.png", "LSTM forecast")
                    safe_image(col_dir/"sarimax_forecast.png", "SARIMAX forecast")
                    if _exists(col_dir/"forecast.csv"):
                        st.caption("Forecast CSV"); download_button_for(col_dir/"forecast.csv")
            else:
                st.info("No artifacts for the selected series yet.")
        else:
            st.caption("Pick a variable in the sidebar to see series-level artifacts.")

        # Per-mission summary table (if present)
        st.markdown("---")
        summary_path = ANOM_DIR / f"{rr}_forecast_summary.csv"
        if _exists(summary_path):
            st.markdown("**Per-mission forecast summary**")
            df = load_csv(summary_path)
            st.dataframe(df)
            download_button_for(summary_path)

# ======================= Pattern Analysis =======================
with tabs[2]:
    st.subheader("STL / FFT / Welch & seasonal overlays")
    if rr and var:
        if vtype == "telemetry":
            base = PATTERN_DIR
        else:
            base = PATTERN_DIR / "radiation"
        for suffix, title in [("Raw","Raw"), ("STL","STL"), ("FFT","FFT"), ("WELCH","Welch PSD")]:
            safe_image(base / f"{var}_{rr}_{suffix}.png", f"{rr} • {var} • {title}")
    st.markdown("---")
    st.markdown("**Seasonality summary (Section 4)**")
    seas_path = PATTERN_DIR / "seasonality_summary.csv"
    if _exists(seas_path):
        seas = load_csv(seas_path)
        st.dataframe(seas.head(50))
        download_button_for(seas_path)

    st.markdown("**Seasonal overlay figures (Section 10)**")
    overlay_cols = st.columns(3)
    overlays = [
        "Temp_degC_ISS","RH_percent_ISS","CO2_ppm_ISS",
        "GCR_Dose_mGy_d","SAA_Dose_mGy_d","Total_Dose_mGy_d","Accumulated_Dose_mGy_d"
    ]
    for i, v in enumerate(overlays):
        with overlay_cols[i % 3]:
            safe_image(PATTERN_DIR / f"fig_seasonal_overlay_{v}.png", f"Overlay — {v}")

# ========================= Relationships =========================
with tabs[3]:
    st.subheader("Correlations & Granger causality")
    if rr:
        c1, c2 = st.columns(2)
        with c1:
            safe_image(REL_DIR / "plots" / f"{rr}_pearson.png", f"{rr} • Pearson")
            safe_image(REL_DIR / "plots" / f"{rr}_spearman.png", f"{rr} • Spearman")
        with c2:
            safe_image(REL_DIR / "plots" / f"{rr}_granger_directed.png", f"{rr} • Directed Granger (−log10 q)")
            safe_image(REL_DIR / "plots" / f"{rr}_network.png", f"{rr} • Correlation network")
        # CSVs
        for fname in [f"{rr}_pearson_corr.csv", f"{rr}_spearman_corr.csv", f"{rr}_partial_corr.csv"]:
            p = REL_DIR / fname
            if _exists(p):
                st.caption(fname); download_button_for(p)
    # Combined tables
    st.markdown("---")
    for fname in ["granger_var_results.csv", "correlations_tidy.csv"]:
        p = REL_DIR / fname
        if _exists(p):
            st.markdown(f"**{fname}**")
            st.dataframe(load_csv(p).head(200))
            download_button_for(p)

# ===================== Policy & Compliance =====================
with tabs[4]:
    st.subheader("Prescriptive policy & compliance (Section 8)")
    # Policy JSONs
    pol_cands = [POLICY_DIR / "prescriptive_policy_calibrated.json", POLICY_DIR / "prescriptive_policy.json"]
    pol = None
    for p in pol_cands:
        if _exists(p):
            pol = json.loads(p.read_text())
            st.markdown(f"**Loaded policy:** `{p.name}`"); download_button_for(p)
            st.json(pol.get("meta", {}))
            break
    if pol and rr in pol.get("missions", {}):
        st.markdown(f"**Thresholds — {rr}**")
        st.json(pol["missions"][rr].get("thresholds", {}))
        st.markdown("**Radiation guidance**")
        st.json(pol["missions"][rr].get("radiation", {}))
        st.markdown("**Scheduling preferences**")
        st.json(pol["missions"][rr].get("scheduling", {}))
    # Compliance
    comp_summ = POLICY_DIR / "compliance_summary.csv"
    if _exists(comp_summ):
        st.markdown("---")
        st.markdown("**Compliance summary (all missions)**")
        st.dataframe(load_csv(comp_summ))
        download_button_for(comp_summ)
    if rr:
        comp_path = POLICY_DIR / "compliance" / f"{rr}_compliance.csv"
        if _exists(comp_path):
            st.markdown(f"**Compliance series — {rr}**")
            df = load_csv(comp_path, nrows=100000)
            st.dataframe(df.head(5000))  # show a slice to keep UI snappy
            download_button_for(comp_path)

# ===================== Insights & Narratives =====================
with tabs[5]:
    st.subheader("Cross-mission insights (Section 7)")
    # Core CSVs
    for fname in [
        "all_missions_metrics_raw.csv",
        "volatility_by_variable.csv",
        "anomaly_calendar_all_missions.csv",
        "mission_anomaly_rates.csv",
        "group_comparison_metrics.csv",
        "top_forecastable_by_mission.csv",
        "top_anomalous_by_mission.csv",
        "top_seasonality_by_mission.csv",
        "omics_datasets_summary.csv",
    ]:
        p = INS_DIR / fname
        if _exists(p):
            st.markdown(f"**{fname}**")
            st.dataframe(load_csv(p).head(1000))
            download_button_for(p)

    # Narratives
    st.markdown("---")
    st.markdown("**Mission narratives**")
    if rr and _exists(INS_DIR / f"{rr}_narrative.md"):
        st.markdown(load_text(INS_DIR / f"{rr}_narrative.md") or "")
        download_button_for(INS_DIR / f"{rr}_narrative.md")
    # Plots
    st.markdown("---")
    st.markdown("**Insight plots**")
    safe_image(INS_DIR / "plots" / "anomaly_intensity_by_type.png", "Anomaly intensity by type (AE)")
    safe_image(INS_DIR / "plots" / "top_forecastable_by_mission.png", "Top forecastable by mission")

# ======================= Global QA (Section 9) =======================
with tabs[6]:
    st.subheader("Global QA & summary (Section 9)")
    # Main tables/figures
    for fname in [
        "coverage_summary.csv","quality_counts.csv","anomaly_rate_per_day.csv",
        "forecast_best_by_variable.csv","granger_edge_counts.csv","seasonality_top15.csv",
    ]:
        p = SEC9_DIR / fname
        if _exists(p):
            st.markdown(f"**{fname}**")
            st.dataframe(load_csv(p).head(2000))
            download_button_for(p)
    # Figures
    fig_names = [
        "fig_coverage_heatmap.png","fig_rmse_by_model.png","fig_anomaly_rate_by_mission.png",
        "fig_granger_edges.png","fig_seasonality_strength_hist.png"
    ]
    grid = st.columns(3)
    for i, fname in enumerate(fig_names):
        with grid[i % 3]:
            safe_image(SEC9_DIR / fname, fname.replace("_"," "))
    # Per-mission quality figs
    st.markdown("---")
    st.markdown("**Per-mission quality composition**")
    if rr:
        safe_image(SEC9_DIR / f"fig_quality_{rr}.png", f"{rr} • quality composition")
    # Section 9 tables subdir
    tbl_dir = SEC9_DIR / "tables"
    if _exists(tbl_dir):
        st.markdown("---")
        st.markdown("**Tables (top-k digests)**")
        files = list_files_recursive(tbl_dir, exts={".csv"})
        if len(files):
            st.dataframe(files[["relpath","size","mtime"]])
            for _, row in files.iterrows():
                p = tbl_dir / row["relpath"]
                with st.expander(row["relpath"]):
                    st.dataframe(load_csv(p))
                    download_button_for(p)

# =================== Omics (GLDS-98/99/104) ===================
with tabs[7]:
    st.subheader("Omics summaries & figures")
    # Section 7 top genes & PCA tables
    for glds in ["GLDS-98","GLDS-99","GLDS-104"]:
        st.markdown(f"### {glds}")
        # Top up/down from insights
        up   = INS_DIR / f"{glds}_top_upregulated.csv"
        down = INS_DIR / f"{glds}_top_downregulated.csv"
        pca  = INS_DIR / f"{glds}_pca_table.csv"
        if _exists(up):
            st.caption("Top upregulated"); st.dataframe(load_csv(up)); download_button_for(up)
        if _exists(down):
            st.caption("Top downregulated"); st.dataframe(load_csv(down)); download_button_for(down)
        if _exists(pca):
            st.caption("PCA 2D table"); st.dataframe(load_csv(pca).head(200)); download_button_for(pca)
        # Section 10 figures
        safe_image(OMICS_DIR / f"fig_volcano_{glds}.png", f"{glds} • volcano")
        safe_image(OMICS_DIR / f"fig_pca_{glds}.png",      f"{glds} • PCA")

# =================== Explorer & Downloads ===================
with tabs[8]:
    st.subheader("Full artifact browser")
    colA, colB = st.columns([2,1])
    with colA:
        root_choice = st.selectbox("Pick a root directory to browse", [
            str(OUT), str(ANOM_DIR), str(PATTERN_DIR), str(REL_DIR),
            str(INS_DIR), str(POLICY_DIR), str(PREP_DIR), str(SEC9_DIR), str(OMICS_DIR)
        ], index=0)
        root = Path(root_choice)
        ext_filter = st.multiselect(
            "Filter by extension",
            [".csv",".png",".json",".md",".log"],
            default=[".csv",".png",".json",".md"]
        )
        files = list_files_recursive(root, exts=set(ext_filter) if ext_filter else None)
        q = st.text_input("Search filename contains", "")
        if q:
            files = files[files["relpath"].str.contains(q, case=False, na=False)]
        if len(files):
            show = files.copy()
            show["size"] = show["size"].apply(human_size)
            st.dataframe(show[["relpath","ext","size","mtime"]], use_container_width=True, height=400)
        else:
            st.info("No files found with current filters.")
    with colB:
        st.markdown("**Preview / Download**")
        if len(files):
            pick = st.selectbox("Choose a file", files["relpath"].tolist())
            fpath = root / pick
            st.caption(pick)
            if fpath.suffix.lower() in {".csv"}:
                n = st.slider("Rows to preview", 5, 2000, 50, 5)
                df = load_csv(fpath, nrows=n)
                if df is not None: st.dataframe(df)
            elif fpath.suffix.lower() in {".json",".md",".log"}:
                st.code(load_text(fpath) or "", language="markdown" if fpath.suffix.lower()==".md" else "json")
            elif fpath.suffix.lower() in {".png",".jpg",".jpeg"}:
                safe_image(fpath, fpath.name)
            download_button_for(fpath)

st.caption("Data Mining for Space Habitats — all artifacts are preliminary pending SARIMAX benchmarking & label-based calibration.")



2025-09-14 10:41:49.352 
  command:

    streamlit run /opt/anaconda3/envs/dlclass/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-09-14 10:41:49.352 No runtime found, using MemoryCacheStorageManager
2025-09-14 10:41:49.354 No runtime found, using MemoryCacheStorageManager
2025-09-14 10:41:49.355 No runtime found, using MemoryCacheStorageManager
2025-09-14 10:41:49.355 No runtime found, using MemoryCacheStorageManager
2025-09-14 10:41:49.355 No runtime found, using MemoryCacheStorageManager
2025-09-14 10:41:49.356 No runtime found, using MemoryCacheStorageManager
2025-09-14 10:41:49.357 No runtime found, using MemoryCacheStorageManager
2025-09-14 10:41:49.358 Session state does not function when running a script without `streamlit run`
2025-09-14 10:41:49.358 No runtime found, using MemoryCacheStorageManager
2025-09-14 10:41:49.656 No runtime found, using MemoryCacheStorageManager
2025-09-14 10:41:50.064 No runtime found, using MemoryCacheStorageManager
2025-09-14 1

DeltaGenerator()

# In Terminal only 
# create & activate a venv
python -m venv .venv
# macOS/Linux:
source .venv/bin/activate

pip install --upgrade pip
pip install -r requirements.txt

# launch the dashboard
streamlit run app.py


### Section 11: Evaluation and Insights

#### 11.1 Forecasting and Anomaly Detection Performance

**LSTM Forecasting Performance**

**Key Observations:**

**Anomaly Detection Effectiveness**

#### 11.2 Biological Linkage via Omics Data (RR-1)

**Findings:**

#### 11.3 Pattern and Relationship Insights

**STL and FFT Observations:**

**Correlation and Causality Analysis:**

#### 11.4 Dashboard Usability

#### 11.5 Key Findings and Future Applications

**Future Work Recommendations:**

### 11.6 Descriptive Statistical Summary

#### Telemetry Variables (RR-1)

#### Radiation Variables (RR-1)


### 11.7 Prescriptive Statistical Insights

#### Environmental Monitoring Recommendations

#### Radiation-Aware Activity Scheduling

#### Sampling and Health Monitoring Triggers

#### Onboard Predictive Infrastructure

#### Habitat Design and Sensor Deployment

#### Recommendations for Artemis and Future Missions

---

### Section 12: Conclusion and Lessons Learned

#### Summary of Insights

#### Limitations

#### Project Challenges and Solutions

#### Recommendations and Next Steps


# End of code